In [2]:
import sys, os, math, string, datetime as dt, time, re
import pandas as pd, numpy as np, matplotlib.pyplot as plt, sqlite3 as sql
import emosent, emoji, string
import yfinance as yf

In [3]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk import sentiment
from nltk import RegexpParser
from nltk.help import upenn_tagset

In [4]:
import csv, sys
import requests
from api_key import cryptor

In [4]:
tree = os.walk("../data/raw")

In [5]:
for folders, subfolders, files in tree:
    print(folders)
    print('--')
    print(subfolders)
    print('----')
    print(files)
    print("\n")

../data/raw
--
['BullTrades', 'Learn-n', 'Legacy Trading', 'LSV', 'Misc', 'OmegaTrades', 'Pennybois', 'Phantom Trades', 'Templar Trading', 'WayTrade']
----
['.gitkeep', 'intrinsicValueCalculator.xlsx', 'text2.db', 'text3.db', 'word_test.csv', '~$intrinsicValueCalculator.xlsx']


../data/raw\BullTrades
--
[]
----
['options-chat.db', 'stocks-chat.db']


../data/raw\Learn-n
--
[]
----
['trade-chat.db']


../data/raw\Legacy Trading
--
[]
----
['general.db', 'profits.db', 'text.db', 'trading-discussion.db']


../data/raw\LSV
--
[]
----
['options.db', 'trading.db']


../data/raw\Misc
--
[]
----
['stock-chat.db', 'text.db', 'text2.db', 'text3.db', 'text4.db', 'text5.db', 'text6.db', 'trade-ideas.db', 'wetlqd-ideas.db']


../data/raw\OmegaTrades
--
[]
----
['general.db', 'potentialdupe.db', 'sentiment.db', 'tech.db', 'text.db', 'winning.db']


../data/raw\Pennybois
--
[]
----
['stock-check.db']


../data/raw\Phantom Trades
--
[]
----
['lt-investing.db', 'trading-floor.db']


../data/raw\Templa

In [6]:
def getalldata(folder):
    master_df = pd.DataFrame(columns=["id", "name", "content", "timestamp"])
    tree = os.walk(folder)
    for folders, subfolder, files in tree:
        if folders == "../data/raw":
            next
        else:
            
            for file in files: # get database file
                connection_str = folders.replace("\\", "/") +"/" + file #setup connection string
                connect = sql.connect(connection_str)
                mycur = connect.cursor()
                mycur.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name") #get available tables
                available_tables = mycur.fetchall()
                server_name = folders.replace("../data/raw\\", "") #server name to append to df
                channel_name = file.replace(".db", "") # channel name to append to df
                for i, tbl in enumerate(available_tables):
                    df = pd.read_sql(f"SELECT * FROM {tbl[0]}", con=connect)
                    df["channel"] = channel_name
                    df["server"] = server_name
                    master_df = pd.concat([master_df, df], axis=0, ignore_index=True)
                connect.close()
                print(f"processed {channel_name}. Beep Bop Boop.....")
    return master_df

In [7]:
total_df = getalldata("../data/raw")

processed options-chat. Beep Bop Boop.....
processed stocks-chat. Beep Bop Boop.....
processed trade-chat. Beep Bop Boop.....
processed general. Beep Bop Boop.....
processed profits. Beep Bop Boop.....
processed text. Beep Bop Boop.....
processed trading-discussion. Beep Bop Boop.....
processed options. Beep Bop Boop.....
processed trading. Beep Bop Boop.....
processed stock-chat. Beep Bop Boop.....
processed text. Beep Bop Boop.....
processed text2. Beep Bop Boop.....
processed text3. Beep Bop Boop.....
processed text4. Beep Bop Boop.....
processed text5. Beep Bop Boop.....
processed text6. Beep Bop Boop.....
processed trade-ideas. Beep Bop Boop.....
processed wetlqd-ideas. Beep Bop Boop.....
processed general. Beep Bop Boop.....
processed potentialdupe. Beep Bop Boop.....
processed sentiment. Beep Bop Boop.....
processed tech. Beep Bop Boop.....
processed text. Beep Bop Boop.....
processed winning. Beep Bop Boop.....
processed stock-check. Beep Bop Boop.....
processed lt-investing. B

In [8]:
total_df.server.unique()

array(['BullTrades', 'Learn-n', 'Legacy Trading', 'LSV', 'Misc',
       'OmegaTrades', 'Pennybois', 'Phantom Trades', 'Templar Trading',
       'WayTrade'], dtype=object)

In [9]:
total_df.channel.unique()

array(['options-chat', 'stocks-chat', 'trade-chat', 'general', 'profits',
       'text', 'trading-discussion', 'options', 'trading', 'stock-chat',
       'text2', 'text3', 'text4', 'text5', 'text6', 'trade-ideas',
       'wetlqd-ideas', 'potentialdupe', 'sentiment', 'tech', 'winning',
       'stock-check', 'lt-investing', 'trading-floor', 'day-trading',
       'main-trading', 'option-trading', 'general-trading'], dtype=object)

In [10]:
total_df.server = total_df.server.apply(lambda x: x if x != "Misc" else "misc")

In [11]:
total_df = total_df.sort_values(["server", "channel"])

In [12]:
total_df.drop_duplicates(subset=["id", "content"])[lambda x: (x.server == "misc")].channel.value_counts()

text6    4905
text       38
Name: channel, dtype: int64

## Dealing with Duplicate Dbs in /Misc

In [13]:
# stock-chat.db = Legacy Trading/profits.db = text3.db
total_df[lambda x: x.channel == "stock-chat"].drop_duplicates(subset=["id", "content"]).sort_values("timestamp")

,id,name,content,timestamp,channel,server
59572,187336630749757440,Cyborg#8226,Damn nice,2020-10-14T02:30:06.383000+00:00,stock-chat,misc
59567,187336630749757440,Cyborg#8226,What’d you pay for it,2020-10-14T02:30:29.588000+00:00,stock-chat,misc
59563,290014935499538435,toan112#9456,FB looks good,2020-10-14T02:30:32.627000+00:00,stock-chat,misc
59559,290014935499538435,toan112#9456,2.55 as it said,2020-10-14T02:30:42.804000+00:00,stock-chat,misc
59555,719919449863356430,bernie#3240,510 for both contracts,2020-10-14T02:30:46.699000+00:00,stock-chat,misc
...,...,...,...,...,...,...
57272,700394878462328832,Jaime#0289,AMD is wedging and this supply of confluent wi...,2020-12-08T15:18:52.685000+00:00,stock-chat,misc
57268,700394878462328832,Jaime#0289,/NQ at a fresh 4h demands hopefully we can see...,2020-12-08T15:21:21.489000+00:00,stock-chat,misc
57265,700394878462328832,Jaime#0289,/ES and /NQ are flying. No idea why equities a...,2020-12-08T16:03:05.514000+00:00,stock-chat,misc
57266,479056760255021056,.Austin#4801,I’ll be posting some juicy charting and more p...,2020-12-08T16:37:44.868000+00:00,stock-chat,misc


In [14]:
(total_df[lambda x: (x.channel == "profits")&(x.server == "Legacy Trading")]).drop_duplicates(subset=["id", "content"]).sort_values("timestamp")

,id,name,content,timestamp,channel,server
41234,187336630749757440,Cyborg#8226,Damn nice,2020-10-14T02:30:06.383000+00:00,profits,Legacy Trading
41229,187336630749757440,Cyborg#8226,What’d you pay for it,2020-10-14T02:30:29.588000+00:00,profits,Legacy Trading
41225,290014935499538435,toan112#9456,FB looks good,2020-10-14T02:30:32.627000+00:00,profits,Legacy Trading
41221,290014935499538435,toan112#9456,2.55 as it said,2020-10-14T02:30:42.804000+00:00,profits,Legacy Trading
41217,719919449863356430,bernie#3240,510 for both contracts,2020-10-14T02:30:46.699000+00:00,profits,Legacy Trading
...,...,...,...,...,...,...
38934,700394878462328832,Jaime#0289,AMD is wedging and this supply of confluent wi...,2020-12-08T15:18:52.685000+00:00,profits,Legacy Trading
38930,700394878462328832,Jaime#0289,/NQ at a fresh 4h demands hopefully we can see...,2020-12-08T15:21:21.489000+00:00,profits,Legacy Trading
38927,700394878462328832,Jaime#0289,/ES and /NQ are flying. No idea why equities a...,2020-12-08T16:03:05.514000+00:00,profits,Legacy Trading
38928,479056760255021056,.Austin#4801,I’ll be posting some juicy charting and more p...,2020-12-08T16:37:44.868000+00:00,profits,Legacy Trading


In [15]:
# verdict: text3.db = stock-chat = Legacy Trading/profits
total_df[lambda x: x.channel == "text3"].drop_duplicates(subset=["id", "content"]).sort_values("timestamp")

,id,name,content,timestamp,channel,server
66444,187336630749757440,Cyborg#8226,Damn nice,2020-10-14T02:30:06.383000+00:00,text3,misc
66439,187336630749757440,Cyborg#8226,What’d you pay for it,2020-10-14T02:30:29.588000+00:00,text3,misc
66435,290014935499538435,toan112#9456,FB looks good,2020-10-14T02:30:32.627000+00:00,text3,misc
66431,290014935499538435,toan112#9456,2.55 as it said,2020-10-14T02:30:42.804000+00:00,text3,misc
66427,719919449863356430,bernie#3240,510 for both contracts,2020-10-14T02:30:46.699000+00:00,text3,misc
...,...,...,...,...,...,...
64144,700394878462328832,Jaime#0289,AMD is wedging and this supply of confluent wi...,2020-12-08T15:18:52.685000+00:00,text3,misc
64140,700394878462328832,Jaime#0289,/NQ at a fresh 4h demands hopefully we can see...,2020-12-08T15:21:21.489000+00:00,text3,misc
64137,700394878462328832,Jaime#0289,/ES and /NQ are flying. No idea why equities a...,2020-12-08T16:03:05.514000+00:00,text3,misc
64138,479056760255021056,.Austin#4801,I’ll be posting some juicy charting and more p...,2020-12-08T16:37:44.868000+00:00,text3,misc


<h3 style="color:red">verdict: delete text3.db and stock-chat.db from /Misc/</h3>

In [16]:
# OmegaTrades/potentialdupe = misc/trade-ideas.db = misc/text5.db 
total_df[lambda x: x.channel == "potentialdupe"].drop_duplicates(subset=["id", "content"]).sort_values("timestamp")

,id,name,content,timestamp,channel,server
159074,673279340326420549,wes10#3165,https://twitter.com/realdonaldtrump/status/131...,2020-10-06T18:54:07.769000+00:00,potentialdupe,OmegaTrades
159068,673279340326420549,wes10#3165,https://www.bloomberg.com/amp/news/articles/20...,2020-11-05T01:54:47.610000+00:00,potentialdupe,OmegaTrades
159069,548320800202293258,SargeMaximus#4491,So why are stocks mooning with this election bs?,2020-11-06T03:02:40.575000+00:00,potentialdupe,OmegaTrades
159070,375968444803186698,kwatson20#2892,> So why are stocks mooning with this election...,2020-11-06T04:03:00.627000+00:00,potentialdupe,OmegaTrades
159063,625730781683122218,TraderMatt#9564,Biden’s lead is good for big tech,2020-11-06T04:03:46.984000+00:00,potentialdupe,OmegaTrades
...,...,...,...,...,...,...
158558,451481789110681631,Zachary1022#1017,holy shit read that yall,2020-12-04T21:46:05.806000+00:00,potentialdupe,OmegaTrades
158554,548320800202293258,SargeMaximus#4491,"Crazy, why do you think whoever it was sold?",2020-12-05T03:22:51.893000+00:00,potentialdupe,OmegaTrades
158551,451481789110681631,Zachary1022#1017,<@!548320800202293258> bc they think the marke...,2020-12-05T03:23:12.691000+00:00,potentialdupe,OmegaTrades
158552,451481789110681631,Zachary1022#1017,"nah but idk, they always sell before we know,t...",2020-12-05T03:24:23.736000+00:00,potentialdupe,OmegaTrades


In [17]:
# the trade ideas databate contains more unique comments
total_df[lambda x: x.channel == "trade-ideas"].drop_duplicates(subset=["id", "content"]).sort_values("timestamp")

,id,name,content,timestamp,channel,server
110841,673279340326420549,wes10#3165,https://twitter.com/realdonaldtrump/status/131...,2020-10-06T18:54:07.769000+00:00,trade-ideas,misc
111887,673279340326420549,wes10#3165,GME halted,2020-10-08T18:07:49.653000+00:00,trade-ideas,misc
111878,673279340326420549,wes10#3165,https://www.google.com/amp/s/www.bloomberg.com...,2020-10-10T20:10:08.122000+00:00,trade-ideas,misc
111873,673279340326420549,wes10#3165,Waymo is an autonomous driving tech company un...,2020-10-10T20:11:13.856000+00:00,trade-ideas,misc
111868,673279340326420549,wes10#3165,https://t.co/u4ZbtW9JX5?amp=1 \n\n\nThe Compan...,2020-10-14T19:48:17.976000+00:00,trade-ideas,misc
...,...,...,...,...,...,...
110325,451481789110681631,Zachary1022#1017,holy shit read that yall,2020-12-04T21:46:05.806000+00:00,trade-ideas,misc
110321,548320800202293258,SargeMaximus#4491,"Crazy, why do you think whoever it was sold?",2020-12-05T03:22:51.893000+00:00,trade-ideas,misc
110318,451481789110681631,Zachary1022#1017,<@!548320800202293258> bc they think the marke...,2020-12-05T03:23:12.691000+00:00,trade-ideas,misc
110319,451481789110681631,Zachary1022#1017,"nah but idk, they always sell before we know,t...",2020-12-05T03:24:23.736000+00:00,trade-ideas,misc


In [18]:
# trade-ideas = text5.db 
total_df[lambda x: (x.channel == "text5") & (x.server == "misc")].drop_duplicates(subset=["id", "content"]).sort_values("timestamp")

,id,name,content,timestamp,channel,server
71008,673279340326420549,wes10#3165,https://twitter.com/realdonaldtrump/status/131...,2020-10-06T18:54:07.769000+00:00,text5,misc
72054,673279340326420549,wes10#3165,GME halted,2020-10-08T18:07:49.653000+00:00,text5,misc
72045,673279340326420549,wes10#3165,https://www.google.com/amp/s/www.bloomberg.com...,2020-10-10T20:10:08.122000+00:00,text5,misc
72040,673279340326420549,wes10#3165,Waymo is an autonomous driving tech company un...,2020-10-10T20:11:13.856000+00:00,text5,misc
72035,673279340326420549,wes10#3165,https://t.co/u4ZbtW9JX5?amp=1 \n\n\nThe Compan...,2020-10-14T19:48:17.976000+00:00,text5,misc
...,...,...,...,...,...,...
70492,451481789110681631,Zachary1022#1017,holy shit read that yall,2020-12-04T21:46:05.806000+00:00,text5,misc
70488,548320800202293258,SargeMaximus#4491,"Crazy, why do you think whoever it was sold?",2020-12-05T03:22:51.893000+00:00,text5,misc
70485,451481789110681631,Zachary1022#1017,<@!548320800202293258> bc they think the marke...,2020-12-05T03:23:12.691000+00:00,text5,misc
70486,451481789110681631,Zachary1022#1017,"nah but idk, they always sell before we know,t...",2020-12-05T03:24:23.736000+00:00,text5,misc


In [19]:
# OmegaTrades/potentialdupe = partial duplicate of misc/trade-ideas.db = misc/text5.db that are partial duplicates of text.db
total_df[lambda x: (x.channel == "text") & (x.server == "misc")].drop_duplicates(subset=["id", "content"]).sort_values("timestamp")

,id,name,content,timestamp,channel,server
60604,673279340326420549,wes10#3165,https://twitter.com/realdonaldtrump/status/131...,2020-10-06T18:54:07.769000+00:00,text,misc
60598,673279340326420549,wes10#3165,https://www.bloomberg.com/amp/news/articles/20...,2020-11-05T01:54:47.610000+00:00,text,misc
60599,548320800202293258,SargeMaximus#4491,So why are stocks mooning with this election bs?,2020-11-06T03:02:40.575000+00:00,text,misc
60600,375968444803186698,kwatson20#2892,> So why are stocks mooning with this election...,2020-11-06T04:03:00.627000+00:00,text,misc
60593,625730781683122218,TraderMatt#9564,Biden’s lead is good for big tech,2020-11-06T04:03:46.984000+00:00,text,misc
...,...,...,...,...,...,...
62112,374316215797284877,! - ƓŎ ƦᗴŞᚡĄᙡŊ - !#4955,FSLY,2020-12-08T15:35:58.628000+00:00,text,misc
62105,374316215797284877,! - ƓŎ ƦᗴŞᚡĄᙡŊ - !#4955,HYLN,2020-12-08T15:40:38.188000+00:00,text,misc
62102,374316215797284877,! - ƓŎ ƦᗴŞᚡĄᙡŊ - !#4955,Cheap leap that will guarantee a profit. IVR r...,2020-12-08T16:10:49.635000+00:00,text,misc
62100,374316215797284877,! - ƓŎ ƦᗴŞᚡĄᙡŊ - !#4955,Very bullish. Broke out and held it's place ab...,2020-12-08T16:24:27.489000+00:00,text,misc


<h3 style="color:red">verdict: delete OmegaTrades/potentialdupe.db, Misc/text5.db and ./trade-ideas.db + rename: text.db to trade-ideas and mv to /OmegaTrades</h3>

In [20]:
# Omega Trades/winning = misc/text2.db
total_df[lambda x: (x.channel == "text2") & (x.server == "misc")].drop_duplicates(subset=["id", "content"])

,id,name,content,timestamp,channel,server
63613,271708801609105408,dylskrrt#0864,another good call,2020-12-08T15:18:53.907000+00:00,text2,misc
63614,673279340326420549,wes10#3165,PLTR this morning and a DKNG daytrade,2020-12-08T16:35:06.936000+00:00,text2,misc
63617,673279340326420549,wes10#3165,Quick trade on DDOG,2020-12-04T16:32:40.061000+00:00,text2,misc
63620,281194229676376065,A_Lucky_Pucker#8227,200 scalping DOCU\nLots of other trades that u...,2020-12-04T16:11:53.140000+00:00,text2,misc
63623,673279340326420549,wes10#3165,Snap lottos 🙂,2020-12-03T16:58:01.890000+00:00,text2,misc
...,...,...,...,...,...,...
64107,281194229676376065,A_Lucky_Pucker#8227,250 on AMD quick scalp play this morning just ...,2020-10-26T13:39:52.625000+00:00,text2,misc
64117,281194229676376065,A_Lucky_Pucker#8227,<@!673279340326420549> Thank you sir ^^,2020-10-21T22:38:42.152000+00:00,text2,misc
64122,281194229676376065,A_Lucky_Pucker#8227,$60.00 profit. \nBut bad risk to reward ratio\...,2020-10-21T15:23:17.722000+00:00,text2,misc
64124,281194229676376065,A_Lucky_Pucker#8227,Entered too early then exited to early,2020-10-21T14:58:06.478000+00:00,text2,misc


In [21]:
# Winning has a few more comments
(total_df[lambda x: (x.channel == "winning")&(x.server == "OmegaTrades")]).drop_duplicates(subset=["id", "content"])

,id,name,content,timestamp,channel,server
167637,271708801609105408,dylskrrt#0864,another good call,2020-12-08T15:18:53.907000+00:00,winning,OmegaTrades
167638,673279340326420549,wes10#3165,PLTR this morning and a DKNG daytrade,2020-12-08T16:35:06.936000+00:00,winning,OmegaTrades
167641,673279340326420549,wes10#3165,Quick trade on DDOG,2020-12-04T16:32:40.061000+00:00,winning,OmegaTrades
167644,281194229676376065,A_Lucky_Pucker#8227,200 scalping DOCU\nLots of other trades that u...,2020-12-04T16:11:53.140000+00:00,winning,OmegaTrades
167647,673279340326420549,wes10#3165,Snap lottos 🙂,2020-12-03T16:58:01.890000+00:00,winning,OmegaTrades
...,...,...,...,...,...,...
168181,673279340326420549,wes10#3165,Let’s go hippo base hits not home runs,2020-10-14T14:49:11.936000+00:00,winning,OmegaTrades
168183,531986968091951105,Hippo#3098,It’s not much but I’m starting to get the ball...,2020-10-14T14:48:51.657000+00:00,winning,OmegaTrades
168189,531986968091951105,Hippo#3098,Apple 116-123,2020-10-13T15:48:46.812000+00:00,winning,OmegaTrades
168192,281194229676376065,A_Lucky_Pucker#8227,512.90 x 80\n513.19 x 79,2020-10-13T15:47:09.237000+00:00,winning,OmegaTrades


<h3 style="color:red">verdict: delete Misc/text2.db</h3>

In [22]:
# text.db = text4.db
total_df[lambda x: (x.channel == "text") & (x.server == "misc")].drop_duplicates(subset=["id", "content"])

,id,name,content,timestamp,channel,server
59577,451481789110681631,Zachary1022#1017,<@!548320800202293258> bc they think the marke...,2020-12-05T03:23:12.691000+00:00,text,misc
59578,451481789110681631,Zachary1022#1017,"nah but idk, they always sell before we know,t...",2020-12-05T03:24:23.736000+00:00,text,misc
59579,548320800202293258,SargeMaximus#4491,Ah true,2020-12-05T03:24:26.764000+00:00,text,misc
59580,548320800202293258,SargeMaximus#4491,"Crazy, why do you think whoever it was sold?",2020-12-05T03:22:51.893000+00:00,text,misc
59584,451481789110681631,Zachary1022#1017,holy shit read that yall,2020-12-04T21:46:05.806000+00:00,text,misc
...,...,...,...,...,...,...
63588,374316215797284877,! - ƓŎ ƦᗴŞᚡĄᙡŊ - !#4955,AAPL: Wait for a break to either the upside or...,2020-11-21T22:29:21.835000+00:00,text,misc
63593,374316215797284877,! - ƓŎ ƦᗴŞᚡĄᙡŊ - !#4955,"MSFT: Trading Sideways, wait for break",2020-11-21T22:29:10.607000+00:00,text,misc
63598,159985870458322944,MEE6#4876,"GG <@!374316215797284877>, you just advanced t...",2020-11-21T22:29:00.484000+00:00,text,misc
63603,374316215797284877,! - ƓŎ ƦᗴŞᚡĄᙡŊ - !#4955,"Costco: Ascending triangle, watch for a break ...",2020-11-21T22:29:00.368000+00:00,text,misc


In [23]:
total_df[lambda x: (x.channel == "text4") & (x.server == "misc")].drop_duplicates(subset=["id", "content"])

,id,name,content,timestamp,channel,server
66449,451481789110681631,Zachary1022#1017,<@!548320800202293258> bc they think the marke...,2020-12-05T03:23:12.691000+00:00,text4,misc
66450,451481789110681631,Zachary1022#1017,"nah but idk, they always sell before we know,t...",2020-12-05T03:24:23.736000+00:00,text4,misc
66451,548320800202293258,SargeMaximus#4491,Ah true,2020-12-05T03:24:26.764000+00:00,text4,misc
66452,548320800202293258,SargeMaximus#4491,"Crazy, why do you think whoever it was sold?",2020-12-05T03:22:51.893000+00:00,text4,misc
66456,451481789110681631,Zachary1022#1017,holy shit read that yall,2020-12-04T21:46:05.806000+00:00,text4,misc
...,...,...,...,...,...,...
70460,374316215797284877,! - ƓŎ ƦᗴŞᚡĄᙡŊ - !#4955,AAPL: Wait for a break to either the upside or...,2020-11-21T22:29:21.835000+00:00,text4,misc
70465,374316215797284877,! - ƓŎ ƦᗴŞᚡĄᙡŊ - !#4955,"MSFT: Trading Sideways, wait for break",2020-11-21T22:29:10.607000+00:00,text4,misc
70470,159985870458322944,MEE6#4876,"GG <@!374316215797284877>, you just advanced t...",2020-11-21T22:29:00.484000+00:00,text4,misc
70475,374316215797284877,! - ƓŎ ƦᗴŞᚡĄᙡŊ - !#4955,"Costco: Ascending triangle, watch for a break ...",2020-11-21T22:29:00.368000+00:00,text4,misc


<h3 style="color:red">remove Misc/text4.db</h3>

In [24]:
# wetlqd-ideas = text6.db
total_df[lambda x: (x.channel == "text6") & (x.server == "misc")].drop_duplicates(subset=["id", "content"])

,id,name,content,timestamp,channel,server
72058,488359813160501251,Merente#5844,Thanks <@464449226521313292> ! I opted for th...,2020-11-30T20:48:43.035000+00:00,text6,misc
72059,689681229989150742,jackaplan#7289,Barely followed the market at all today (didn’...,2020-11-30T20:58:32.872000+00:00,text6,misc
72060,612337363326074936,Trademate#2515,<@131952052069728258> has reached level **13**!,2020-11-30T19:55:20.913000+00:00,text6,misc
72063,131952052069728258,Teecee#3022,<@!464449226521313292> thanks boss,2020-11-30T19:55:20.625000+00:00,text6,misc
72067,464449226521313292,TradingJEDI#2647,<@!399198468159045633> BRO AWSOME SO GLAD YOU ...,2020-11-30T19:44:26.516000+00:00,text6,misc
...,...,...,...,...,...,...
110279,445583268792893449,itsyoboyvic#7198,Heroin,2019-08-02T01:14:44.367000+00:00,text6,misc
110284,445583268792893449,itsyoboyvic#7198,Big crack,2019-08-02T01:14:39.682000+00:00,text6,misc
110289,445583268792893449,itsyoboyvic#7198,Crack,2019-08-02T01:14:35.798000+00:00,text6,misc
110294,579205396166737921,beenpippinsincebeenpippin#4039,@everyone today’s total. This doesn’t calculat...,2019-08-02T01:13:47.923000+00:00,text6,misc


In [25]:
total_df[lambda x: (x.channel == "wetlqd-ideas") & (x.server == "misc")].drop_duplicates(subset=["id", "content"])

,id,name,content,timestamp,channel,server
111891,488359813160501251,Merente#5844,Thanks <@464449226521313292> ! I opted for th...,2020-11-30T20:48:43.035000+00:00,wetlqd-ideas,misc
111892,689681229989150742,jackaplan#7289,Barely followed the market at all today (didn’...,2020-11-30T20:58:32.872000+00:00,wetlqd-ideas,misc
111893,612337363326074936,Trademate#2515,<@131952052069728258> has reached level **13**!,2020-11-30T19:55:20.913000+00:00,wetlqd-ideas,misc
111896,131952052069728258,Teecee#3022,<@!464449226521313292> thanks boss,2020-11-30T19:55:20.625000+00:00,wetlqd-ideas,misc
111900,464449226521313292,TradingJEDI#2647,<@!399198468159045633> BRO AWSOME SO GLAD YOU ...,2020-11-30T19:44:26.516000+00:00,wetlqd-ideas,misc
...,...,...,...,...,...,...
150112,445583268792893449,itsyoboyvic#7198,Heroin,2019-08-02T01:14:44.367000+00:00,wetlqd-ideas,misc
150117,445583268792893449,itsyoboyvic#7198,Big crack,2019-08-02T01:14:39.682000+00:00,wetlqd-ideas,misc
150122,445583268792893449,itsyoboyvic#7198,Crack,2019-08-02T01:14:35.798000+00:00,wetlqd-ideas,misc
150127,579205396166737921,beenpippinsincebeenpippin#4039,@everyone today’s total. This doesn’t calculat...,2019-08-02T01:13:47.923000+00:00,wetlqd-ideas,misc


<h3 style="color:red">remove Misc/text6.db</h3>

<h3>/raw/data Cleanup</h3>
<ul>
<li>rm: text6, text5, text4, text3, text2, stock-chat, trade-ideas, OmegaTrades/potentialDup</li>
<li>rename: misc/text.db -> Omega Trades/trade-ideas.db</li>
</ul>

In [34]:
total_df_no_misc = getalldata("../data/raw")

processed options-chat. Beep Bop Boop.....
processed stocks-chat. Beep Bop Boop.....
processed trade-chat. Beep Bop Boop.....
processed general. Beep Bop Boop.....
processed profits. Beep Bop Boop.....
processed text. Beep Bop Boop.....
processed trading-discussion. Beep Bop Boop.....
processed options. Beep Bop Boop.....
processed trading. Beep Bop Boop.....
processed wetlqd-ideas. Beep Bop Boop.....
processed general. Beep Bop Boop.....
processed sentiment. Beep Bop Boop.....
processed tech. Beep Bop Boop.....
processed text. Beep Bop Boop.....
processed trade-ideas. Beep Bop Boop.....
processed winning. Beep Bop Boop.....
processed stock-check. Beep Bop Boop.....
processed lt-investing. Beep Bop Boop.....
processed trading-floor. Beep Bop Boop.....
processed day-trading. Beep Bop Boop.....
processed main-trading. Beep Bop Boop.....
processed option-trading. Beep Bop Boop.....
processed general-trading. Beep Bop Boop.....


In [35]:
total_df_no_misc = total_df_no_misc.drop_duplicates(["id", "content"])

In [36]:
total_df_no_misc.channel.unique()

array(['options-chat', 'stocks-chat', 'trade-chat', 'general', 'profits',
       'text', 'trading-discussion', 'options', 'trading', 'wetlqd-ideas',
       'sentiment', 'tech', 'trade-ideas', 'winning', 'stock-check',
       'lt-investing', 'trading-floor', 'day-trading', 'main-trading',
       'option-trading', 'general-trading'], dtype=object)

In [74]:
total_df_no_misc = total_df_no_misc.sort_values("timestamp").reset_index(drop=True)

In [75]:
total_df_no_misc.to_sql("comments", con="sqlite:///../data/interim/discord/discord.db", index_label="pk")

In [80]:
total_df_no_misc.shape

(31934, 6)

<h3>Comment Cleaning</h3>
    <ul>
        <li>Starting Point: 31,934 Unique Comments</li>
        <li>In order to get these comments ready for NLP analysis, I must:</li>
        <ul>
        <li>Remove scripted comments (those automatically genreated by bots)</li>
        <li>Identify Single Words Making Up Comments That Are Key to This Analysis:</li>
            <ul>
            <li>Emojis</li>
            <li>Specific Companies and Their Tickers</li>
            <li>Stock Trading Specific Slang</li>
            <li>User Mentions</li>
            </ul>
        </ul>
    </ul>

In [90]:
def catchBot(c):
    botCom = re.compile("\shas reached level \*\*.\d+\*\*")
    bot2Com = re.compile("\<.*\>, you just advanced to")
    botreg = botCom.search(c)
    botreg1 = bot2Com.search(c.strip())
    if (botreg != None) or (botreg1 != None):
        return 1
    else:
        return 0

#### Find Bots

In [91]:
total_df_no_misc = total_df_no_misc.assign(isBot = lambda x: x.content.apply(catchBot))

In [246]:
bots = total_df_no_misc[lambda x: x.isBot == 1].id.unique()

In [257]:
bots = np.concatenate([bots, total_df_no_misc[lambda x: x.name.str.contains("Bot#")].id.unique()], axis=0)

In [258]:
total_df_no_misc.loc[(total_df_no_misc.id.isin(bots)), "isBot"] = 1

#### Find Mentions

In [344]:
def catchMentions(comment):
    userIdCatcherT = r"(<@(!)?\d+>|@everyone)"
    mention = re.findall(userIdCatcherT, comment)
    if mention != []:
        return ",".join([re.sub(r"[<>!@]", "", x[i][0]) for i, x in enumerate([mention])])
    else:
        return ""

In [345]:
total_df = total_df_no_misc.assign(mentions = lambda x: x.content.apply(catchMentions))

#### Find Emojis

In [346]:
def catchEmojis(comment):
    emoji_list = emoji.emoji_lis(comment)
    return ",".join([e["emoji"] for e in emoji_list])

In [347]:
total_df = total_df.assign(emojis = lambda x: x.content.apply(catchEmojis))

In [348]:
total_df[lambda x: x.emojis != ""].head()

,id,name,content,timestamp,channel,server,isBot,mentions,emojis
12,408706621942792192,zeuse4848#9335,😃,2019-08-02T17:55:31.789000+00:00,wetlqd-ideas,Misc,0,,😃
41,570291052494323722,Moocah#1918,Griffin McDonald’s is all doodoo 😂😂 <@57920539...,2019-08-08T17:02:22.143000+00:00,wetlqd-ideas,Misc,0,579205396166737921,"😂,😂"
80,456226577798135808,Deleted User#0000,🔥🔥😳😳,2019-08-16T15:19:59.020000+00:00,wetlqd-ideas,Misc,0,,"🔥,🔥,😳,😳"
87,587888385197342732,ricky#8577,😂,2019-08-16T16:37:53.126000+00:00,wetlqd-ideas,Misc,0,,😂
97,611594621222387738,BreadGame#4233,Good morning everyone 😃,2019-08-19T13:34:26.860000+00:00,wetlqd-ideas,Misc,0,,😃


In [382]:
total_df.to_sql("comments", con="sqlite:///../data/interim/discord/discord.db", index_label="pk", if_exists="replace")

#### Find Companies
* find companies using name, ticker or associated words
    * This project goal is to match and process comment sentiment and match it with the particular companies mentioned.
    * Companies are signified by a variety of words including: their traded ticker abbreviation, their full company name and by the names of members of their C-Suite.
    * In order to accurately and systematically obtain these mentioned companies, I will have to create a simple list of companies based on single words (ticker and name).
    * Once I have an understanding of the most popular companies being talked about in the groups, I can create a more refined parsing function to identify words that most likely trace back to the company.
    * I want to maintain the structure of the comments throughout the analysis (maintain initial structure), so that my NLP analysis can grade sentiment based on the original comment's grammer and word usage

In [349]:
# Discords link to stock starts with a $, but can only pull out 338 comments out of 30,000+. 
total_df[lambda x: x.content.str.contains(r"\$[A-Za-z]")]

,id,name,content,timestamp,channel,server,isBot,mentions,emojis
246,367506726582484996,JK#0643,"Yea, literally all thanks to you guys. Just wa...",2019-08-29T21:06:45.315000+00:00,wetlqd-ideas,Misc,0,,
610,445583268792893449,itsyoboyvic#7198,$EXAS,2019-10-02T19:12:03.224000+00:00,wetlqd-ideas,Misc,0,,
1372,449633258116874250,dakotared#3395,quick scalp on $SCWX with Dell buying out the ...,2019-12-23T18:48:28.342000+00:00,wetlqd-ideas,Misc,0,,
1921,674648888279302204,GOOFYSPY#8332,$IRM got small position. Will hold it overnight.,2020-02-25T20:58:08.310000+00:00,wetlqd-ideas,Misc,0,,
1930,674648888279302204,GOOFYSPY#8332,"$Dis, interesting 🤔",2020-02-26T17:04:51.441000+00:00,wetlqd-ideas,Misc,0,,🤔
...,...,...,...,...,...,...,...,...,...
31855,396160546140979212,jmorri#6551,$GRIL again today <@!101806058359496704>,2020-12-08T20:13:38.815000+00:00,main-trading,Templar Trading,0,101806058359496704,
31856,101806058359496704,Ryan (itsGoggles)#8580,$CANG up too <:peepoyay:735222523683405984>,2020-12-08T20:13:38.984000+00:00,main-trading,Templar Trading,0,,
31858,396160546140979212,jmorri#6551,$AESE for anyone that was playing it just went...,2020-12-08T20:13:53.474000+00:00,main-trading,Templar Trading,0,,
31889,101806058359496704,Ryan (itsGoggles)#8580,$RUN i had on a watchlist like last week but t...,2020-12-08T20:20:47.630000+00:00,option-trading,Templar Trading,0,,


In [19]:
company_regex = re.compile(r"[A-Z]{2,4}")

In [380]:
# identifying companies based on tickers. Better representative. 338 -> 6,438 
total_df[lambda x:(x.content.str.contains(company_regex))]

,id,name,content,timestamp,channel,server,isBot,mentions,emojis
23,445583268792893449,itsyoboyvic#7198,I took stamps .... and IAC both not looking go...,2019-08-08T03:31:33.793000+00:00,wetlqd-ideas,Misc,0,,
26,194305752939102208,D.Ninan#3776,Sold AMD aug23rd call for a fat profit,2019-08-08T13:33:00.968000+00:00,wetlqd-ideas,Misc,0,,
47,194305752939102208,D.Ninan#3776,MSFT profits im still not selling it tho just ...,2019-08-13T14:21:58.192000+00:00,wetlqd-ideas,Misc,0,,
54,344157708523667457,dylan.patel#4692,$139.50 and ATH,2019-08-13T19:01:16.234000+00:00,wetlqd-ideas,Misc,0,,
56,194305752939102208,D.Ninan#3776,$139.50 and ATH? Wym,2019-08-13T19:05:27.213000+00:00,wetlqd-ideas,Misc,0,,
...,...,...,...,...,...,...,...,...,...
31910,428921853403463713,Baraxton#0199,@everyone EDIT is popping off,2020-12-08T20:25:14.542000+00:00,trading,LSV,0,everyone,
31916,785007528152399913,nickb34#9557,$OXY broke the resistance line on Brad's chart,2020-12-08T20:26:51.453000+00:00,trading,LSV,0,,
31919,594593113104842795,VelcroKarma#1526,if PIXY runs from being included in the Benzin...,2020-12-08T20:27:59.342000+00:00,trade-chat,Learn-n,0,,
31930,184420431242067977,Dg#8853,<@428921853403463713> have you done a video in...,2020-12-08T20:39:17.246000+00:00,trading,LSV,0,428921853403463713,


In [4]:
# next to do:
    # gather list of companies on the four American exchanges: NYSE and NASDAQ and their tickers (available via NASDAQ's website)
    # have dictionary parser that translates company name mentions to its assocaited ticker.
    # with the name and tickers, a more thorough pulling of the appropiate companies can take place. The pulling should take place on the str.lower value of the content column.
    # company mentions will be stored just as emojis and mentions are stored for now. (as comma-separated values column)

In [4]:
with sql.connect("../data/interim/discord/discord.db") as con:
    total_df = pd.read_sql("SELECT id, name, content, timestamp, channel, server, isBot, mentions, emojis FROM comments", con=con)

In [124]:
tickers_df = pd.DataFrame()
for fold, _, files in os.walk("../data/external/stock_data/"):
    for f in files:
        if f.startswith("nasdaq_"):
            _temp_df = pd.read_csv(fold+"/"+f)
            _temp_df = _temp_df.assign(exchange = f.split("_")[-1].replace(".csv", "").upper())
            tickers_df = pd.concat([tickers_df, _temp_df], ignore_index=True, axis=0)

In [125]:
tickers_df

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry,exchange
0,AAU,Almaden Minerals Ltd. Common Shares,$0.318,-0.0009,-0.282%,4.363641e+07,Canada,2015.0,196663,NaN,NaN,AMEX
1,ACU,Acme United Corporation. Common Stock,$31.47,-1.6500,-4.982%,1.120234e+08,United States,1988.0,5792,Consumer Durables,Home Furnishings,AMEX
2,ACY,AeroCentury Corp. Common Stock,$35.00,0.0100,0.029%,1.545884e+08,United States,NaN,36527,Technology,Diversified Commercial Services,AMEX
3,ADRA,Adara Acquisition Corp. Class A Common Stock,$9.80,0.0191,0.195%,1.408750e+08,United States,2021.0,401,NaN,NaN,AMEX
4,AE,Adams Resources & Energy Inc. Common Stock,$28.14,-0.4100,-1.436%,1.225497e+08,United States,NaN,1218,Energy,Oilfield Services/Equipment,AMEX
...,...,...,...,...,...,...,...,...,...,...,...,...
8183,ZTS,Zoetis Inc. Class A Common Stock,$227.85,2.1500,0.953%,1.078017e+11,United States,2013.0,641943,Health Care,Biotechnology: Pharmaceutical Preparations,NYSE
8184,ZUO,Zuora Inc. Class A Common Stock,$20.32,0.3500,1.753%,2.531872e+09,NaN,2018.0,190253,Technology,EDP Services,NYSE
8185,ZVIA,Zevia PBC Class A Common Stock,$8.50,0.6900,8.835%,2.928799e+08,NaN,2021.0,84708,Consumer Non-Durables,Specialty Foods,NYSE
8186,ZWS,Zurn Water Solutions Corporation Common Stock,$36.87,0.0500,0.136%,4.474157e+09,United States,2012.0,105372,Public Utilities,Water Supply,NYSE


In [126]:
tickers_df.dropna(subset=["Market Cap"]).sort_values(["Market Cap"], ascending=False)

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry,exchange
284,AAPL,Apple Inc. Common Stock,$174.225,3.0450,1.779%,3.020598e+12,United States,1980.0,67838373,Technology,Computer Manufacturing,NASDAQ
3171,MSFT,Microsoft Corporation Common Stock,$331.5383,-3.3817,-1.01%,2.489183e+12,United States,1986.0,11093825,Technology,Computer Software: Prepackaged Software,NASDAQ
2162,GOOG,Alphabet Inc. Class C Capital Stock,$2964.03,3.3000,0.111%,1.967416e+12,United States,2004.0,418187,Technology,Internet and Information Services,NASDAQ
2163,GOOGL,Alphabet Inc. Class A Common Stock,$2946.56,1.1700,0.04%,1.955820e+12,United States,NaN,448053,Technology,Internet and Information Services,NASDAQ
531,AMZN,Amazon.com Inc. Common Stock,$3519.10,-4.1900,-0.119%,1.784703e+12,United States,1997.0,1220708,Consumer Services,Catalog/Specialty Distribution,NASDAQ
...,...,...,...,...,...,...,...,...,...,...,...,...
4942,XFINU,ExcelFin Acquisition Corp Unit,$10.05,0.0000,0.00%,0.000000e+00,United States,2021.0,1,NaN,NaN,NASDAQ
2316,HIIIW,Hudson Executive Investment Corp. III Warrant,$0.77,-0.0300,-3.75%,0.000000e+00,United States,2021.0,1275,NaN,NaN,NASDAQ
4952,XOMAO,XOMA Corporation Depositary Shares Rep Series ...,$26.25,-0.1500,-0.568%,0.000000e+00,United States,NaN,5488,Health Care,Biotechnology: Pharmaceutical Preparations,NASDAQ
4953,XOMAP,XOMA Corporation 8.625% Series A Cumulative Pe...,$26.35,-0.0500,-0.189%,0.000000e+00,United States,NaN,140,Health Care,Biotechnology: Pharmaceutical Preparations,NASDAQ


In [127]:
tickers_df["Market Cap"].value_counts()

0.000000e+00    1245
4.209000e+08       4
2.808875e+08       4
3.363750e+08       3
2.823250e+08       3
                ... 
2.670478e+08       1
2.729275e+09       1
5.984373e+08       1
3.534605e+09       1
1.907935e+08       1
Name: Market Cap, Length: 6435, dtype: int64

In [128]:
classifiable_companies = tickers_df.dropna(subset=["Market Cap"])[lambda x: x["Market Cap"] > 0].sort_values(["Market Cap"], ascending=False).reset_index(drop=True)

In [129]:
classifiable_companies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6470 entries, 0 to 6469
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Symbol      6470 non-null   object 
 1   Name        6470 non-null   object 
 2   Last Sale   6470 non-null   object 
 3   Net Change  6470 non-null   float64
 4   % Change    6469 non-null   object 
 5   Market Cap  6470 non-null   float64
 6   Country     5802 non-null   object 
 7   IPO Year    3879 non-null   float64
 8   Volume      6470 non-null   int64  
 9   Sector      5560 non-null   object 
 10  Industry    5575 non-null   object 
 11  exchange    6470 non-null   object 
dtypes: float64(3), int64(1), object(8)
memory usage: 606.7+ KB


In [12]:
def findCommonWords(stock_array):
    common_words = {}
    for stock in stock_array:
        for word in stock.lower().split():
            if word in common_words.keys():
                common_words[word] += 1
            else:
                common_words[word] = 1
        
    return common_words


In [13]:
company_words = findCommonWords(classifiable_companies.Name.values)

In [14]:
company_word = dict(sorted(company_words.items(), key=lambda x: x[1], reverse=True))

In [15]:
company_word

{'common': 5288,
 'stock': 4881,
 'inc.': 3333,
 'shares': 1227,
 'class': 1125,
 'a': 1101,
 'corporation': 791,
 'ordinary': 633,
 'corp.': 552,
 'holdings': 416,
 'acquisition': 395,
 'american': 371,
 'fund': 346,
 'group': 315,
 'depositary': 310,
 'income': 240,
 'ltd.': 236,
 'limited': 236,
 'therapeutics': 210,
 'company': 199,
 'of': 197,
 'trust': 185,
 'inc': 160,
 'beneficial': 152,
 'financial': 148,
 'interest': 147,
 '&': 145,
 'representing': 139,
 'technologies': 137,
 'energy': 129,
 'pharmaceuticals': 128,
 'capital': 127,
 'international': 127,
 'bancorp': 126,
 'share': 112,
 'global': 112,
 'plc': 105,
 'municipal': 96,
 'holding': 88,
 'ii': 84,
 'technology': 80,
 'partners': 80,
 '(the)': 78,
 'incorporated': 77,
 'and': 70,
 'corp': 70,
 's.a.': 69,
 'first': 69,
 'health': 67,
 'new': 66,
 'industries': 63,
 'each': 62,
 'investment': 61,
 'one': 59,
 'blackrock': 58,
 'services': 57,
 'co.': 56,
 'systems': 56,
 'units': 56,
 'nuveen': 55,
 'resources': 52,

In [16]:
legal_phrases_re = re.compile(r"common stock|class a|ltd.|limited|common shares|ordinary shares|class b|class c|class d|holdings|inc\.|capital stock|corporation|corp\.|company|incorporated|depositary shares|\(the\)|group holding|\(de\)")

In [17]:
classifiable_companies = classifiable_companies.assign(single_name = lambda x: x.Name.apply(lambda comp: re.sub(legal_phrases_re, "", comp.lower())))

In [18]:
tickers_and_names = classifiable_companies.loc[:, ["Symbol", "Name", "single_name"]]

In [19]:
tickers_and_names.head(20)

,Symbol,Name,single_name
0,AAPL,Apple Inc. Common Stock,apple
1,MSFT,Microsoft Corporation Common Stock,microsoft
2,GOOG,Alphabet Inc. Class C Capital Stock,alphabet
3,GOOGL,Alphabet Inc. Class A Common Stock,alphabet
4,AMZN,Amazon.com Inc. Common Stock,amazon.com
5,TSLA,Tesla Inc. Common Stock,tesla
6,FB,Meta Platforms Inc. Class A Common Stock,meta platforms
7,NVDA,NVIDIA Corporation Common Stock,nvidia
8,TSM,Taiwan Semiconductor Manufacturing Company Ltd.,taiwan semiconductor manufacturing
9,JPM,JP Morgan Chase & Co. Common Stock,jp morgan chase & co.


In [20]:
tickers_and_names.sample(20, random_state=11)

,Symbol,Name,single_name
3113,SGML,Sigma Lithium Corporation Common Shares,sigma lithium
2621,NBHC,National Bank Holdings Corporation Common Stock,national bank
1295,DCP,DCP Midstream LP Common Units,dcp midstream lp common units
2649,PTNR,Partner Communications Company Ltd. American D...,partner communications american
2521,NE,Noble Corporation plc Ordinary Shares,noble plc
1490,TLRY,Tilray Inc. Class 2 Common Stock,tilray class 2
53,MS,Morgan Stanley Common Stock,morgan stanley
5954,OPGN,OpGen Inc. Common Stock,opgen
1787,COOP,Mr. Cooper Group Inc. Common Stock,mr. cooper group
6166,ESP,Espey Mfg. & Electronics Corp. Common Stock,espey mfg. & electronics


In [21]:
stop_words = set(stopwords.words("english"))

In [22]:
# understand basic linguistic structure of comments. Words used. To see how I should approach picking out companies and analyzing sentiment


In [23]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [24]:
re.sub(r"[\’\'\'\"\"\“\”!\?@#$%&\(\)\*,-.\\\{\}+~\/:;<>\[\]^`|=_]", "", "’”!#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~")

''

In [37]:
def wordCounts(comment_array):
    comment_dictionary = {} 
    for comment in comment_array:
        punc_regex = r"[\’\'\'\"\"\“\”!\?@#$%&\(\)\*,-.\\\{\}+~\/:;<>\[\]^`|=_]"
        comment = comment.lower()
        comment = re.sub(punc_regex, "", comment)
        comment = re.sub(r"[0-9]", "", comment)
        emoji_list = emoji.emoji_lis(comment)
        emoji_list = [e["emoji"] for e in emoji_list]
        words = word_tokenize(comment)
        for w in words:
            if (w in comment_dictionary.keys()) & (w not in stop_words) & (w not in emoji_list):
                comment_dictionary[w] += 1
            elif (w not in stop_words) & (w not in emoji_list):
                comment_dictionary[w] = 1
    comments_words = dict(sorted(comment_dictionary.items(), key=lambda x: x[1], reverse=True))
    
    return pd.DataFrame.from_dict(comments_words, orient="index", columns=["count"]).reset_index().rename(columns={"index": "word"})

In [26]:
wordy_df = wordCounts(total_df.content.values)

In [27]:
wordy_df = wordy_df.assign(no_vow = lambda x: x.word.apply(lambda x: 1 if re.findall(r"[aeiou]+", x) != [] else 0))

In [28]:
total_df[lambda x: x.content.str.contains("nflx")]

,id,name,content,timestamp,channel,server,isBot,mentions,emojis
483,310277096683601920,BassgodTV#2261,bad entry into nflx,2019-09-19T20:01:42.511000+00:00,wetlqd-ideas,Misc,0,,
777,579205396166737921,beenpippinsincebeenpippin#4039,Some more legs of that nflx earnings play comi...,2019-10-18T16:02:04.230000+00:00,wetlqd-ideas,Misc,0,,
2675,579205396166737921,beenpippinsincebeenpippin#4039,Just playing around small with the iv shifts ...,2020-04-21T15:00:34.317000+00:00,wetlqd-ideas,Misc,0,,
6958,443880858529693716,OnionKnight#2425,Trimming from nflx gain today,2020-09-15T13:59:14.966000+00:00,wetlqd-ideas,Misc,0,,
8641,480583337661038602,DCruz#3069,> Has anyone taken a look at nflx\n<@713103211...,2020-09-27T03:22:20.051000+00:00,options-chat,BullTrades,0,713103211141136602,🙂
15937,748645854482792570,tdub2322#9571,!ta nflx,2020-10-24T06:00:48.017000+00:00,stock-check,Pennybois,0,,
15938,748645854482792570,tdub2322#9571,!cd nflx,2020-10-24T06:01:15.377000+00:00,stock-check,Pennybois,0,,
22725,636343146246307846,Chabias_Trades#3203,I’m in nflx and dhr for large cap swings,2020-11-15T04:53:08.813000+00:00,options-chat,BullTrades,0,,
27708,734512402124636251,rawitg#6093,!cd nflx,2020-11-29T01:40:49.301000+00:00,stock-check,Pennybois,0,,
30546,499289176945197057,kdbar#4223,I prefer dis if you're looking at nflx. They w...,2020-12-06T01:55:34.569000+00:00,option-trading,Templar Trading,0,,


In [29]:
# !abbv represent commands said in the channel in order to get a response from specific bot 
total_df[lambda x: x.content.str.contains(r"!ta")].iloc[0, 2]

'<@744455713732165652> if you go to the stockbot room and type !ta stockname it will tell you the support.... That would be a good entry point if you can get in around there'

In [30]:
total_df[lambda x: x.content.str.contains(r"!ta")].iloc[0, :]

id                                          719195862869475360
name                                             Sarizzle#3273
content      <@744455713732165652> if you go to the stockbo...
timestamp                     2020-09-23T02:10:33.705000+00:00
channel                                             trade-chat
server                                                 Learn-n
isBot                                                        0
mentions                                    744455713732165652
emojis                                                        
Name: 7995, dtype: object

In [31]:
total_df.loc[7992:8005, "content"]

7992                   <@!719195862869475360> yes 1 month
7993    <@744455713732165652> Awesome I am going on fo...
7994    <@!719195862869475360> I’m learning about that...
7995    <@744455713732165652> if you go to the stockbo...
7996                                              !ta aks
7997                                          For example
7998                                                ☝️👁️👍
7999                                             !ta aapl
8000               <@!719195862869475360> here’s my play!
8001                   Do you have both a put and a call?
8002                                    I am in AMD also!
8003                      <@!719195862869475360> yes I do
8004                           I just noticed iron condor
8005                   Lol it’s ok I do that all the time
Name: content, dtype: object

In [32]:
total_df[lambda x: x.content.str.contains(r"![a-z]+")]

,id,name,content,timestamp,channel,server,isBot,mentions,emojis
3350,716994215862599744,sunny1293#1607,!alert <@!579205396166737921>,2020-06-17T16:59:41.393000+00:00,wetlqd-ideas,Misc,0,579205396166737921,
5200,200451257566101505,nzo#0913,!rank,2020-08-28T01:41:04.718000+00:00,general,OmegaTrades,0,,
5220,507295065308397588,shumps#9758,!rank,2020-08-28T01:56:52.242000+00:00,general,OmegaTrades,0,,
5235,476692246801022976,t0ny32#1039,!futures,2020-08-28T03:05:54.882000+00:00,general-trading,WayTrade,0,,
5312,366373605853888515,baby#4110,!rank,2020-08-28T15:40:14.679000+00:00,general,OmegaTrades,0,,
...,...,...,...,...,...,...,...,...,...
31673,731738204662923284,hippoQQ#2886,!ta fsr,2020-12-08T18:50:04.472000+00:00,stock-check,Pennybois,0,,
31691,715388996456415293,yellowserverguy#9736,!ta IBIO,2020-12-08T19:23:56.585000+00:00,stock-check,Pennybois,0,,
31694,764933310375526400,Cashman36#5176,!ta snss,2020-12-08T19:31:04.429000+00:00,stock-check,Pennybois,0,,
31698,726547154348408884,Rakesh#5604,!ta prog,2020-12-08T19:39:00.636000+00:00,stock-check,Pennybois,0,,


In [33]:
total_df.loc[5190:5205, :]

,id,name,content,timestamp,channel,server,isBot,mentions,emojis
5190,717204128673169532,BigBucktShaq#3361,I got that 9/11 call,2020-08-28T00:45:05.384000+00:00,general-trading,WayTrade,0,,
5191,743310228816134145,Ghosty 👻#0670,It's a shame oracle didn't have the bread to c...,2020-08-28T00:45:44.935000+00:00,general-trading,WayTrade,0,,
5192,743310228816134145,Ghosty 👻#0670,💀fat L,2020-08-28T00:45:53.339000+00:00,general-trading,WayTrade,0,,💀
5193,743310228816134145,Ghosty 👻#0670,Lmao imagine being a tiktok exec 💀,2020-08-28T00:46:38.291000+00:00,general-trading,WayTrade,0,,💀
5194,743310228816134145,Ghosty 👻#0670,Got every major player at your knees 🤑,2020-08-28T00:47:00.224000+00:00,general-trading,WayTrade,0,,🤑
5195,387329318021365774,endowdly#0373,> <@!387329318021365774> I thought that said 8...,2020-08-28T00:47:02.155000+00:00,wetlqd-ideas,Misc,0,387329318021365774,
5196,387329318021365774,endowdly#0373,less small (still small) action today,2020-08-28T00:49:17.227000+00:00,wetlqd-ideas,Misc,0,,
5197,622169977314869282,Riq#2278,Nah I didn’t <@717204128673169532>,2020-08-28T00:50:53.591000+00:00,general-trading,WayTrade,0,717204128673169532,
5198,250629554115313664,stamford™#1812,<@!712352964542595083> what app is that?,2020-08-28T01:13:12.071000+00:00,general-trading,WayTrade,0,712352964542595083,
5199,712352964542595083,KyaleTheChef#1672,Investing.com <@250629554115313664>,2020-08-28T01:15:57.596000+00:00,general-trading,WayTrade,0,250629554115313664,


In [34]:
wordy_df[lambda x: x.no_vow <1].head(30)

,word,count,no_vow
28,k,514,0
69,spy,313,0
205,c,134,0
213,cd,130,0
218,rn,128,0
231,try,124,0
290,pltr,98,0
312,btc,90,0
316,dkng,89,0
328,gg,86,0


In [35]:
total_df[lambda x: x.content.str.contains(r"\s*jks\s*")]

,id,name,content,timestamp,channel,server,isBot,mentions,emojis
14176,636343146246307846,Chabias_Trades#3203,plz die jks,2020-10-19T03:00:23.742000+00:00,stocks-chat,BullTrades,0,,
14184,546379668463288330,clashofclans2000#9874,"se is more linear, jks is just parabolic",2020-10-19T03:07:42.264000+00:00,stocks-chat,BullTrades,0,,
14220,758030851366846546,purplefloridian(andrew)#8481,Someone asked me if they should jump in jks,2020-10-19T03:30:40.311000+00:00,stocks-chat,BullTrades,0,,
14438,546379668463288330,clashofclans2000#9874,<@!636343146246307846> sell puts on jks,2020-10-19T22:56:18.191000+00:00,stocks-chat,BullTrades,0,636343146246307846,
14782,636343146246307846,Chabias_Trades#3203,i have to kill jks,2020-10-21T01:35:27.635000+00:00,stocks-chat,BullTrades,0,,
15055,387830168665849858,johnjacobs#4668,!ta jks,2020-10-21T17:15:09.184000+00:00,stock-check,Pennybois,0,,
15395,546379668463288330,clashofclans2000#9874,jks is insane,2020-10-22T21:49:51.976000+00:00,stocks-chat,BullTrades,0,,
15467,712352964542595083,KyaleTheChef#1672,im interested in jks now,2020-10-23T02:28:54.377000+00:00,general-trading,WayTrade,0,,
15615,546379668463288330,clashofclans2000#9874,impossible to predict jks,2020-10-23T04:03:58.887000+00:00,stocks-chat,BullTrades,0,,
15681,218856794896859139,Duki#1711,I also have a side of 50k in the brokerage I d...,2020-10-23T18:20:26.611000+00:00,stocks-chat,BullTrades,0,,


In [36]:
total_df[lambda x: x.content.str.contains("amzn")].sample(1, random_state=13).content

24583    20k is worth it in 2 years amzn will be soarin...
Name: content, dtype: object

In [38]:
import nltk

In [39]:
nltk.download("tagsets")

[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\Kyle\AppData\Roaming\nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


True

In [40]:
sample_tokens = word_tokenize(total_df[lambda x: x.content.str.contains("amzn")].sample(1, random_state=6).content.values[0])
sample_pos_tags = pos_tag(sample_tokens)

In [41]:
pattern = 'NP: {<DT>?<JJ>*<NN>}'

In [42]:
comment_parse = RegexpParser(pattern)
tree = comment_parse.parse(sample_pos_tags)

In [43]:
print(tree)

(S
  Was/NNP
  (NP b****in/NN)
  about/IN
  amzn/JJ
  (NP all/DT day/NN)
  and/CC
  then/RB
  sold/VBN
  at/IN
  breakeven/JJ
  </NNS
  :/:
  KEKW:756336180555022458/NNP
  (NP >/NN))


In [44]:
tree.draw()

In [ ]:
# There are Discord Specific Emojis still encased in <:abbv:> 

In [47]:
total_df[lambda x: x.content.str.contains("756336180555022458")]

,id,name,content,timestamp,channel,server,isBot,mentions,emojis
8045,709787787955535912,mov#1337,He doesn't have the S pen <:KEKW:7563361805550...,2020-09-23T03:41:47.545000+00:00,options-chat,BullTrades,0,,
12207,282684580924751873,deathdarkstar#5596,you guys still talking about hyln <:KEKW:75633...,2020-10-12T03:59:22.929000+00:00,options-chat,BullTrades,0,,
12427,636343146246307846,Chabias_Trades#3203,> <@!636343146246307846> do you want it green ...,2020-10-13T03:00:37.511000+00:00,options-chat,BullTrades,0,636343146246307846,
12428,636343146246307846,Chabias_Trades#3203,But dkng has to be green <:KEKW:75633618055502...,2020-10-13T03:00:51.830000+00:00,options-chat,BullTrades,0,,
14134,636343146246307846,Chabias_Trades#3203,WKHS bullish <:KEKW:756336180555022458>,2020-10-19T01:02:21.433000+00:00,stocks-chat,BullTrades,0,,
14442,636343146246307846,Chabias_Trades#3203,<:KEKW:756336180555022458>,2020-10-19T23:04:08.647000+00:00,stocks-chat,BullTrades,0,,
14942,636343146246307846,Chabias_Trades#3203,Nkla <:KEKW:756336180555022458>,2020-10-21T03:59:14.858000+00:00,stocks-chat,BullTrades,0,,
20296,636343146246307846,Chabias_Trades#3203,im gonna bully mov for being late now <:KEKW:7...,2020-11-07T23:01:53.686000+00:00,stocks-chat,BullTrades,0,,
23345,739919332175249408,HD to the Moon 280-290#6413,Play it so i can get puts<:KEKW:75633618055502...,2020-11-17T04:41:36.485000+00:00,options-chat,BullTrades,0,,
23353,739919332175249408,HD to the Moon 280-290#6413,<@592138987079073805> tell me what you get so ...,2020-11-17T04:42:50.222000+00:00,options-chat,BullTrades,0,592138987079073805,


In [49]:
total_df[lambda x: x.content.str.contains("<:.+:\d+>")]

,id,name,content,timestamp,channel,server,isBot,mentions,emojis
312,493522304660406299,Keaton#8892,👌 <:blobmuscles:613453963567562754> <:7489_blo...,2019-09-06T14:58:36.741000+00:00,wetlqd-ideas,Misc,0,,👌
1081,197410928088580097,Aesop#4590,Just can't get enough of Disney <:sflake:38814...,2019-11-19T00:48:27.294000+00:00,wetlqd-ideas,Misc,0,,
1227,162943158932602890,MaykoGames#8282,All aboard the <@443933735445069825> gain trai...,2019-12-04T14:34:01.975000+00:00,wetlqd-ideas,Misc,0,443933735445069825,
1307,593901936097361921,crispanda#7117,Damn show me the way <@504078660857364480> <:S...,2019-12-16T22:08:51.418000+00:00,wetlqd-ideas,Misc,0,504078660857364480,🤑
1544,283102527724060672,GreenGiant#2259,Just went from trial to paid member. Thank you...,2020-01-15T17:22:17.134000+00:00,wetlqd-ideas,Misc,0,,
...,...,...,...,...,...,...,...,...,...
31790,747440897708392560,EvolvingBull#1736,"Everyone is bullish on V, so now you know what...",2020-12-08T19:58:31.517000+00:00,options-chat,BullTrades,0,,
31803,396593452340609024,ExtraGravy#4730,People told me TSLA was gonna rugpull to 550 t...,2020-12-08T19:59:07.159000+00:00,options-chat,BullTrades,0,,
31810,704074357135114302,MMs hate me#2020,Was b****in about amzn all day and then sold a...,2020-12-08T20:01:34.900000+00:00,options-chat,BullTrades,0,,
31856,101806058359496704,Ryan (itsGoggles)#8580,$CANG up too <:peepoyay:735222523683405984>,2020-12-08T20:13:38.984000+00:00,main-trading,Templar Trading,0,,


In [ ]:
# The most important tokens left to be selected are urls, discord specific emotes (<:[A-Za-z]+:\d+>) and finally companies mentioned
# Being able to best ascertain the companies mentioned  by specific comments (company name, company ticker, other associated words) will allow me to begin to collect the associated industry
# sentiment, which will be the general finanical data of the most popular companies, the Finicial Times news during this period and its sentiment and the general news about a company around this time
# gathers from the news api. These three separate data sources will provide sentiment scores representing retail investor sentiment (from the discord private chats), finicial news sector sentiment (from the scraping of the Financial News)
# and the Overall News Media's Sentiment of a Company (from the News API).

# The three sentiments of the separate sectors will give us the general mood of these sectors in time to match with the financial data for a company during this same time.
# Having these separate data souces broken down by type will allow us to tell the story of a company by the moods of the separate cohorts. 

In [91]:
url_regex = r"https*\:.+|www\..+"

In [82]:
discord_emote_regex = r"<:.+:\d+>"

In [28]:
def add_specialized_token_column(regex, comment_column, new_column_name):
    total_df.loc[:, new_column_name] = total_df.loc[:, comment_column].apply(lambda x: ",".join([r for r in re.findall(regex, x)]))
    return total_df

In [106]:
total_df_w_url = add_specialized_token_column(url_regex, "content", "links")

In [108]:
total_df_w_url.links.unique()

array(['', 'https://gyazo.com/82a81327c14e0d221c62b3308e6c584b',
       'https://www.optionsprofitcalculator.com/calculator/2-legs.html#',
       'https://www.myfxbook.com/members/FXMomentumProfit/momentum-profit-4/4288793',
       'https://youtu.be/GP2VRfedxJQ',
       'https://tenor.com/view/the-matrix-choose-pill-gif-5437251',
       'https://tenor.com/view/wow-ted2-ted2gifs-gif-4289696',
       'https://tenor.com/view/yuge-huge-donald-trump-trump-donald-gif-7357158',
       'https://seekingalpha.com/article/4342364',
       'https://cdn.discordapp.com/attachments/478633056392183855/709360297944809482/unknown.png',
       'https://www.google.com/amp/s/www.marketwatch.com/amp/story/teslas-stock-is-forming-a-bubble-and-new-buyers-should-buckle-up-for-a-crash-2020-07-07 <@376116514841165824>',
       'https://i.imgur.com/k9bTodP.png', 'https://youtu.be/89z_IdhODrk',
       'https://i.imgur.com/jUlMdxk.png', 'https://youtu.be/9BWGIZrxx0I',
       'https://i.gyazo.com/52ac9b30d0bbf28a06f

In [109]:
total_df_w_new_cols = add_specialized_token_column(discord_emote_regex, "content", "chat_emotes")

In [112]:
total_df_w_new_cols.chat_emotes.unique()

array(['',
       '<:blobmuscles:613453963567562754> <:7489_blob_shrek:613455178871603230>',
       '<:sflake:388149501761814538>',
       '<:blobmuscles:613453963567562754>',
       '<:Spitfire:653459285501804582>', '<:Calls:653459284759150603>',
       '<:Cry:653459282926370836>', '<:donald:626506346984112138>',
       '<:money:613455802115686430>', '<:popcorn:613453964238913547>',
       '<:yikes:653459285002420255>', '<:hmmm:613455179416600592>',
       '<:Cheers:653449290060136448>',
       '<:huehuepeepo:735225484056592566>', '<:kekw:735225483838488608>',
       '<:sadpep:735222276282122360>', '<:spear:613453964599623690>',
       '<:ok:653459285493153802>', '<:big_win:689349590650060870>',
       '<:Green_Up:685637813957427272>', '<:elon:743202010131726377>',
       '<:PepeHands:648264600466358284>',
       '<:peeposalute:735222277116788847>',
       '<:Moneybags:690004258590163021> <:Moneybags:690004258590163021>',
       '<:RIP:685383059151126729>',
       '<:emoji_2:722415314

In [115]:
total_df_w_new_cols[lambda x: (x.chat_emotes!="")]

,id,name,content,timestamp,channel,server,isBot,mentions,emojis,links,chat_emotes
312,493522304660406299,Keaton#8892,👌 <:blobmuscles:613453963567562754> <:7489_blo...,2019-09-06T14:58:36.741000+00:00,wetlqd-ideas,Misc,0,,👌,,<:blobmuscles:613453963567562754> <:7489_blob_...
1081,197410928088580097,Aesop#4590,Just can't get enough of Disney <:sflake:38814...,2019-11-19T00:48:27.294000+00:00,wetlqd-ideas,Misc,0,,,,<:sflake:388149501761814538>
1227,162943158932602890,MaykoGames#8282,All aboard the <@443933735445069825> gain trai...,2019-12-04T14:34:01.975000+00:00,wetlqd-ideas,Misc,0,443933735445069825,,,<:blobmuscles:613453963567562754>
1307,593901936097361921,crispanda#7117,Damn show me the way <@504078660857364480> <:S...,2019-12-16T22:08:51.418000+00:00,wetlqd-ideas,Misc,0,504078660857364480,🤑,,<:Spitfire:653459285501804582>
1544,283102527724060672,GreenGiant#2259,Just went from trial to paid member. Thank you...,2020-01-15T17:22:17.134000+00:00,wetlqd-ideas,Misc,0,,,,<:Calls:653459284759150603>
...,...,...,...,...,...,...,...,...,...,...,...
31790,747440897708392560,EvolvingBull#1736,"Everyone is bullish on V, so now you know what...",2020-12-08T19:58:31.517000+00:00,options-chat,BullTrades,0,,,,<:weSmart:768305799625179196>
31803,396593452340609024,ExtraGravy#4730,People told me TSLA was gonna rugpull to 550 t...,2020-12-08T19:59:07.159000+00:00,options-chat,BullTrades,0,,,,<:KEKW:756336180555022458>
31810,704074357135114302,MMs hate me#2020,Was b****in about amzn all day and then sold a...,2020-12-08T20:01:34.900000+00:00,options-chat,BullTrades,0,,,,<:KEKW:756336180555022458>
31856,101806058359496704,Ryan (itsGoggles)#8580,$CANG up too <:peepoyay:735222523683405984>,2020-12-08T20:13:38.984000+00:00,main-trading,Templar Trading,0,,,,<:peepoyay:735222523683405984>


In [134]:
discord_emote_regex = r"<:.+:\d+>"

In [188]:
total_df_w_new_cols = add_specialized_token_column(discord_emote_regex, "content", "chat_emotes")

In [189]:
emote_name = r":\w+:"

In [190]:
total_df_w_new_cols[lambda x: x.chat_emotes != ""].chat_emotes.apply(lambda x: ",".join([e.replace(":", "") for e in re.findall(emote_name, x)]))

312      blobmuscles,7489_blob_shrek
1081                          sflake
1227                     blobmuscles
1307                        Spitfire
1544                           Calls
                    ...             
31790                        weSmart
31803                           KEKW
31810                           KEKW
31856                       peepoyay
31884                        wesmart
Name: chat_emotes, Length: 555, dtype: object

In [191]:
total_df_w_new_cols.loc[:, "chat_emotes"] = total_df_w_new_cols.chat_emotes.apply(lambda x: ",".join([e.replace(":", "") for e in re.findall(emote_name, x)]))

In [193]:
total_df_w_new_cols[lambda x: x.chat_emotes != ""]

,id,name,content,timestamp,channel,server,isBot,mentions,emojis,links,chat_emotes
312,493522304660406299,Keaton#8892,👌 <:blobmuscles:613453963567562754> <:7489_blo...,2019-09-06T14:58:36.741000+00:00,wetlqd-ideas,Misc,0,,👌,,"blobmuscles,7489_blob_shrek"
1081,197410928088580097,Aesop#4590,Just can't get enough of Disney <:sflake:38814...,2019-11-19T00:48:27.294000+00:00,wetlqd-ideas,Misc,0,,,,sflake
1227,162943158932602890,MaykoGames#8282,All aboard the <@443933735445069825> gain trai...,2019-12-04T14:34:01.975000+00:00,wetlqd-ideas,Misc,0,443933735445069825,,,blobmuscles
1307,593901936097361921,crispanda#7117,Damn show me the way <@504078660857364480> <:S...,2019-12-16T22:08:51.418000+00:00,wetlqd-ideas,Misc,0,504078660857364480,🤑,,Spitfire
1544,283102527724060672,GreenGiant#2259,Just went from trial to paid member. Thank you...,2020-01-15T17:22:17.134000+00:00,wetlqd-ideas,Misc,0,,,,Calls
...,...,...,...,...,...,...,...,...,...,...,...
31790,747440897708392560,EvolvingBull#1736,"Everyone is bullish on V, so now you know what...",2020-12-08T19:58:31.517000+00:00,options-chat,BullTrades,0,,,,weSmart
31803,396593452340609024,ExtraGravy#4730,People told me TSLA was gonna rugpull to 550 t...,2020-12-08T19:59:07.159000+00:00,options-chat,BullTrades,0,,,,KEKW
31810,704074357135114302,MMs hate me#2020,Was b****in about amzn all day and then sold a...,2020-12-08T20:01:34.900000+00:00,options-chat,BullTrades,0,,,,KEKW
31856,101806058359496704,Ryan (itsGoggles)#8580,$CANG up too <:peepoyay:735222523683405984>,2020-12-08T20:13:38.984000+00:00,main-trading,Templar Trading,0,,,,peepoyay


In [196]:
total_df_w_new_cols

,id,name,content,timestamp,channel,server,isBot,mentions,emojis,links,chat_emotes
0,579205396166737921,beenpippinsincebeenpippin#4039,Play account today,2019-08-02T01:10:52.637000+00:00,wetlqd-ideas,Misc,0,,,,
1,579205396166737921,beenpippinsincebeenpippin#4039,@everyone today’s total. This doesn’t calculat...,2019-08-02T01:13:47.923000+00:00,wetlqd-ideas,Misc,0,everyone,,,
2,445583268792893449,itsyoboyvic#7198,Crack,2019-08-02T01:14:35.798000+00:00,wetlqd-ideas,Misc,0,,,,
3,445583268792893449,itsyoboyvic#7198,Big crack,2019-08-02T01:14:39.682000+00:00,wetlqd-ideas,Misc,0,,,,
4,445583268792893449,itsyoboyvic#7198,Heroin,2019-08-02T01:14:44.367000+00:00,wetlqd-ideas,Misc,0,,,,
...,...,...,...,...,...,...,...,...,...,...,...
31929,428921853403463713,Baraxton#0199,<@!644997307976122388> all of my shorter video...,2020-12-08T20:38:45.351000+00:00,trading,LSV,0,644997307976122388,,,
31930,184420431242067977,Dg#8853,<@428921853403463713> have you done a video in...,2020-12-08T20:39:17.246000+00:00,trading,LSV,0,428921853403463713,,,
31931,428921853403463713,Baraxton#0199,"<@!184420431242067977> nope, not yet",2020-12-08T20:39:32.831000+00:00,trading,LSV,0,184420431242067977,,,
31932,428921853403463713,Baraxton#0199,they're the airline wifi company right?,2020-12-08T20:39:38.597000+00:00,trading,LSV,0,,,,


In [198]:
total_df_w_new_cols.to_sql("comments", con="sqlite:///../data/interim/discord/discord.db", index_label="pk", if_exists="replace")

In [ ]:
# Finding Companies 
# We want to get a list of all tokens and their frequencies as the starting point to company mentions.
# to do so, we need to copy the content column, lower its words, strip out the specialized tokens in a row's columns, remove all puncutations and remove stop words.
# From here, we should get a good starting lexicon for the chatrooms and analyze it to see which tokens refer back to a specific company

In [847]:
token_df = total_df_w_new_cols.copy(deep=True)

In [849]:
# Use the previously defined regular expressions to find the non-word tokens in a sentence
userIdCatcher = r"(<@(!)?\d+>|@everyone)"
discord_emote_regex = r"<:.+:\d+>"
url_regex = r"https*\:.+|www\..+"
punc_regex = r"[\’\'\'\"\"\“\”!\?@#$%&\(\)\*,-.\\\{\}+~\/:;<>\[\]^`|=_’]"
contract_regex = r"[\'\’\’]"
emoji_regex = emoji.get_emoji_regexp()

In [850]:
token_df = token_df[lambda x: x.isBot == 0]

In [851]:
def clean_comment(comment):
    comment = re.sub("|".join([x for x in [userIdCatcher, discord_emote_regex, url_regex]]), "", comment)
    comment = re.sub(contract_regex, "", comment)
    comment = re.sub(emoji_regex, "", comment)
    comment = re.sub(punc_regex, " ", comment)
    return comment.replace("\n", " ").lower()

In [852]:
token_df = token_df.assign(for_tokenizer = lambda x: x.content.apply(clean_comment)).reset_index(drop=True)

In [853]:
token_df.head()

,id,name,content,timestamp,channel,server,isBot,mentions,emojis,links,chat_emotes,for_tokenizer
0,579205396166737921,beenpippinsincebeenpippin#4039,Play account today,2019-08-02T01:10:52.637000+00:00,wetlqd-ideas,Misc,0,,,,,play account today
1,579205396166737921,beenpippinsincebeenpippin#4039,@everyone today’s total. This doesn’t calculat...,2019-08-02T01:13:47.923000+00:00,wetlqd-ideas,Misc,0,everyone,,,,todays total this doesnt calculate the after...
2,445583268792893449,itsyoboyvic#7198,Crack,2019-08-02T01:14:35.798000+00:00,wetlqd-ideas,Misc,0,,,,,crack
3,445583268792893449,itsyoboyvic#7198,Big crack,2019-08-02T01:14:39.682000+00:00,wetlqd-ideas,Misc,0,,,,,big crack
4,445583268792893449,itsyoboyvic#7198,Heroin,2019-08-02T01:14:44.367000+00:00,wetlqd-ideas,Misc,0,,,,,heroin


In [854]:
def wordCounts(comment_array):
    comment_dictionary = {} 
    for comment in comment_array:
        words = word_tokenize(comment)
        for w in words:
            if (w in comment_dictionary.keys()) & (w not in stop_words):
                comment_dictionary[w] += 1
            elif (w not in stop_words):
                comment_dictionary[w] = 1
    comments_words = dict(sorted(comment_dictionary.items(), key=lambda x: x[1], reverse=True))
    return pd.DataFrame.from_dict(comments_words, orient="index", columns=["count"]).reset_index().rename(columns={"index": "word"})

In [855]:
lex_df = wordCounts(token_df.for_tokenizer.values)

In [856]:
lex_df.shape

(13868, 2)

In [857]:
# sample of key company tokens
lex_df[lambda x: x.word == "amzn"] # stock ticker

,word,count
498,amzn,60


In [858]:
lex_df[lambda x: x.word == "amazon"] # single world company

,word,count
692,amazon,42


In [859]:
lex_df[lambda x: x.word == "bezos"] # ceo name

,word,count
9605,bezos,1


In [860]:
lex_df[lambda x: x.word == "aws"] # business silos 

,word,count


In [861]:
# sample of key company tokens
lex_df[lambda x: x.word == "tsla"] # stock ticker

,word,count
100,tsla,254


In [862]:
lex_df[lambda x: x.word == "tesla"] # single word company name

,word,count
151,tesla,187


In [863]:
lex_df[lambda x: x.word == "elon"] # ceo name

,word,count
735,elon,39


In [864]:
lex_df[lambda x: x.word == "musk"]

,word,count
2742,musk,7


In [865]:
def pairCounts(comment_array):
    comment_dictionary = {} 
    for comment in comment_array:
        words = word_tokenize(comment)
        pairs = [" ".join(pair) for pair in nltk.bigrams(words)]
        for pair in pairs:
            couple = pair.split(" ")
            if (pair in comment_dictionary.keys()) & (couple[0] not in stop_words) & (couple[1] not in stop_words):
                comment_dictionary[pair] += 1
            elif (pair not in stop_words) & (couple[0] not in stop_words) & (couple[1] not in stop_words):
                comment_dictionary[pair] = 1
    comments_words = dict(sorted(comment_dictionary.items(), key=lambda x: x[1], reverse=True))
    return pd.DataFrame.from_dict(comments_words, orient="index", columns=["count"]).reset_index().rename(columns={"index": "word"})

In [866]:
word_pairs = pairCounts(token_df.for_tokenizer.values)

In [867]:
word_pairs.head(25)

,word,count
0,gon na,382
1,long term,208
2,next week,147
3,looks like,115
4,got ta,109
5,day trade,107
6,dont think,83
7,dont know,74
8,wan na,71
9,last week,70


In [868]:
def pairCounts(comment_array):
    comment_dictionary = {} 
    for comment in comment_array:
        words = word_tokenize(comment)
        pairs = [" ".join(pair) for pair in nltk.bigrams(words)]
        for pair in pairs:
            if (pair in comment_dictionary.keys()):
                comment_dictionary[pair] += 1
            elif (pair not in stop_words):
                comment_dictionary[pair] = 1
    comments_words = dict(sorted(comment_dictionary.items(), key=lambda x: x[1], reverse=True))
    return pd.DataFrame.from_dict(comments_words, orient="index", columns=["count"]).reset_index().rename(columns={"index": "word"})

In [869]:
word_pairs2 = pairCounts(token_df.for_tokenizer.values) # with stop words

In [870]:
word_pairs2.head(25)

,word,count
0,in the,511
1,i think,459
2,of the,435
3,on the,428
4,for the,409
5,for a,391
6,gon na,382
7,i have,381
8,if you,335
9,to be,329


In [871]:
word_pairs2.iloc[45:65, :]

,word,count
45,thank you,167
46,would be,167
47,to see,163
48,as well,158
49,i would,157
50,you have,157
51,out of,155
52,is the,155
53,to buy,155
54,the day,150


In [872]:
word_pairs.head(25)

,word,count
0,gon na,382
1,long term,208
2,next week,147
3,looks like,115
4,got ta,109
5,day trade,107
6,dont think,83
7,dont know,74
8,wan na,71
9,last week,70


In [873]:
word_pairs[lambda x: x["count"] == 2].head(25)

,word,count
3085,better ones,2
3086,day red,2
3087,well thats,2
3088,calls looking,2
3089,roku calls,2
3090,50 calls,2
3091,around 110,2
3092,tomorrow big,2
3093,bro bro,2
3094,small profits,2


In [874]:
lex_df.head(28)

,word,count
0,im,1559
1,like,1338
2,good,1203
3,day,1018
4,lol,975
5,get,882
6,think,873
7,dont,824
8,one,788
9,got,759


In [875]:
lex_df.iloc[50:75, :]

,word,count
50,ill,390
51,next,389
52,gon,386
53,last,381
54,shares,368
55,could,360
56,profit,357
57,10,357
58,look,353
59,earnings,350


In [876]:
lex_df.iloc[275:295, :]

,word,count
275,isnt,109
276,set,108
277,thoughts,108
278,enough,108
279,everything,107
280,less,106
281,using,106
282,baba,105
283,bullish,105
284,learn,104


In [877]:
token_df[lambda x: x.for_tokenizer.str.contains("biden")].iloc[-10, 2]

'Hey guys. Recovery stocks such as hotel cruise and airlines might or might not be good.  Look out for updates for Biden’s plan for Covid. Ie lockdown or not'

In [878]:
token_df[lambda x: x.for_tokenizer.str.contains("trump")]

,id,name,content,timestamp,channel,server,isBot,mentions,emojis,links,chat_emotes,for_tokenizer
102,456226577798135808,Deleted User#0000,That’s epic brother! Like a 70% gain is so ni...,2019-08-19T16:12:13.325000+00:00,wetlqd-ideas,Misc,0,513854555239350274,,,,thats epic brother like a 70 gain is so nic...
706,630894290880561152,Elvis#4387,Tried to ride the rally wave but got brought d...,2019-10-10T14:18:15.061000+00:00,wetlqd-ideas,Misc,0,,,,,tried to ride the rally wave but got brought d...
1271,197410928088580097,Aesop#4590,Blessed by Trump this morning.,2019-12-12T15:14:18.865000+00:00,wetlqd-ideas,Misc,0,,,,,blessed by trump this morning
2161,397826230424174602,Ant#3194,"Swinging the protection, maybe a trump pump ca...",2020-03-11T19:49:07.360000+00:00,wetlqd-ideas,Misc,0,,,,,swinging the protection maybe a trump pump ca...
2244,579205396166737921,beenpippinsincebeenpippin#4039,Thanks trump,2020-03-17T16:01:13.308000+00:00,wetlqd-ideas,Misc,0,,,,,thanks trump
...,...,...,...,...,...,...,...,...,...,...,...,...
24925,681688842352590892,MomImTrying#8143,Trump also said he's making executive decision...,2020-11-22T02:32:49.933000+00:00,general-trading,WayTrade,0,,,,,trump also said hes making executive decision ...
24983,498323828762738690,Bundy#0025,"China stocks too , just not touching that unti...",2020-11-22T04:47:38.345000+00:00,options-chat,BullTrades,0,,,,,china stocks too just not touching that unti...
27003,366373605853888515,baby#4110,cuz if trump was elected,2020-11-27T17:29:06.047000+00:00,sentiment,OmegaTrades,0,,,,,cuz if trump was elected
28233,497160533611380740,problueprogreen#3255,But Trump is pushing as hard as he can if he d...,2020-12-01T04:21:10.733000+00:00,trade-chat,Learn-n,0,,,,,but trump is pushing as hard as he can if he d...


In [879]:
token_df.timestamp = pd.to_datetime(token_df.timestamp)

In [528]:
token_df.groupby([pd.Grouper(key="timestamp", freq="M"), "server"]).count()

,,id,name,content,channel,isBot,mentions,emojis,links,chat_emotes,for_tokenizer
timestamp,server,,,,,,,,,,
2019-08-31 00:00:00+00:00,Misc,264,264,264,264,264,264,264,264,264,264
2019-09-30 00:00:00+00:00,Misc,318,318,318,318,318,318,318,318,318,318
2019-10-31 00:00:00+00:00,Misc,303,303,303,303,303,303,303,303,303,303
2019-11-30 00:00:00+00:00,Misc,307,307,307,307,307,307,307,307,307,307
2019-12-31 00:00:00+00:00,Misc,226,226,226,226,226,226,226,226,226,226
2020-01-31 00:00:00+00:00,Misc,236,236,236,236,236,236,236,236,236,236
2020-02-29 00:00:00+00:00,Misc,321,321,321,321,321,321,321,321,321,321
2020-03-31 00:00:00+00:00,Misc,458,458,458,458,458,458,458,458,458,458
2020-04-30 00:00:00+00:00,Misc,366,366,366,366,366,366,366,366,366,366


In [880]:
token_df.groupby([pd.Grouper(key="timestamp", freq="M"), "server"]).count()

,,id,name,content,channel,isBot,mentions,emojis,links,chat_emotes,for_tokenizer
timestamp,server,,,,,,,,,,
2019-08-31 00:00:00+00:00,Misc,264,264,264,264,264,264,264,264,264,264
2019-09-30 00:00:00+00:00,Misc,318,318,318,318,318,318,318,318,318,318
2019-10-31 00:00:00+00:00,Misc,303,303,303,303,303,303,303,303,303,303
2019-11-30 00:00:00+00:00,Misc,307,307,307,307,307,307,307,307,307,307
2019-12-31 00:00:00+00:00,Misc,226,226,226,226,226,226,226,226,226,226
2020-01-31 00:00:00+00:00,Misc,236,236,236,236,236,236,236,236,236,236
2020-02-29 00:00:00+00:00,Misc,321,321,321,321,321,321,321,321,321,321
2020-03-31 00:00:00+00:00,Misc,458,458,458,458,458,458,458,458,458,458
2020-04-30 00:00:00+00:00,Misc,366,366,366,366,366,366,366,366,366,366


In [881]:
word_pairs2[lambda x: x.word.str.contains("trump")]

,word,count
926,trump is,23
1495,if trump,16
2624,a trump,10
2816,like trump,10
2819,trump wins,10
...,...,...
93168,donald trump,1
93169,trump jr,1
94326,trump also,1
94468,until trump,1


In [882]:
word_pairs2[lambda x: x.word.str.contains("biden")]

,word,count
2827,a biden,10
3718,biden is,8
3760,biden win,8
4470,biden wins,7
5431,if biden,6
...,...,...
94498,biden should,1
96232,of bidens,1
96233,bidens new,1
96504,biden to,1


In [883]:
token_df

,id,name,content,timestamp,channel,server,isBot,mentions,emojis,links,chat_emotes,for_tokenizer
0,579205396166737921,beenpippinsincebeenpippin#4039,Play account today,2019-08-02 01:10:52.637000+00:00,wetlqd-ideas,Misc,0,,,,,play account today
1,579205396166737921,beenpippinsincebeenpippin#4039,@everyone today’s total. This doesn’t calculat...,2019-08-02 01:13:47.923000+00:00,wetlqd-ideas,Misc,0,everyone,,,,todays total this doesnt calculate the after...
2,445583268792893449,itsyoboyvic#7198,Crack,2019-08-02 01:14:35.798000+00:00,wetlqd-ideas,Misc,0,,,,,crack
3,445583268792893449,itsyoboyvic#7198,Big crack,2019-08-02 01:14:39.682000+00:00,wetlqd-ideas,Misc,0,,,,,big crack
4,445583268792893449,itsyoboyvic#7198,Heroin,2019-08-02 01:14:44.367000+00:00,wetlqd-ideas,Misc,0,,,,,heroin
...,...,...,...,...,...,...,...,...,...,...,...,...
31636,428921853403463713,Baraxton#0199,<@!644997307976122388> all of my shorter video...,2020-12-08 20:38:45.351000+00:00,trading,LSV,0,644997307976122388,,,,all of my shorter videos are price action dis...
31637,184420431242067977,Dg#8853,<@428921853403463713> have you done a video in...,2020-12-08 20:39:17.246000+00:00,trading,LSV,0,428921853403463713,,,,have you done a video in gogo by any chance
31638,428921853403463713,Baraxton#0199,"<@!184420431242067977> nope, not yet",2020-12-08 20:39:32.831000+00:00,trading,LSV,0,184420431242067977,,,,nope not yet
31639,428921853403463713,Baraxton#0199,they're the airline wifi company right?,2020-12-08 20:39:38.597000+00:00,trading,LSV,0,,,,,theyre the airline wifi company right


In [443]:
tickers_and_names = tickers_and_names.assign(ticker = lambda x: x.Symbol.apply(str.lower))

In [532]:
tickers_and_names.head(25)

,Symbol,Name,single_name,ticker
0,AAPL,Apple Inc. Common Stock,apple,aapl
1,MSFT,Microsoft Corporation Common Stock,microsoft,msft
2,GOOG,Alphabet Inc. Class C Capital Stock,alphabet,goog
3,GOOGL,Alphabet Inc. Class A Common Stock,alphabet,googl
4,AMZN,Amazon.com Inc. Common Stock,amazon.com,amzn
5,TSLA,Tesla Inc. Common Stock,tesla,tsla
6,FB,Meta Platforms Inc. Class A Common Stock,meta platforms,fb
7,NVDA,NVIDIA Corporation Common Stock,nvidia,nvda
8,TSM,Taiwan Semiconductor Manufacturing Company Ltd.,taiwan semiconductor manufacturing,tsm
9,JPM,JP Morgan Chase & Co. Common Stock,jp morgan chase & co.,jpm


In [446]:
ticker_array = tickers_and_names.ticker.values

In [535]:
tickers_and_names[lambda x: x.ticker == "nio"]

,Symbol,Name,single_name,ticker
215,NIO,NIO Inc. American depositary shares each repr...,nio american each representing one ordinar...,nio


In [538]:
token_df[lambda x: x.for_tokenizer.str.contains("elon")].loc[3651, "content"]

'Ya NeuroLink is what I’m investing in too, I mean who wouldn’t as we near the technological singularity. Elon is just one of many people who will change the world❤️'

In [574]:
token_df.loc[7733, "content"]

'https://www.globenewswire.com/news-release/2020/09/21/2096779/0/en/Nikola-Corporation-Investors-Lawsuit-filed-to-recover-losses-suffered-as-a-result-of-alleged-fraud.html'

In [626]:
token_df.loc[7790, "content"]

'market should rally before elections'

In [627]:
sample_tokens = word_tokenize(token_df.loc[7790, "content"])
sample_pos_tags = pos_tag(sample_tokens)

In [628]:
pattern = 'NP: {<DT>?<JJ>*<NN>}'

In [629]:
comment_parse = RegexpParser(pattern)
tree = comment_parse.parse(sample_pos_tags)

In [630]:
print(tree)

(S (NP market/NN) should/MD rally/VB before/IN elections/NNS)


In [657]:
tree.draw()

In [667]:
tickers_and_names.Symbol = tickers_and_names.Symbol.apply(str.strip)

In [689]:
tickers_and_names.assign(length = lambda x: x.Symbol.apply(len))[lambda x: x.length == 1]

,Symbol,Name,single_name,ticker,length
10,V,Visa Inc.,visa,v,1
62,T,AT&T Inc.,at&t,t,1
91,C,Citigroup Inc. Common Stock,citigroup,c,1
151,F,Ford Motor Company Common Stock,ford motor,f,1
195,D,Dominion Energy Inc. Common Stock,dominion energy,d,1
243,E,ENI S.p.A. Common Stock,eni s.p.a.,e,1
263,A,Agilent Technologies Inc. Common Stock,agilent technologies,a,1
281,U,Unity Software Inc. Common Stock,unity software,u,1
328,O,Realty Income Corporation Common Stock,realty income,o,1
487,W,Wayfair Inc. Class A Common Stock,wayfair,w,1


In [687]:
company_regex = re.compile(r"[A-Z]{2,5}")

In [884]:
token_df[lambda x: x.content.str.contains(company_regex)]

,id,name,content,timestamp,channel,server,isBot,mentions,emojis,links,chat_emotes,for_tokenizer
23,445583268792893449,itsyoboyvic#7198,I took stamps .... and IAC both not looking go...,2019-08-08 03:31:33.793000+00:00,wetlqd-ideas,Misc,0,,,,,i took stamps and iac both not looking go...
26,194305752939102208,D.Ninan#3776,Sold AMD aug23rd call for a fat profit,2019-08-08 13:33:00.968000+00:00,wetlqd-ideas,Misc,0,,,,,sold amd aug23rd call for a fat profit
47,194305752939102208,D.Ninan#3776,MSFT profits im still not selling it tho just ...,2019-08-13 14:21:58.192000+00:00,wetlqd-ideas,Misc,0,,,,,msft profits im still not selling it tho just ...
54,344157708523667457,dylan.patel#4692,$139.50 and ATH,2019-08-13 19:01:16.234000+00:00,wetlqd-ideas,Misc,0,,,,,139 50 and ath
56,194305752939102208,D.Ninan#3776,$139.50 and ATH? Wym,2019-08-13 19:05:27.213000+00:00,wetlqd-ideas,Misc,0,,,,,139 50 and ath wym
...,...,...,...,...,...,...,...,...,...,...,...,...
31617,428921853403463713,Baraxton#0199,@everyone EDIT is popping off,2020-12-08 20:25:14.542000+00:00,trading,LSV,0,everyone,,,,edit is popping off
31623,785007528152399913,nickb34#9557,$OXY broke the resistance line on Brad's chart,2020-12-08 20:26:51.453000+00:00,trading,LSV,0,,,,,oxy broke the resistance line on brads chart
31626,594593113104842795,VelcroKarma#1526,if PIXY runs from being included in the Benzin...,2020-12-08 20:27:59.342000+00:00,trade-chat,Learn-n,0,,,,,if pixy runs from being included in the benzin...
31637,184420431242067977,Dg#8853,<@428921853403463713> have you done a video in...,2020-12-08 20:39:17.246000+00:00,trading,LSV,0,428921853403463713,,,,have you done a video in gogo by any chance


In [885]:
token_df[lambda x: x.for_tokenizer.str.contains(r" h ")]

,id,name,content,timestamp,channel,server,isBot,mentions,emojis,links,chat_emotes,for_tokenizer
7588,708874403517169715,KeithC 🎄#8186,This is what I saw. Cnblast saw the opposite i...,2020-09-21 03:38:28.525000+00:00,options-chat,BullTrades,0,,,,,this is what i saw cnblast saw the opposite i...
8388,709787787955535912,mov#1337,C&H on NFLX 4H?,2020-09-26 03:10:46.953000+00:00,options-chat,BullTrades,0,,,,,c h on nflx 4h
10225,712352964542595083,KyaleTheChef#1672,were we bamboozled by that c&h breakout? <@!49...,2020-10-05 01:29:34.167000+00:00,general-trading,WayTrade,0,494525003484889088,,,,were we bamboozled by that c h breakout
10640,700394878462328832,Jaime#0289,Hey I’m not gonna lie the inverse h&l actually...,2020-10-06 22:54:18.693000+00:00,trading-discussion,Legacy Trading,0,,🤣,,,hey im not gonna lie the inverse h l actually ...
11094,726686965470330911,D.S#0641,<@!693425461774123068> Do you see AREC as a lo...,2020-10-08 04:57:59.234000+00:00,day-trading,Templar Trading,0,693425461774123068,,,,do you see arec as a longer hold based on a ...
11289,330206399542067210,TheTrueSigns(Day_Trader)#7112,"letters = [""A"", ""B"", ""C"", ""D"", ""E"", ""F"", ""G"", ...",2020-10-09 03:40:13.839000+00:00,main-trading,Templar Trading,0,,,,,letters a b c d e f g ...
12329,396702258529697793,Father#4214,Inverse H&S played out today,2020-10-13 03:23:06.356000+00:00,options-chat,BullTrades,0,,,,,inverse h s played out today
15082,712710686186274878,sare#9771,thats h ow you get blocked,2020-10-22 03:56:46.384000+00:00,trading-discussion,Legacy Trading,0,,,,,thats h ow you get blocked
15619,709787787955535912,mov#1337,Looks like potential C&H on JMIA so should run...,2020-10-23 21:49:30.747000+00:00,stocks-chat,BullTrades,0,,,,,looks like potential c h on jmia so should run...
24584,194656275882180608,Ex0rcist#0280,SPLK for a swing. Daily chart might have an in...,2020-11-21 01:33:11.696000+00:00,profits,Legacy Trading,0,,,,,splk for a swing daily chart might have an in...


In [886]:
token_df[lambda x: x.for_tokenizer.str.contains(r"disney")].iloc[-3, 2]

'FRX-U: SPAC lead by SHAQ:\n\nThe basketball superstar Shaquille O\'Neal, Martin Luther King III, and three former Disney executives are joining forces to back a new blank-check company.\nForest Road Acquisition Corp. aims to raise $250 million for deals in the tech and media space.\nPotential targets include intellectual property, "audience aggregation platforms," and "companies in need of capital,"'

In [ ]:
# have to identify companies by ticker or by name
# identify trades
# calculate a updated sentiment score based on the specialized language of the group/trading in general and emote usage. 

In [887]:
token_df = token_df.assign(named_companies = lambda x: x.content.apply(lambda x: ",".join([x for x in re.findall(company_regex, x)])))

In [888]:
c_dict = {}
for val in token_df.named_companies.values:
    if "," in val:
        for v in val.split(","):
            if v in c_dict.keys():
                c_dict[v] += 1
            else:
                c_dict[v] = 1
    else:
        if val in c_dict.keys():
            c_dict[val] += 1
        else:
            c_dict[val] = 1

In [889]:
company_starter_df = pd.DataFrame.from_dict(c_dict, orient="index", columns=["counts"]).reset_index().rename(columns={"index": "company"}).sort_values("counts", ascending=False).reset_index(drop=True)

In [890]:
company_starter_df.head(25)

,company,counts
0,,25292
1,SPY,166
2,TSLA,165
3,NIO,151
4,AMD,133
5,AAPL,114
6,ER,90
7,AH,86
8,DKNG,75
9,PLTR,74


In [891]:
company_starter_df.tail(40)

,company,counts
2291,HYG,1
2292,FUCKE,1
2293,ROCKE,1
2294,XLU,1
2295,EXC,1
2296,DSGT,1
2297,XLV,1
2298,FRTA,1
2299,QE,1
2300,SENAT,1


In [892]:
token_df[lambda x: x.named_companies.str.contains("ER")]

,id,name,content,timestamp,channel,server,isBot,mentions,emojis,links,chat_emotes,for_tokenizer,named_companies
283,579472664926879755,simo#6874,Crwd over 120+ after ER,2019-09-04 16:35:46.747000+00:00,wetlqd-ideas,Misc,0,,,,,crwd over 120 after er,ER
578,611594621222387738,BreadGame#4233,ER play. Sold too early but still a big win #MTN,2019-09-28 00:42:56.186000+00:00,wetlqd-ideas,Misc,0,,,,,er play sold too early but still a big win mtn,"ER,MTN"
793,623984968921776150,brycat23#0278,SLIGHT WERK,2019-10-21 19:32:02.200000+00:00,wetlqd-ideas,Misc,0,,,,,slight werk,"SLIGH,WERK"
802,595081393449467904,dumbskull#0584,Post ER trade this morning for quick profits,2019-10-22 15:16:30.915000+00:00,wetlqd-ideas,Misc,0,,,,,post er trade this morning for quick profits,ER
820,595081393449467904,dumbskull#0584,Tesla ER,2019-10-24 13:40:17.450000+00:00,wetlqd-ideas,Misc,0,,,,,tesla er,ER
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29578,752855855451471932,Brad Lyons#1111,I will play ER's when my research confirms it,2020-12-04 04:44:08.376000+00:00,options,LSV,0,,,,,i will play ers when my research confirms it,ER
29749,366373605853888515,baby#4110,"I’ve been VERY green this week, hb yall ?",2020-12-04 19:50:40.190000+00:00,general,OmegaTrades,0,,,,,ive been very green this week hb yall,VERY
30232,261566586060537857,TwitchyKibos#5632,TANK VERY.CN INSTEAD!,2020-12-05 23:30:35.028000+00:00,main-trading,Templar Trading,0,,,,,tank very cn instead,"TANK,VERY,CN,INSTE,AD"
30469,351907948815187971,Pennybags#4565,i stay away from Forex for now - and i would d...,2020-12-06 20:32:12.681000+00:00,trade-chat,Learn-n,0,,,,,i stay away from forex for now and i would d...,VERY


In [807]:
token_df.assign(isUpper = lambda x:x.content.apply(str.isupper))

,id,name,content,timestamp,channel,server,isBot,mentions,emojis,links,chat_emotes,for_tokenizer,named_companies,isUpper
23,445583268792893449,itsyoboyvic#7198,I took stamps .... and IAC both not looking go...,2019-08-08 03:31:33.793000+00:00,wetlqd-ideas,Misc,0,,,,,i took stamps and iac both not looking go...,IAC,False
26,194305752939102208,D.Ninan#3776,Sold AMD aug23rd call for a fat profit,2019-08-08 13:33:00.968000+00:00,wetlqd-ideas,Misc,0,,,,,sold amd aug23rd call for a fat profit,AMD,False
47,194305752939102208,D.Ninan#3776,MSFT profits im still not selling it tho just ...,2019-08-13 14:21:58.192000+00:00,wetlqd-ideas,Misc,0,,,,,msft profits im still not selling it tho just ...,MSFT,False
54,344157708523667457,dylan.patel#4692,$139.50 and ATH,2019-08-13 19:01:16.234000+00:00,wetlqd-ideas,Misc,0,,,,,139 50 and ath,ATH,False
56,194305752939102208,D.Ninan#3776,$139.50 and ATH? Wym,2019-08-13 19:05:27.213000+00:00,wetlqd-ideas,Misc,0,,,,,139 50 and ath wym,ATH,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31617,428921853403463713,Baraxton#0199,@everyone EDIT is popping off,2020-12-08 20:25:14.542000+00:00,trading,LSV,0,everyone,,,,edit is popping off,EDIT,False
31623,785007528152399913,nickb34#9557,$OXY broke the resistance line on Brad's chart,2020-12-08 20:26:51.453000+00:00,trading,LSV,0,,,,,oxy broke the resistance line on brad s chart,OXY,False
31626,594593113104842795,VelcroKarma#1526,if PIXY runs from being included in the Benzin...,2020-12-08 20:27:59.342000+00:00,trade-chat,Learn-n,0,,,,,if pixy runs from being included in the benzin...,PIXY,False
31637,184420431242067977,Dg#8853,<@428921853403463713> have you done a video in...,2020-12-08 20:39:17.246000+00:00,trading,LSV,0,428921853403463713,,,,have you done a video in gogo by any chance,GOGO,False


In [804]:
non_ticker_expressions = {
    "ath ": "all time High",
    "pdt": "pattern day trader",
    "rh": "RobinHood",
    "td": "TD Ameritrade", 
    "tos": "Think or Swim",
    "leg": "a trade",
    "pump": "overvalue",
    "exp": "expiration",
    "dd": "due diligence",
    "er": "earnings report"
}

In [9]:
with sql.connect("../data/interim/discord/discord.db") as con:
    total_df_w_new_cols = pd.read_sql("SELECT * FROM comments", con=con)

In [10]:
total_df_w_new_cols = total_df_w_new_cols.drop("pk", axis=1)

In [11]:
total_df_w_new_cols.timestamp = pd.to_datetime(total_df_w_new_cols.timestamp)

In [15]:
total_df_w_new_cols.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31934 entries, 0 to 31933
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype              
---  ------       --------------  -----              
 0   id           31934 non-null  object             
 1   name         31934 non-null  object             
 2   content      31934 non-null  object             
 3   timestamp    31934 non-null  datetime64[ns, UTC]
 4   channel      31934 non-null  object             
 5   server       31934 non-null  object             
 6   isBot        31934 non-null  int64              
 7   mentions     31934 non-null  object             
 8   emojis       31934 non-null  object             
 9   links        31934 non-null  object             
 10  chat_emotes  31934 non-null  object             
dtypes: datetime64[ns, UTC](1), int64(1), object(9)
memory usage: 2.7+ MB


In [16]:
total_df_w_new_cols

,id,name,content,timestamp,channel,server,isBot,mentions,emojis,links,chat_emotes
0,579205396166737921,beenpippinsincebeenpippin#4039,Play account today,2019-08-02 01:10:52.637000+00:00,wetlqd-ideas,Misc,0,,,,
1,579205396166737921,beenpippinsincebeenpippin#4039,@everyone today’s total. This doesn’t calculat...,2019-08-02 01:13:47.923000+00:00,wetlqd-ideas,Misc,0,everyone,,,
2,445583268792893449,itsyoboyvic#7198,Crack,2019-08-02 01:14:35.798000+00:00,wetlqd-ideas,Misc,0,,,,
3,445583268792893449,itsyoboyvic#7198,Big crack,2019-08-02 01:14:39.682000+00:00,wetlqd-ideas,Misc,0,,,,
4,445583268792893449,itsyoboyvic#7198,Heroin,2019-08-02 01:14:44.367000+00:00,wetlqd-ideas,Misc,0,,,,
...,...,...,...,...,...,...,...,...,...,...,...
31929,428921853403463713,Baraxton#0199,<@!644997307976122388> all of my shorter video...,2020-12-08 20:38:45.351000+00:00,trading,LSV,0,644997307976122388,,,
31930,184420431242067977,Dg#8853,<@428921853403463713> have you done a video in...,2020-12-08 20:39:17.246000+00:00,trading,LSV,0,428921853403463713,,,
31931,428921853403463713,Baraxton#0199,"<@!184420431242067977> nope, not yet",2020-12-08 20:39:32.831000+00:00,trading,LSV,0,184420431242067977,,,
31932,428921853403463713,Baraxton#0199,they're the airline wifi company right?,2020-12-08 20:39:38.597000+00:00,trading,LSV,0,,,,


In [20]:
total_df_w_new_cols[lambda x: x.content.str.contains(company_regex)]

,id,name,content,timestamp,channel,server,isBot,mentions,emojis,links,chat_emotes
23,445583268792893449,itsyoboyvic#7198,I took stamps .... and IAC both not looking go...,2019-08-08 03:31:33.793000+00:00,wetlqd-ideas,Misc,0,,,,
26,194305752939102208,D.Ninan#3776,Sold AMD aug23rd call for a fat profit,2019-08-08 13:33:00.968000+00:00,wetlqd-ideas,Misc,0,,,,
47,194305752939102208,D.Ninan#3776,MSFT profits im still not selling it tho just ...,2019-08-13 14:21:58.192000+00:00,wetlqd-ideas,Misc,0,,,,
54,344157708523667457,dylan.patel#4692,$139.50 and ATH,2019-08-13 19:01:16.234000+00:00,wetlqd-ideas,Misc,0,,,,
56,194305752939102208,D.Ninan#3776,$139.50 and ATH? Wym,2019-08-13 19:05:27.213000+00:00,wetlqd-ideas,Misc,0,,,,
...,...,...,...,...,...,...,...,...,...,...,...
31910,428921853403463713,Baraxton#0199,@everyone EDIT is popping off,2020-12-08 20:25:14.542000+00:00,trading,LSV,0,everyone,,,
31916,785007528152399913,nickb34#9557,$OXY broke the resistance line on Brad's chart,2020-12-08 20:26:51.453000+00:00,trading,LSV,0,,,,
31919,594593113104842795,VelcroKarma#1526,if PIXY runs from being included in the Benzin...,2020-12-08 20:27:59.342000+00:00,trade-chat,Learn-n,0,,,,
31930,184420431242067977,Dg#8853,<@428921853403463713> have you done a video in...,2020-12-08 20:39:17.246000+00:00,trading,LSV,0,428921853403463713,,,


In [25]:
company_regex = re.compile(r"[A-Z]{2,4}")

In [26]:
total_df_w_new_cols[lambda x: x.content.str.contains(company_regex)]

,id,name,content,timestamp,channel,server,isBot,mentions,emojis,links,chat_emotes
23,445583268792893449,itsyoboyvic#7198,I took stamps .... and IAC both not looking go...,2019-08-08 03:31:33.793000+00:00,wetlqd-ideas,Misc,0,,,,
26,194305752939102208,D.Ninan#3776,Sold AMD aug23rd call for a fat profit,2019-08-08 13:33:00.968000+00:00,wetlqd-ideas,Misc,0,,,,
47,194305752939102208,D.Ninan#3776,MSFT profits im still not selling it tho just ...,2019-08-13 14:21:58.192000+00:00,wetlqd-ideas,Misc,0,,,,
54,344157708523667457,dylan.patel#4692,$139.50 and ATH,2019-08-13 19:01:16.234000+00:00,wetlqd-ideas,Misc,0,,,,
56,194305752939102208,D.Ninan#3776,$139.50 and ATH? Wym,2019-08-13 19:05:27.213000+00:00,wetlqd-ideas,Misc,0,,,,
...,...,...,...,...,...,...,...,...,...,...,...
31910,428921853403463713,Baraxton#0199,@everyone EDIT is popping off,2020-12-08 20:25:14.542000+00:00,trading,LSV,0,everyone,,,
31916,785007528152399913,nickb34#9557,$OXY broke the resistance line on Brad's chart,2020-12-08 20:26:51.453000+00:00,trading,LSV,0,,,,
31919,594593113104842795,VelcroKarma#1526,if PIXY runs from being included in the Benzin...,2020-12-08 20:27:59.342000+00:00,trade-chat,Learn-n,0,,,,
31930,184420431242067977,Dg#8853,<@428921853403463713> have you done a video in...,2020-12-08 20:39:17.246000+00:00,trading,LSV,0,428921853403463713,,,


In [131]:
# Key Abbreviations that point to a trade, result or trading platform
trade_abbvs = {"ath ": "all time High",
 "pdt": "pattern day trader",
"rh": "RobinHood",
"td": "TD Ameritrade", 
"tos": "Think or Swim",
"leg": "a trade",
"pump": "overvalue",
"exp": "expiration",
"dd": "due diligence",
"er": "earnings report",
"pm": "pre-market", # opening
"ah": "after-hours", # trading hours,
"am": "after-market", # analogous to ah
"gg": "good game", # bot comment and normal internet positive abbv
"ta": "technical analysis",
"est": "eastern standard time", # stock engages timezones
"ipo": "intial public offering", # new stocks
"ev": "electric vehicles", # or expected value
"us": "United States",
"itm": "in the money", 
"otm": "out of the money",
}

In [30]:
total_df = total_df_w_new_cols.copy(deep=True)

In [32]:
companies_starter = add_specialized_token_column(company_regex, "content", "companies")

In [33]:
companies_starter = companies_starter[lambda x: x.companies != ""]

In [39]:
ticker_dict= {}

In [41]:
def getCompany(company_str):
    companies = company_str.split(",")
    for c in companies:
        if c not in ticker_dict.keys():
            ticker_dict[c] = 1
        else:
            ticker_dict[c]+=1
    return None

In [43]:
companies_starter.companies.apply(getCompany)

23       None
26       None
47       None
54       None
56       None
         ... 
31910    None
31916    None
31919    None
31930    None
31933    None
Name: companies, Length: 6483, dtype: object

In [49]:
company_counts = pd.DataFrame.from_dict(ticker_dict, "index", columns=["counts"]).reset_index().rename(columns={"index": "company"})

In [53]:
company_counts = company_counts.sort_values("counts", ascending=False, ignore_index=True)

In [101]:
company_counts.head(65)

,company,counts
0,SPY,167
1,TSLA,166
2,NIO,151
3,US,140
4,AMD,133
...,...,...
60,RKT,28
61,ES,27
62,JKS,25
63,COVI,25


In [103]:
companies_starter[lambda x: x.companies.str.contains("ES")]

,id,name,content,timestamp,channel,server,isBot,mentions,emojis,links,chat_emotes,companies
555,385261276986605568,Sporadic#0001,Put that in TESTIMONIALS,2019-09-26 22:31:06.803000+00:00,wetlqd-ideas,Misc,0,,,,,"TEST,IMON,IALS"
688,464449226521313292,TradingJEDI#2647,YES,2019-10-09 14:31:25.385000+00:00,wetlqd-ideas,Misc,0,,,,,YES
1004,623984968921776150,brycat23#0278,TWLO TO THE RESCUE,2019-11-11 19:36:50.705000+00:00,wetlqd-ideas,Misc,0,,,,,"TWLO,TO,THE,RESC,UE"
1848,432049873563680786,Bueller#0001,YESSSSSSSSSS <@!560675627271979020>,2020-02-19 19:44:07.028000+00:00,wetlqd-ideas,Misc,0,560675627271979020,,,,"YESS,SSSS,SSSS"
1881,340252069359255552,unclaimedhades#7455,"THANKS, YALL ARE GOOD COACHES!! <a:5769_Jotaro...",2020-02-22 05:28:56.993000+00:00,wetlqd-ideas,Misc,0,,,,,"THAN,KS,YALL,ARE,GOOD,COAC,HES"
...,...,...,...,...,...,...,...,...,...,...,...,...
31565,700394878462328832,Jaime#0289,/ES and /NQ are flying. No idea why equities a...,2020-12-08 16:03:05.514000+00:00,profits,Legacy Trading,0,,,,,"ES,NQ"
31854,361604519278411787,pardue#1925,AESE running,2020-12-08 20:13:36.592000+00:00,day-trading,Templar Trading,0,,,,,AESE
31858,396160546140979212,jmorri#6551,$AESE for anyone that was playing it just went...,2020-12-08 20:13:53.474000+00:00,main-trading,Templar Trading,0,,,,,AESE
31867,396160546140979212,jmorri#6551,"<@!693425461774123068>, you get any starters i...",2020-12-08 20:15:04.429000+00:00,day-trading,Templar Trading,0,693425461774123068,,,,AESE


In [132]:
trade_terms = [k.upper().strip() for k in list(trade_abbvs.keys())]

In [134]:
company_counts[lambda x: ~(x.company.isin(trade_terms))].shape

(2334, 2)

In [135]:
company_counts_real = company_counts[lambda x: ~(x.company.isin(trade_terms))]

In [137]:
company_counts_real.to_csv("../data/interim/trading_counts.csv", index=False)

In [139]:
company_counts_real

,company,counts
0,SPY,167
1,TSLA,166
2,NIO,151
4,AMD,133
5,AAPL,114
...,...,...
2348,NDS,1
2349,FRIE,1
2350,SAID,1
2351,BEIN,1


In [145]:
company_counts_real.head(50)

,company,counts
0,SPY,167
1,TSLA,166
2,NIO,151
4,AMD,133
5,AAPL,114
9,DKNG,75
10,PLTR,74
11,ETF,73
13,PT,61
14,ACB,60


In [171]:
companies_lowered = company_counts_real.company.apply(str.lower).values

In [184]:
top100 = company_counts_real.head(100).company.values

In [185]:
hundred_dict = {k: "" for k in top100}

In [47]:
hundred_dict = {'SPY': 'S&P 500',
 'TSLA': 'Telsa',
 'NIO': 'NIO Inc.',
 'AMD': 'Advanced Micro Devices',
 'AAPL': 'Apple',
 'DKNG': 'DraftKings',
 'PLTR': 'Palantir Technologies',
 'ETF': 'Exchange Traded Funds',
 'PT': 'Pintec Technology',
 'ACB': 'Aurora Cannabis',
 'ZM': 'Zoom',
 'BABA': 'Alibaba',
 'FB': 'Facebook',
 'BTC': 'Bitcoin',
 'BA': 'Boeing ',
 'VIX': '',
 'WKHS': '',
 'SQ': '',
 'LOL': '',
 'BYND': '',
 'NVDA': '',
 'IDEX': '',
 'SPAC': '',
 'KEKW': '',
 'AMZN': '',
 'PFE': '',
 'TO': '',
 'MSFT': '',
 'NKLA': '',
 'GNUS': '',
 'FSLY': '',
 'SHLL': '',
 'PTON': '',
 'RSI': '',
 'ROKU': '',
 'PINS': '',
 'CRSR': '',
 'NQ': '',
 'PLUG': '',
 'WWR': '',
 'SPAQ': '',
 'HYLN': '',
 'SNDL': '',
 'JD': 'JD.com',
 'SNAP': 'Snap Inc.',
 'RKT': '',
 'ES': '',
 'JKS': '',
 'COVI': '',
 'NNDM': '',
 'MACD': '',
 'GME': '',
 'SE': '',
 'IT': '',
 'MA': '',
 'SQQQ': '',
 'KO': '',
 'WSB': '',
 'XD': '',
 'FOMO': '',
 'MARA': '',
 'HEAR': '',
 'CRM': '',
 'CGC': '',
 'IZEA': '',
 'CBAT': '',
 'SL': '',
 'VWAP': '',
 'VXX': '',
 'APPL': '',
 'INTC': '',
 'SPCE': '',
 'AAL': '',
 'SPI': '',
 'ON': '',
 'AREC': '',
 'NFLX': '',
 'QQQ': '',
 'UONE': '',
 'BLNK': '',
 'HD': '',
 'ING': '',
 'FDA': '',
 'DIS': '',
 'MO': '',
 'OOOO': '',
 'THE': '',
 'TLRY': '',
 'ADTX': '',
 'YOU': '',
 'LMNL': '',
 'JMIA': '',
 'USD': '',
 'EXAS': '',
 'IV': '',
 'SOLO': '',
 'XERS': '',
 'DPW': '',
 'GILD': '',
 'GRAF': ''}

In [48]:
classifiable_companies[lambda x: x.Symbol == "WKHS"]

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry,exchange
3032,WKHS,Workhorse Group Inc. Common Stock,$5.515,0.155,2.892%,860263391.0,United States,NaN,1917372,Capital Goods,Auto Manufacturing,NASDAQ


In [49]:
classed_dict = {k: v for k, v in zip(classifiable_companies.Symbol.values, classifiable_companies.Name.values)}

In [51]:
for k in hundred_dict.keys():
    try:
        hundred_dict[k] = classed_dict[k]
    except KeyError:
        hundred_dict[k] = ""

In [52]:
empty_keys = []
for k, v in hundred_dict.items():
    if v == "":
        empty_keys.append(k)

In [53]:
len(empty_keys)

27

In [54]:
hundred_dict['SPY'] = "S&P 500"
hundred_dict['ETF'] = "Exchange Traded Fund"
hundred_dict['BTC'] = "Bitcoin"
hundred_dict['VIX'] = "Volatility Index"
hundred_dict['SHLL'] = "Tortoise Acquistion"
hundred_dict['NQ'] = "Non-Qualified Options"
hundred_dict['MACD'] = "Moving Average Convergence Divergence"
hundred_dict['SQQQ'] = "Short QQQ ETF"
hundred_dict['WSB'] = "Wallstreet Bets"
hundred_dict['VWAP'] = "Volume-Weighted Average Trading"
hundred_dict['VXX'] = "Vix Short"
hundred_dict['APPL'] = "Apple Inc."
hundred_dict['QQQ'] = "Invesco QQQ Trust Shares"
hundred_dict['USD'] = "United States Dollar"
hundred_dict['IV'] = "Implied Volatility"
hundred_dict['GRAF'] = "VLDR"

In [55]:
empty_keys = []
for k, v in hundred_dict.items():
    if v == "":
        empty_keys.append(k)

In [56]:
len(empty_keys)

11

In [57]:
empty_keys

['LOL', 'SPAC', 'KEKW', 'TO', 'COVI', 'XD', 'FOMO', 'SL', 'FDA', 'OOOO', 'THE']

In [58]:
# Write to find the 100 companies mentions and their lowercase counterparts.

In [59]:
hundred_dict

{'SPY': 'S&P 500',
 'TSLA': 'Tesla Inc. Common Stock',
 'NIO': 'NIO Inc. American depositary shares each  representing one Class A ordinary share',
 'AMD': 'Advanced Micro Devices Inc. Common Stock',
 'AAPL': 'Apple Inc. Common Stock',
 'DKNG': 'DraftKings Inc. Class A Common Stock',
 'PLTR': 'Palantir Technologies Inc. Class A Common Stock',
 'ETF': 'Exchange Traded Fund',
 'PT': 'Pintec Technology Holdings Limited American Depositary Shares',
 'ACB': 'Aurora Cannabis Inc. Common Shares',
 'ZM': 'Zoom Video Communications Inc. Class A Common Stock',
 'BABA': 'Alibaba Group Holding Limited American Depositary Shares each representing eight Ordinary share',
 'FB': 'Meta Platforms Inc. Class A Common Stock',
 'BTC': 'Bitcoin',
 'BA': 'Boeing Company (The) Common Stock',
 'VIX': 'Volatility Index',
 'WKHS': 'Workhorse Group Inc. Common Stock',
 'SQ': 'Square Inc. Class A Common Stock',
 'LOL': '',
 'BYND': 'Beyond Meat Inc. Common Stock',
 'NVDA': 'NVIDIA Corporation Common Stock',
 'IDEX

In [60]:
list(hundred_dict.keys())

['SPY',
 'TSLA',
 'NIO',
 'AMD',
 'AAPL',
 'DKNG',
 'PLTR',
 'ETF',
 'PT',
 'ACB',
 'ZM',
 'BABA',
 'FB',
 'BTC',
 'BA',
 'VIX',
 'WKHS',
 'SQ',
 'LOL',
 'BYND',
 'NVDA',
 'IDEX',
 'SPAC',
 'KEKW',
 'AMZN',
 'PFE',
 'TO',
 'MSFT',
 'NKLA',
 'GNUS',
 'FSLY',
 'SHLL',
 'PTON',
 'RSI',
 'ROKU',
 'PINS',
 'CRSR',
 'NQ',
 'PLUG',
 'WWR',
 'SPAQ',
 'HYLN',
 'SNDL',
 'JD',
 'SNAP',
 'RKT',
 'ES',
 'JKS',
 'COVI',
 'NNDM',
 'MACD',
 'GME',
 'SE',
 'IT',
 'MA',
 'SQQQ',
 'KO',
 'WSB',
 'XD',
 'FOMO',
 'MARA',
 'HEAR',
 'CRM',
 'CGC',
 'IZEA',
 'CBAT',
 'SL',
 'VWAP',
 'VXX',
 'APPL',
 'INTC',
 'SPCE',
 'AAL',
 'SPI',
 'ON',
 'AREC',
 'NFLX',
 'QQQ',
 'UONE',
 'BLNK',
 'HD',
 'ING',
 'FDA',
 'DIS',
 'MO',
 'OOOO',
 'THE',
 'TLRY',
 'ADTX',
 'YOU',
 'LMNL',
 'JMIA',
 'USD',
 'EXAS',
 'IV',
 'SOLO',
 'XERS',
 'DPW',
 'GILD',
 'GRAF']

In [61]:
terms_list = ['SPY', 'ETF' , 'BTC', 'VIX',
'NQ','MACD', 'SQQQ', 'WSB', 'VWAP','VXX', 
'APPL','QQQ','USD','IV'] 
#hundred_dict['SHLL'] = "Tortoise Acquistion"
#hundred_dict['GRAF'] = "VLDR"

In [62]:
skip_keys = empty_keys + terms_list

In [230]:
start_date = total_df_w_new_cols.timestamp[0].strftime("%Y-%m-%d")

In [248]:
end_date = (total_df_w_new_cols.timestamp[31933]+ dt.timedelta(1)).strftime('%Y-%m-%d') 

In [249]:
end_date

'2020-12-09'

In [63]:
y_tickers = [f for f in list(hundred_dict.keys()) if f not in skip_keys + empty_keys]

In [242]:
spy_ticker = yf.Ticker('SPY')

In [244]:
sample_df = yf.download('SPY', start=start_date, end=end_date)

[*********************100%***********************]  1 of 1 completed


In [261]:
stock_movements_df = pd.DataFrame()

In [262]:
for c in y_tickers:
    _temp_df = yf.download(c, start=start_date, end=end_date)
    _temp_df = _temp_df.assign(ticker=c).reset_index().rename(columns={"index": "date"})
    stock_movements_df = pd.concat([stock_movements_df, _temp_df], axis=0, ignore_index=True)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [265]:
stock_movements_df = stock_movements_df.rename(columns={"Adj Close": "Adj_Close"})

In [267]:
with sql.connect("../data/interim/stocks.db") as con:
    stock_movements_df.to_sql("daily", con=con, index_label="pk", if_exists="replace")

In [12]:
start_date = total_df_w_new_cols.timestamp[0].strftime("%Y-%m-%d")

In [13]:
end_date = (total_df_w_new_cols.timestamp[31933]+ dt.timedelta(1)).strftime('%Y-%m-%d') 

In [15]:
# Amount of Days between first and last comment collected
total_df_w_new_cols.timestamp[0] - total_df_w_new_cols.timestamp[31933]

Timedelta('-495 days +04:29:14.339000')

In [21]:
dt.datetime(2019, 8, 2, 16, 0, 0) - dt.datetime(2019, 8, 2, 9, 30, 0)

datetime.timedelta(seconds=23400)

In [27]:
CSV_URL = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol=IBM&interval=1min&slice=year1month1&apikey={ALPHA_VANTAGE_API_KEY}'

with requests.Session() as s:
    download = s.get(CSV_URL)
    decoded_content = download.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)

In [28]:
len(my_list)

8201

In [30]:
my_list[1]

['2021-12-23 19:55:00', '130.6', '130.6', '130.6', '130.6', '103']

In [31]:
my_list[-1]

['2021-11-29 04:01:00', '116.92', '116.92', '116.92', '116.92', '900']

In [32]:
CSV_URL = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol=IBM&interval=1min&slice=year2month12&apikey={ALPHA_VANTAGE_API_KEY}'

with requests.Session() as s:
    download = s.get(CSV_URL)
    decoded_content = download.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    new_list = list(cr)

In [37]:
new_list[-1]

['2020-01-07 06:21:00',
 '116.163603611',
 '116.163603611',
 '116.163603611',
 '116.163603611',
 '104']

In [65]:
[i for i, x in enumerate(y_tickers) if x == "GRAF"]

[74]

In [67]:
y_tickers.pop(74)

'GRAF'

In [68]:
y_tickers.append("VLDR")

In [81]:
len(my_list)

8201

In [185]:
CSV_URL = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol=IBM&interval=1min&slice=year2month6&apikey={ALPHA_VANTAGE_API_KEY}'

with requests.Session() as s:
    download = s.get(CSV_URL)
    decoded_content = download.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    new_list = list(cr)

In [103]:
new_list[-1]

['2020-07-06 04:01:00',
 '107.092161156',
 '107.092161156',
 '107.092161156',
 '107.092161156',
 '251']

In [105]:
new_list[1]

['2020-08-03 18:34:00',
 '110.253912854',
 '110.253912854',
 '110.253912854',
 '110.253912854',
 '104']

In [106]:
numbs = [6, 5, 4, 3, 2, 1]

In [180]:
[i for i, x in enumerate(y_tickers) if x == "TSLA"]

[0]

In [247]:
f = 0
for tick in ticker_set:
    print(f"Starting {tick}")
    for n in numbs:
        CSV_URL = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol={tick}&interval=1min&slice=year2month{n}&apikey={ALPHA_VANTAGE_API_KEY}'
        download = pd.read_csv(CSV_URL, sep=",", header=0, encoding="utf-8").assign(ticker = tick)    
        with sql.connect("../data/interim/stocks.db") as con:
            download.to_sql("minute", con=con, index=False, if_exists="append")
        f += 1
        if f % 5 == 0:
            print(f"Added {f} Sleeping...")
            time.sleep(68)
    print(f"Added {tick}")

Starting CGC
Added 5 Sleeping...
Added CGC
Starting SOLO
Added 10 Sleeping...
Added SOLO
Starting VLDR
Added 15 Sleeping...
Added VLDR
Starting SPI
Added 20 Sleeping...
Added SPI
Starting LMNL
Added 25 Sleeping...
Added 30 Sleeping...
Added LMNL
Starting EXAS
Added 35 Sleeping...
Added EXAS
Starting INTC
Added 40 Sleeping...
Added INTC
Starting NFLX
Added 45 Sleeping...
Added NFLX
Starting DIS
Added 50 Sleeping...
Added DIS
Starting ADTX
Added 55 Sleeping...
Added 60 Sleeping...
Added ADTX
Starting JMIA
Added 65 Sleeping...
Added JMIA
Starting AAL
Added 70 Sleeping...
Added AAL
Starting DPW
Added 75 Sleeping...
Added DPW
Starting CBAT
Added 80 Sleeping...
Added CBAT


In [211]:
with sql.connect("../data/interim/stocks.db") as con:
    download.to_sql("minute", con=con, index=False, if_exists="append")

0

In [248]:
with sql.connect("../data/interim/stocks.db") as con:
    t = pd.read_sql("SELECT * FROM minute WHERE ticker = 'TSLA'", con=con)

In [232]:
ticker_set = (set(y_tickers)).difference(set(t.ticker.unique()))

In [246]:
ticker_set = ticker_set.difference({"ING", "TLRY", "MARA", "YOU", "UONE", "IZEA","AREC","BLNK","HD","SPCE","SHLL","GILD","MO","HEAR","CRM","XERS", 'TSLA', "ON"})

In [251]:
t.time = pd.to_datetime(t.time)

In [252]:
t.time.min()

Timestamp('2020-07-06 04:01:00')

In [253]:
t.time.max()

Timestamp('2020-12-31 20:00:00')

In [257]:
[hundred_dict[y] for y in y_tickers if y != "VLDR" else hundred_dict["GRAF"]]

SyntaxError: invalid syntax (<ipython-input-257-50bced56fd36>, line 1)

In [7]:
news_dict = {'SPY': 'S&P 500',
 'TSLA': 'Tesla',
 'NIO': 'NIO',
 'AMD': 'Advanced Micro Devices',
 'AAPL': 'Apple',
 'DKNG': 'DraftKings',
 'PLTR': 'Palantir Technologies',
 'PT': 'Pintec Technology',
 'ACB': 'Aurora Cannabis',
 'ZM': 'Zoom',
 'BABA': 'Alibaba',
 'FB': 'Facebook',
 'BTC': 'Bitcoin',
 'BA': 'Boeing',
 'WKHS': 'Workhorse Group',
 'SQ': 'Square',
 'BYND': 'Beyond Meat',
 'NVDA': 'NVIDIA Corporation',
 'IDEX': 'Ideanomics',
 'AMZN': 'Amazon',
 'PFE': 'Pfizer',
 'MSFT': 'Microsoft',
 'NKLA': 'Nikola',
 'GNUS': 'Genius Brands',
 'FSLY': 'Fastly',
 'SHLL': 'Tortoise Acquistion',
 'PTON': 'Peloton',
 'RSI': 'Rush Street Interactive',
 'ROKU': 'Roku',
 'PINS': 'Pinterest',
 'CRSR': 'Corsair Gaming',
 'PLUG': 'Plug Power Inc.',
 'WWR': 'Westwater Resources',
 'SPAQ': 'Spartan Acquisition',
 'HYLN': 'Hyliion',
 'SNDL': 'Sundial Growers',
 'JD': 'JD.com',
 'SNAP': 'Snap Inc.',
 'RKT': 'Rocket Companies Inc.',
 'ES': 'Eversource Energy',
 'JKS': 'JinkoSolar',
 'NNDM': 'Nano Dimension',
 'GME': 'GameStop',
 'SE': 'Sea Limited',
 'IT': 'Gartner',
 'MA': 'Mastercard',
 'SQQQ': 'Short QQQ ETF',
 'KO': 'Coca-Cola Company',
 'MARA': 'Marathon Digital',
 'HEAR': 'Turtle Beach',
 'CRM': 'Salesforce',
 'CGC': 'Canopy Growth',
 'IZEA': 'IZEA Worldwide',
 'CBAT': 'CBAK Energy Technology Inc. Common Stock',
 'VXX': 'Vix Short',
 'APPL': 'Apple',
 'INTC': 'Intel',
 'SPCE': 'Virgin Galactic',
 'AAL': 'American Airlines',
 'SPI': 'SPI Energy',
 'ON': 'ON Semiconductor',
 'AREC': 'American Resources',
 'NFLX': 'Netflix',
 'QQQ': 'Invesco QQQ Trust Shares',
 'UONE': 'Urban One',
 'BLNK': 'Blink Charging',
 'HD': 'Home Depot',
 'ING': 'ING Group',
 'DIS': 'Disney',
 'MO': 'Altria',
 'TLRY': 'Tilray',
 'ADTX': 'Aditxt',
 'YOU': 'Clear Secure',
 'LMNL': 'Liminal BioSciences',
 'JMIA': 'Jumia Technologies',
 'EXAS': 'Exact Sciences',
 'SOLO': 'Electrameccanica Vehicles',
 'XERS': 'Xeris Biopharma',
 'DPW': 'Ault Global',
 'GILD': 'Gilead Sciences',
 'GRAF': 'Velodyne Lidar'}

In [8]:
set(news_dict.keys()).difference(set(daily_data.ticker.unique()))

{'APPL', 'BTC', 'GRAF', 'QQQ', 'SHLL', 'SPAQ', 'SPY', 'SQQQ', 'VXX', 'YOU'}

In [60]:
with sql.connect("../data/interim/stocks.db") as con:
    daily_data = pd.read_sql("SELECT * FROM daily", con=con, index_col="pk", parse_dates={"Date": "%Y-%m-%d"})
    minute = pd.read_sql("SELECT * FROM minute WHERE ticker = 'TSLA'", con=con, parse_dates={"time": "%Y-%m-%d %H:%M:%S"})

In [57]:
len(daily_data.ticker.unique())

71

In [ ]:
# Calc Volatility and Turnover

In [79]:
daily_data[lambda x: (x.ticker == "TSLA")]

,Date,Open,High,Low,Close,Adj_Close,Volume,ticker
pk,,,,,,,,
0,2019-08-02,46.270000,47.254002,45.846001,46.868000,46.868000,30682500.0,TSLA
1,2019-08-05,45.919998,46.273998,45.155998,45.664001,45.664001,35141500.0,TSLA
2,2019-08-06,46.375999,46.500000,45.150002,46.150002,46.150002,27821000.0,TSLA
3,2019-08-07,45.299999,46.714001,45.160000,46.683998,46.683998,23882500.0,TSLA
4,2019-08-08,46.889999,47.959999,46.529999,47.660000,47.660000,26371500.0,TSLA
...,...,...,...,...,...,...,...,...
337,2020-12-02,556.440002,571.539978,541.210022,568.820007,568.820007,47775700.0,TSLA
338,2020-12-03,590.020020,598.969971,582.429993,593.380005,593.380005,42552000.0,TSLA
339,2020-12-04,591.010010,599.039978,585.500000,599.039978,599.039978,29401300.0,TSLA


In [ ]:
for c in y_tickers:
    _temp_df = yf.download(c, start=start_date, end=end_date)
    _temp_df = _temp_df.assign(ticker=c).reset_index().rename(columns={"index": "date"})
    stock_movements_df = pd.concat([stock_movements_df, _temp_df], axis=0, ignore_index=True)

In [140]:
tsla = yf.ticker.Ticker("VXX")
dl = tsla.history("5y", start_date="2019-01-01", end_date="2021-12-31")
spy_info = tsla.info

In [131]:
set(news_dict.keys()).difference(set(daily_data.ticker.unique()))

{'APPL', 'BTC', 'GRAF', 'QQQ', 'SHLL', 'SPAQ', 'SPY', 'SQQQ', 'VXX', 'YOU'}

In [202]:
non_companies = ["BTC-USD", "QQQ","SPY", "VXX", "SQQQ"]

In [138]:
# list(daily_data.ticker.unique()) + ["BTC-USD", "QQQ", "VLDR", "SPY", "VXX"]

In [271]:
def create_daily_data(ticker):
    tick = yf.ticker.Ticker(ticker)
    historical_data = tick.history("5y")
    outstanding = tick.info.get("sharesOutstanding")
    if outstanding == None:
        outstanding = 1
    daily_close = historical_data["Close"]
    pct_change = daily_close.pct_change().fillna(0)
    periods = 2
    # calc volatility
    vola = (pct_change.rolling(periods).std() * np.sqrt(periods)).fillna(0)
    historical_data = historical_data.assign(Volatility = vola)
    historical_data = historical_data.assign(Turnover = lambda x: x.Volume / outstanding)
    historical_data = historical_data.assign(company = ticker)
    return historical_data.reset_index().round({"Volatility": 6, "Turnover": 6})

In [165]:
companies = list(daily_data.ticker.unique())

In [166]:
companies.append("VLDR")

In [ ]:
companies = companies + non_companies

In [177]:
for i, l in enumerate(companies):
    new_df = create_daily_data(l)
    full_daily_data = pd.concat([full_daily_data, new_df], axis=0, ignore_index=True)
    if i % 5 == 0:
        print(f"Finished with {i}")

Finished with 0
Finished with 5
Finished with 10
Finished with 15
Finished with 20
Finished with 25
Finished with 30
Finished with 35
Finished with 40


In [209]:
full_daily_data = full_daily_data.rename(columns={"Stock Splits": "Stock_Splits"})

In [211]:
with sql.connect("../data/interim/stocks.db") as con:
    full_daily_data.to_sql("daily", con=con, index_label="pk", if_exists="replace")

In [119]:
tick_sun = yf.ticker.Ticker("DIS")

In [120]:
analysis_df = tick_sun.analysis

22104 | INFO | Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
22104 | INFO | NumExpr defaulting to 8 threads.


In [428]:
analysis_df.T

Period,0Q,+1Q,0Y,+1Y,+5Y,-5Y
Max Age,1,1,1,1,1,1
End Date,2021-12-31 00:00:00,2022-03-31 00:00:00,2022-09-30 00:00:00,2023-09-30 00:00:00,NaT,NaT
Growth,NaN,NaN,0.901,0.311,0.3788,0.01592
Earnings Estimate Avg,0.64,1.11,3.86,5.06,NaN,NaN
Earnings Estimate Low,0.4,0.71,3.11,4.24,NaN,NaN
Earnings Estimate High,1.12,1.48,4.91,6.43,NaN,NaN
Earnings Estimate Year Ago Eps,NaN,NaN,2.03,3.86,NaN,NaN
Earnings Estimate Number Of Analysts,14,13,16,15,NaN,NaN
Earnings Estimate Growth,NaN,NaN,0.901,0.311,NaN,NaN
Revenue Estimate Avg,1.8783e+10,1.7702e+10,7.42695e+10,8.30456e+10,NaN,NaN


In [429]:
tick_sun.cashflow

,2021-10-02,2020-10-03,2019-09-28,2018-09-29
Change To Liabilities,2.410000e+09,-2.293000e+09,1.910000e+08,2.350000e+08
Total Cashflows From Investing Activities,-3.163000e+09,-3.637000e+09,-4.118000e+09,-5.336000e+09
Net Borrowings,-3.699000e+09,1.123300e+10,3.677000e+09,-2.583000e+09
Total Cash From Financing Activities,-4.385000e+09,8.480000e+09,-1.090000e+09,-8.843000e+09
Change To Operating Activities,1.710000e+08,-1.570000e+08,9.320000e+08,-9.270000e+08
Issuance Of Stock,4.350000e+08,3.050000e+08,3.180000e+08,2.100000e+08
Net Income,1.995000e+09,-2.864000e+09,1.105400e+10,1.259800e+10
Change In Cash,-1.951000e+09,1.249900e+10,1.300000e+09,9.100000e+07
Effect Of Exchange Rate,3.000000e+07,3.800000e+07,-9.800000e+07,-2.500000e+07
Total Cash From Operating Activities,5.567000e+09,7.618000e+09,6.606000e+09,1.429500e+10


In [430]:
tick_sun.balancesheet

,2021-10-02,2020-10-03,2019-09-28,2018-09-29
Intangible Assets,1.711500e+10,1.917300e+10,2.321500e+10,6.812000e+09
Total Liab,1.013850e+11,1.040370e+11,9.113200e+10,4.464300e+10
Total Stockholder Equity,8.855300e+10,8.358300e+10,8.887700e+10,4.877300e+10
Minority Interest,1.367100e+10,1.392900e+10,1.397500e+10,5.182000e+09
Other Current Liab,5.372000e+09,4.809000e+09,5.677000e+09,5.378000e+09
Total Assets,2.036090e+11,2.015490e+11,1.939840e+11,9.859800e+10
Common Stock,5.547100e+10,5.449700e+10,5.390700e+10,3.677900e+10
Other Current Assets,3.000000e+09,3.046000e+09,9.790000e+08,6.350000e+08
Retained Earnings,4.042900e+10,3.831500e+10,4.249400e+10,8.267900e+10
Other Liab,1.853900e+10,2.158100e+10,2.153000e+10,9.699000e+09


In [431]:
tick_sun.earnings

,Revenue,Earnings
Year,,
2018,59434000000,12598000000
2019,69607000000,11054000000
2020,65388000000,-2864000000
2021,67418000000,1995000000


In [432]:
tick_sun.get_calendar()

,0,1
Earnings Date,2022-02-09 10:59:00,2022-02-14 12:00:00
Earnings Average,0.64,0.64
Earnings Low,0.4,0.4
Earnings High,1.12,1.12
Revenue Average,18783000000,18783000000
Revenue Low,18069000000,18069000000
Revenue High,19900500000,19900500000


In [239]:
tick_sun.financials

,2021-10-02,2020-10-03,2019-09-28,2018-09-29
Research Development,None,None,None,None
Effect Of Accounting Charges,None,None,None,None
Income Before Tax,2.561e+09,-1.743e+09,1.3923e+10,1.4729e+10
Minority Interest,1.3671e+10,1.3929e+10,1.3975e+10,5.182e+09
Net Income,1.995e+09,-2.864e+09,1.1054e+10,1.2598e+10
Selling General Administrative,1.3684e+10,1.2382e+10,1.1349e+10,8.86e+09
Gross Profit,2.2287e+10,2.1508e+10,2.7546e+10,2.6708e+10
Ebit,3.492e+09,3.781e+09,1.203e+10,1.4837e+10
Operating Income,3.492e+09,3.781e+09,1.203e+10,1.4837e+10
Other Operating Expenses,None,None,None,None


In [433]:
tick_sun.sustainability

,Value
2021-9,
palmOil,False
controversialWeapons,False
gambling,False
socialScore,7.06
nuclear,False
furLeather,False
alcoholic,False
gmo,False
catholic,False


In [434]:
tick_sun.get_major_holders()

,0,1
0,0.12%,% of Shares Held by All Insider
1,66.52%,% of Shares Held by Institutions
2,66.60%,% of Float Held by Institutions
3,3944,Number of Institutions Holding Shares


In [435]:
recommend_df = tick_sun.recommendations

In [439]:
recommend_df[lambda x: x.index < dt.datetime(2020, 12, 31)].value_counts('Firm')

Firm
Morgan Stanley            17
Credit Suisse             16
Citigroup                 13
UBS                       12
Imperial Capital          12
Nomura                    11
Wells Fargo               11
BMO Capital               11
Stifel Nicolaus           11
Barclays                  10
JP Morgan                  9
Rosenblatt                 8
Loop Capital               8
Deutsche Bank              8
Guggenheim                 7
FBR Capital                6
RBC Capital                6
Wunderlich                 6
B. Riley FBR               5
Topeka                     5
Pivotal Research           5
Macquarie                  5
Goldman Sachs              5
Atlantic Equities          4
Bernstein                  3
Hilliard Lyons             3
Bank of America            3
Cowen & Co.                3
PiperJaffray               2
Argus Research             2
KeyBanc                    2
BTIG Research              2
CLSA                       2
MoffettNathanson           2
Consumer 

In [440]:
dis_info = tick_sun.info

In [441]:
dis_info

{'zip': '91521',
 'sector': 'Communication Services',
 'fullTimeEmployees': 152000,
 'longBusinessSummary': 'The Walt Disney Company, together with its subsidiaries, operates as an entertainment company worldwide. It operates through two segments, Disney Media and Entertainment Distribution; and Disney Parks, Experiences and Products. The company engages in the film and episodic television content production and distribution activities, as well as operates television broadcast networks under the ABC, Disney, ESPN, Freeform, FX, Fox, National Geographic, and Star brands; and studios that produces motion pictures under the Walt Disney Pictures, Twentieth Century Studios, Marvel, Lucasfilm, Pixar, and Searchlight Pictures banners. It also offers direct-to-consumer streaming services through Disney+, Disney+ Hotstar, ESPN+, Hulu, and Star+; sale/licensing of film and television content to third-party television and subscription video-on-demand services; theatrical, home entertainment, and 

In [442]:
tick_sun.get_institutional_holders()

,Holder,Shares,Date Reported,% Out,Value
0,"Vanguard Group, Inc. (The)",137572834,2021-09-29,0.0757,23273196327
1,Blackrock Inc.,119795456,2021-09-29,0.0659,20265797291
2,State Street Corporation,71939331,2021-09-29,0.0396,12169976625
3,Morgan Stanley,35294697,2021-09-29,0.0194,5970803891
4,State Farm Mutual Automobile Insurance Co,34404214,2021-09-29,0.0189,5820160882
5,"FMR, LLC",29338716,2021-09-29,0.0161,4963230585
6,"Geode Capital Management, LLC",28284577,2021-09-29,0.0156,4784901891
7,Price (T.Rowe) Associates Inc,26497603,2021-09-29,0.0146,4482599499
8,Bank of America Corporation,25996713,2021-09-29,0.0143,4397863938
9,Northern Trust Corporation,24632835,2021-09-29,0.0136,4167136696


In [463]:
tick_sun.recommendations[lambda x: x.index >= dt.datetime(2020, 12, 1)]

,Firm,To Grade,From Grade,Action
Date,,,,
2020-12-01 16:11:33,UBS,Neutral,,main
2020-12-02 16:32:17,Citigroup,Buy,,main
2020-12-09 09:40:28,KeyBanc,Overweight,,init
2020-12-09 12:47:44,Wells Fargo,Overweight,Equal-Weight,up
2020-12-09 14:50:13,Morgan Stanley,Overweight,,main
2020-12-09 15:27:57,Credit Suisse,Outperform,,main
2020-12-10 11:55:42,Truist Securities,Buy,,init
2020-12-11 09:41:57,KeyBanc,Overweight,,main
2020-12-11 16:49:05,Credit Suisse,Outperform,,main


In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.keys import Keys
import random

In [6]:
caps = DesiredCapabilities().CHROME

In [7]:
caps["pageLoadStrategy"] = "eager"

In [9]:
dates = []

In [21]:
# test_df = pd.DataFrame()

In [11]:
# ticker of company
# title of article
# category (news or opinion)
# content article content
# release_date
# provider
# url
# article_id

In [122]:
recommend_df = tick_sun.recommendations

In [135]:
test_df.groupby("ticker").count().sort_values("id")

,id,title,link,date,publisher,article
ticker,,,,,,
JKS,2,2,2,2,2,2
VLDR,3,3,3,3,3,3
SPI,4,4,4,4,4,4
VXX,4,4,4,4,4,4
CRSR,4,4,4,4,4,4
...,...,...,...,...,...,...
NFLX,608,608,608,608,608,608
PFE,612,612,612,612,612,612
GME,790,790,790,790,790,790


In [652]:
# # {"company": "HYLN","start": , "end": , "url":""}, # {"company": "LMNL","start": , "end": , "url":""}, # {"company": "BTC-USD","start": , "end": , "url":""},
co_array= [{"company": "TSLA", "start": 330, "stop": 185, "url": "https://www.investing.com/equities/tesla-motors-news"},
{"company": "NIO", "start": 23, "stop":13,  "url": "https://www.investing.com/equities/nio-inc-news"},
{"company":"AMD", "start":26, "stop":14,  "url": "https://www.investing.com/equities/adv-micro-device-news"},
{"company":"AAPL","start": 375, "stop":350,  "url": "https://www.investing.com/equities/apple-computer-inc-news"}, 
{"company":"DKNG", "start":22, "stop":8,  "url": "https://www.investing.com/equities/diamond-eagle-acquisition-corp-news"},
{"company":"PLTR", "start":9, "stop":1,  "url": "https://www.investing.com/equities/palantir-technologies-inc-news"},
{"company":"ZM", "start":29, "stop":12,  "url": "https://www.investing.com/equities/zoom-video-communications-news"},
{"company":"BABA", "start":100, "stop":45,  "url": "https://www.investing.com/equities/alibaba-news"},
{"company": "ACB","start": 10, "stop": 1, "url":"https://www.investing.com/equities/aurora-cannabis-news?cid=1055316"},
{"company": "CGC","start": 6, "stop": 1, "url":"https://www.investing.com/equities/tweed-marijuana-inc-news?cid=1057241"},
{"company": "IZEA","start": 1, "stop": 1, "url":"https://www.investing.com/equities/izea-inc-news"},
{"company": "CBAT","start": 1, "stop": 1, "url":"https://www.investing.com/equities/china-bak-battery-news"},
{"company":"BA", "start":160, "stop":50,  "url": "https://www.investing.com/equities/boeing-co-news"},
{"company":"WKHS", "start":3, "stop":1,  "url": "https://www.investing.com/equities/amp-holding-inc-news"},
{"company":"PFE", "start":250, "stop":238,  "url": "https://www.investing.com/equities/pfizer-news"},
{"company":"FB", "start":404, "stop":130, "url": "https://www.investing.com/equities/facebook-inc-news"},
{"company":"SQ", "start":20, "stop":7,  "url": "https://www.investing.com/equities/square-inc-news"},
{"company":"BYND", "start":21, "stop":8,  "url": "https://www.investing.com/equities/beyond-meat-inc-news"} ,
{"company":"NVDA", "start":52, "stop":23,  "url": "https://www.investing.com/equities/nvidia-corp-news"},
{"company":"AMZN", "start":335, "stop":155, "url": "https://www.investing.com/equities/amazon-com-inc-news"},
{"company": "MSFT","start": 220, "stop": 70, "url":"https://www.investing.com/equities/microsoft-corp-news"},
{"company":"NKLA", "start":13, "stop":3, "url": "https://www.investing.com/equities/nikola-corp-news/"},
{"company":"PTON", "start":30, "stop":12, "url":"https://www.investing.com/equities/peloton-interactive-inc-news"},
{"company":"ROKU", "start":16, "stop":7,  "url": "https://www.investing.com/equities/roku-news"},
{"company":"PINS", "start":15, "stop":7,  "url": "https://www.investing.com/equities/pinterest-inc-news"},
{"company": "DIS", "start": 57, "stop": 25, "url": "https://www.investing.com/equities/disney-news"},
{"company": "GNUS","start": 2, "stop": 1, "url":"https://www.investing.com/equities/genius-brands-intl.-news"},
{"company": "FSLY","start": 5, "stop": 1, "url":"https://www.investing.com/equities/fastly-inc-news"},
{"company": "RSI","start": 1, "stop": 1, "url":"https://www.investing.com/equities/dmy-technology-group-news"},
{"company": "CRSR","start": 1, "stop": 1, "url":"https://www.investing.com/equities/corsair-gaming-inc-news"},
{"company": "PLUG","start": 10, "stop": 1, "url":"https://www.investing.com/equities/plug-power-news"},
{"company": "WWR","start": 1, "stop": 1, "url":"https://www.investing.com/equities/uranium-resources-news"},
{"company": "SNDL","start": 5, "stop": 1, "url":"https://www.investing.com/equities/sundial-growers-inc-news"},
{"company": "JD","start": 35, "stop": 15, "url":"https://www.investing.com/equities/jd.com-inc-adr-news"},
{"company": "SNAP","start": 28, "stop": 10, "url":"https://www.investing.com/equities/snap-inc-news"},
{"company": "RKT","start": 2, "stop": 1, "url":"https://www.investing.com/equities/rocket-companies-inc-news"},
{"company": "ES","start": 3, "stop": 1, "url":"https://www.investing.com/equities/northeast-utilities-news"},
{"company": "JKS","start": 3, "stop": 1, "url":"https://www.investing.com/equities/jinkosolar-holding-comp-ltd-news"},
{"company": "NNDM","start": 2, "stop": 1, "url":"https://www.investing.com/equities/nano-dimension-ltd-news"},
{"company": "GME","start": 99, "stop": 20, "url":"https://www.investing.com/equities/gamestop-corp-news"},
{"company": "SE","start": 7, "stop": 1, "url":"https://www.investing.com/equities/sea-limited-news"},
{"company": "IT","start": 10, "stop": 1, "url":"https://www.investing.com/equities/gartner-news"},
{"company": "MA","start": 40, "stop": 10, "url":"https://www.investing.com/equities/mastercard-cl-a-news"},
{"company": "KO","start": 38, "stop": 8, "url":"https://www.investing.com/equities/coca-cola-co-news"},
{"company": "MARA","start": 4, "stop": 2, "url":"https://www.investing.com/equities/marathon-pa-news"},
{"company": "HEAR","start": 1, "stop": 1, "url":"https://www.investing.com/equities/parametric-sound-corp-news"},
{"company": "CRM","start": 38, "stop": 12, "url":"https://www.investing.com/equities/salesforce-com-news"},
{"company": "IZEA","start": 1, "stop": 1, "url":"https://www.investing.com/equities/izea-inc-news"},
{"company": "CBAT","start": 1, "stop": 1, "url":"https://www.investing.com/equities/china-bak-battery-news"},
{"company": "INTC","start": 58, "stop": 20, "url":"https://www.investing.com/equities/intel-corp-news"},
{"company": "SPCE","start": 16, "stop": 1, "url":"https://www.investing.com/equities/social-capital-hedosophia-news"},
{"company": "AAL","start": 75, "stop": 23, "url":"https://www.investing.com/equities/american-airlines-group-news"},
{"company": "SPI","start": 1, "stop": 1, "url":"https://www.investing.com/equities/spi-energy-co-ltd-news"},
{"company": "ON","start": 2, "stop": 1, "url":"https://www.investing.com/equities/on-semiconductor-news"},
{"company": "AREC","start": 2, "stop": 1, "url":"https://www.investing.com/equities/american-resources-news"},
{"company": "NFLX","start": 92, "stop": 31, "url":"https://www.investing.com/equities/netflix,-inc.-news"},
{"company": "UONE","start": 1, "stop": 1, "url":"https://www.investing.com/equities/radio-one-(a)-news"},
{"company": "BLNK","start": 4, "stop": 3, "url":"https://www.investing.com/equities/car-charging-group-news"},
{"company": "HD","start": 28, "stop": 11, "url":"https://www.investing.com/equities/home-depot-news"},
{"company": "ING","start": 7, "stop": 2, "url":"https://www.investing.com/equities/ing-group-nv-news"},
{"company": "MO","start": 10, "stop": 1, "url":"https://www.investing.com/equities/altria-group-news"},
{"company": "TLRY","start": 11, "stop": 4, "url":"https://www.investing.com/equities/tilray-inc-news"},
{"company": "ADTX","start": 1, "stop": 1, "url":"https://www.investing.com/equities/aditx-therapeutics-inc-news"},
{"company": "JMIA","start": 1, "stop": 1, "url":"https://www.investing.com/equities/jumia-technologies-ag-news"},
{"company": "EXAS","start": 3, "stop": 1, "url":"https://www.investing.com/equities/exact-sciences-co-news"},
{"company": "SOLO","start": 1, "stop": 1, "url":"https://www.investing.com/equities/electrameccanica-vehicles-news"},
{"company": "XERS","start": 1, "stop": 1, "url":"https://www.investing.com/equities/xeris-pharmaceuticals-news"},
{"company": "DPW","start": 13, "stop": 4, "url":"https://www.investing.com/equities/deutsche-post-news"},
{"company": "GILD","start": 30, "stop": 6, "url":"https://www.investing.com/equities/gilead-sciences-inc-news"},
{"company": "VLDR","start": 1, "stop": 1, "url":"https://www.investing.com/equities/graf-industrial-corp-news"},
{"company": "QQQ","start": 9, "stop": 1, "url":"https://www.investing.com/etfs/powershares-qqqq-news"},
{"company": "SPY","start": 190, "stop": 5, "url":"https://www.investing.com/etfs/spdr-s-p-500-news"},
{"company": "VXX","start": 1, "stop": 1, "url":"https://www.investing.com/etfs/ipath-series-b-sp500-vix-st-futures-news"},
{"company": "SQQQ","start": 1, "stop": 1, "url":"https://www.investing.com/etfs/ultrapro-short-qqq-news"}]

In [47]:
import re

In [57]:
pat = re.compile("\?cid=\d+")

In [88]:
driver = webdriver.Chrome("../../../Python/scraping/chromedriver2.exe", desired_capabilities=caps)
i = 5376
for c in co_array:
    base_url = c["url"]
    ending_url = ""
    if re.findall(pat, c["url"]) != []:
        ending_url = re.findall(pat, base_url)[0]
        base_url = re.sub(pat, "", base_url)
    starting_url = base_url + "/" + str(c["stop"]) + ending_url
    
    ticker = c["company"]
    shuffle = False
    if (c["start"] - c["stop"]) >= 100:
        shuffle = True
    elif c["start"] == c["stop"]:
        c["start"] = 2
    for k in range(c["stop"], c["start"], 1): # c["start"] - c["stop"]
        driver.get(starting_url)
        main_window = driver.window_handles[0]
        dater = driver.find_elements(By.CSS_SELECTOR, "section#leftColumn > div.mediumTitle1")
        
        for d in dater:
            articles = d.find_elements(By.TAG_NAME, "article")
            if shuffle:
                random.shuffle(articles)
                articles = articles[:3]

            for art in articles:    
                id = art.get_attribute("data-id")
                link = art.find_element(By.TAG_NAME, "a").get_attribute("href") # title #leftColumn > div.mediumTitle1 > article:nth-child(2) > div.textDiv > p
                details = art.find_element(By.TAG_NAME, "div")
                title = art.find_element(By.CLASS_NAME, "title").text
                if title == "":
                    next
                for j, s in enumerate(details.find_elements(By.TAG_NAME, "span")):
                    if j == 1:
                        publisher = s.text.replace("By", "").strip()
                    elif j == 2:
                        date = s.text.replace("-", "").strip()
                link = art.find_element(By.TAG_NAME, "a").get_attribute("href")
                if link.startswith("https://www.investing.com"):
                    driver.execute_script(f"window.open(\"{link}\",\"_blank\");")
                    driver.switch_to.window(window_name=driver.window_handles[1])
                    artsy = driver.find_element(By.CSS_SELECTOR, "div.articlePage").text
                    driver.close()
                    driver.switch_to.window(window_name=driver.window_handles[0])
                    test_df.loc[i, ["id", "title", "link", "date", "publisher", "article", "ticker"]] = [id, title, link, date, publisher, artsy, ticker]
                i += 1
                
        starting_url = base_url + "/" + str(k) + ending_url
        time.sleep(4)
    print(f"finished with {ticker}")
driver.quit()

ipykernel_launcher:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object


finished with MSFT
finished with NKLA
finished with PTON
finished with ROKU
finished with PINS
finished with DIS
finished with GNUS
finished with FSLY
finished with RSI
finished with CRSR
finished with PLUG
finished with WWR
finished with SNDL
finished with JD
finished with SNAP
finished with RKT
finished with ES
finished with JKS
finished with NNDM
finished with GME
finished with SE
finished with IT
finished with MA
finished with KO
finished with MARA
finished with HEAR
finished with CRM
finished with IZEA
finished with CBAT
finished with INTC
finished with SPCE
finished with AAL
finished with SPI
finished with ON
finished with AREC
finished with NFLX
finished with UONE
finished with BLNK
finished with HD
finished with ING
finished with MO
finished with TLRY
finished with ADTX
finished with JMIA
finished with EXAS
finished with SOLO
finished with XERS
finished with DPW
finished with GILD
finished with VLDR
finished with QQQ
finished with SPY
finished with VXX
finished with SQQQ


In [83]:
driver.quit()

In [91]:
test_df.to_csv("../data/interim/articlesCK.csv", index_label="pk")

In [109]:
re.sub(re.compile(".+\(Reuters\) - "),"",  test_df.loc[11202, "article"].replace("\n", ""))

"Gilead Sciences Inc (NASDAQ:GILD) on Thursday said its first-quarter product sales rose 16%, including $1.46 billion in sales of its COVID-19 antiviral drug remdesivir, but sales of flagship HIV and hepatitis C drugs were down due to the effects of the pandemic.The biotech company reported adjusted earnings of $2.08 per share, edging out the average analyst estimate of $2.07 as compiled by Refinitiv. Net income rose to $1.72 billion from $1.54 billion.The 16% rise in revenue for the quarter to $6.4 billion was still short of Wall Street estimates of $6.73 billion.Gilead said the COVID-19 pandemic continued to hurt sales of its treatments for hepatitis C and HIV, due to fewer people going to their doctors during the pandemic. It now expects a more gradual recovery in the market starting in the current quarter.Excluding remdesivir, its product sales fell 11% to $4.9 billion.Sale of remdesivir, which is sold under the brand name Veklury, were also below analysts' estimates of $1.56 billi

In [243]:
test_df.groupby("publisher").count().sort_values("id", ascending=False)#.id.describe()

,id,title,link,date,article,ticker
publisher,,,,,,
Reuters,5644,5644,5644,5644,5644,5644
Investing.com,4868,4868,4868,4868,4868,4868
Cointelegraph,400,400,400,400,400,400
Bloomberg,385,385,385,385,385,385
StockNews,334,334,334,334,334,334
DailyCoin,93,93,93,93,93,93
BTC Peers,67,67,67,67,67,67
CoinQuora,47,47,47,47,47,47
Seeking Alpha,17,17,17,17,17,17


In [231]:
sample_text_art = test_df.loc[4152, "article"].replace("\n", " ")#.replace("© Reuters. FILE PHOTO: American Airlines passenger planes crowd a runway where they are parked due to flight reductions to slow the spread of coronavirus disease (COVID-19), at Tulsa International Airport in Tulsa, Oklahoma, U.S. March 23, 2020. REUTERS/Nick Oxford/ By David Shepardson WASHINGTON (Reuters) -", "")

In [236]:
test_df.loc[test_df.publisher == "Reuters - Sep 08, 2020 5", "date"] = "Sep 08, 2020"
test_df.loc[test_df.publisher == "Reuters - Sep 08, 2020 5", "publisher"] = "Reuters"

In [238]:
test_df.loc[test_df.publisher == "Reuters - Jan 28, 2021", "date"] = "Jan 28, 2021"
test_df.loc[test_df.publisher == "Reuters - Jan 28, 2021", "publisher"] = "Reuters"

In [240]:
test_df.loc[test_df.publisher == "Reuters - Jul 29, 2020", "date"] = "Jul 29, 2020"
test_df.loc[test_df.publisher == "Reuters - Jul 29, 2020", "publisher"] = "Reuters"

In [242]:
test_df.loc[test_df.publisher == "Reuters - Oct 22, 2020", "date"] = "Oct 22, 2020"
test_df.loc[test_df.publisher == "Reuters - Oct 22, 2020", "publisher"] = "Reuters"

In [327]:
test_df[lambda x: x.publisher == "TipRanks"].iloc[4, 1]

'Why Plug Power Could Have Huge Upside Potential'

In [ ]:
# (Reuters) - 
# Investing.com - | --
# (Bloomberg) --
# StockNews => .+\n | copyright Reuters + Title\n
# DailyCoin => Title\n
# BTC Peers => Title\n
# CoinQuora => Title\n
# Seeking Alpha => Title\n
# copyright Reuters + Title\n

In [641]:
b = -1

In [642]:
r_regex = re.compile(r".*\(Reuters\) -")
iv_regex = re.compile(r".*Investing.com (–|--|-)")
bloom_regex = re.compile(r".*\(Bloomberg\) --")

In [643]:
def cleanArticle(article, title, index):
    global b
    b += 1
    i = index[b]
    article = article.replace("\n", " ").strip()
    title = title[i]
    article = re.sub(re.compile(r".*" + title), "", article)
    article = re.sub(r_regex, "", article)
    article = re.sub(iv_regex, "", article)
    article = re.sub(bloom_regex, "", article)
    return article.strip()

In [644]:
cleaned_articles = test_df.assign(articles2 = lambda x: x.article.apply(cleanArticle, title=x.title, index=x.article.index))

In [650]:
cleaned_articles.drop_duplicates().reset_index(drop=True).drop("article", axis=1).rename(columns={"articles2": "article"}).to_csv("../data/interim/articlesCK.csv", index_label="pk")

In [654]:
cleaned_articles = cleaned_articles.drop_duplicates().reset_index(drop=True).drop("article", axis=1).rename(columns={"articles2": "article"})

In [657]:
cleaned_articles.date = pd.to_datetime(cleaned_articles.date)

In [670]:
cleaned_articles.sort_values(["ticker", "date"]).groupby("ticker").agg(["first", "last", "count"]).date.sort_values("count", ascending=False).sort_values("first")

In [15]:
ticker = yf.ticker.Ticker("TSLA")

In [669]:
recommend_df.groupby(["Firm", "To Grade"]).count().sort_index(level=0).tail(20)

From Grade  Action
Firm              To Grade                          
RBC Capital       Sector Perform           3       3
                  Top Pick                 1       1
Rosenblatt        Buy                      8       8
                  Neutral                  1       1
Sterne Agee       Neutral                  1       1
Stifel Nicolaus   Buy                     10      10
                  Hold                     1       1
Tigress Financial Buy                      1       1
Topeka            Buy                      2       2
                  Hold                     2       2
                  Neutral                  1       1
Truist Securities Buy                      3       3
UBS               Buy                      9       9
                  Neutral                  5       5
Wells Fargo       Equal-Weight             2       2
                  Market Perform           1       1
                  Outperform               4       4
                  Overweight               8       8
Wunderlich        Buy                      1       1
                  Hold                     5       5

In [267]:
# Vanguard GICS Sectors
gics_sectors = {"Communication Services ETF":	"VOX",
"Consumer Discretionary ETF":"VCR",
"Consumer Staples":	"VDC",
"Energy ETF": "VDE",
"Financials ETF": "VFH",
"Health Care ETF": "VHT",
"Industrials ETF": "VIS",
"Information Technology ETF": "VGT",
"Materials ETF" : "VAW",
"Real Estate ETF": "VNQ",
"Utilities ETF": "VPU"}

In [279]:
full_daily_data = pd.DataFrame()

In [282]:
for i, l in enumerate(gics_sectors.values()):
    new_df = create_daily_data(l)
    full_daily_data = pd.concat([full_daily_data, new_df], axis=0, ignore_index=True)
    print(f"Finished with {l}")

Finished with VOX
Finished with VCR
Finished with VDC
Finished with VDE
Finished with VFH
Finished with VHT
Finished with VIS
Finished with VGT
Finished with VAW
Finished with VNQ
Finished with VPU


In [294]:
full_daily_data = full_daily_data.rename(columns={"Stock Splits": "Stock_Splits"})

In [17]:
with sql.connect("../data/interim/stocks.db") as con:
    company_dailys = pd.read_sql("SELECT * FROM daily", con=con)

In [291]:
full_daily_data.index.name = "pk"

In [305]:
daily_with_gics = pd.concat([company_dailys, full_daily_data], axis=0, ignore_index=True)

In [327]:
daily_with_gics["Date"] = pd.to_datetime(daily_with_gics["Date"])

In [332]:
with sql.connect("../data/interim/stocks.db") as con:
    daily_with_gics.to_sql("daily", con=con, index_label="pk", if_exists="replace")

In [343]:
company_dailys[lambda x: x.company == "AMZN"].drop("pk", axis=1)

,Date,Open,High,Low,Close,Volume,Dividends,Stock_Splits,Volatility,Turnover,company
17774,2017-01-03 00:00:00,757.919983,758.760010,747.700012,753.669983,3521100,0.0,0.0,0.0000,0.0070,AMZN
17775,2017-01-04 00:00:00,758.390015,759.679993,754.200012,757.179993,2510500,0.0,0.0,0.0047,0.0050,AMZN
17776,2017-01-05 00:00:00,761.549988,782.400024,760.260010,780.450012,5830100,0.0,0.0,0.0261,0.0116,AMZN
17777,2017-01-06 00:00:00,782.359985,799.440002,778.479980,795.989990,5986200,0.0,0.0,0.0108,0.0119,AMZN
17778,2017-01-09 00:00:00,798.000000,801.770020,791.770020,796.919983,3446100,0.0,0.0,0.0187,0.0068,AMZN
...,...,...,...,...,...,...,...,...,...,...,...
19028,2021-12-27 00:00:00,3420.739990,3458.860107,3384.310059,3393.389893,2934400,0.0,0.0,0.0084,0.0058,AMZN
19029,2021-12-28 00:00:00,3403.649902,3443.520020,3382.709961,3413.219971,2731900,0.0,0.0,0.0140,0.0054,AMZN
19030,2021-12-29 00:00:00,3416.800049,3424.239990,3372.010010,3384.020020,1787700,0.0,0.0,0.0144,0.0035,AMZN
19031,2021-12-30 00:00:00,3394.000000,3417.760010,3370.479980,3372.889893,1879200,0.0,0.0,0.0053,0.0037,AMZN


In [345]:
company_dailys.groupby("company").count().sort_values("Open", ascending=False)

,pk,Date,Open,High,Low,Close,Volume,Dividends,Stock_Splits,Volatility,Turnover
company,,,,,,,,,,,
VOX,3777,3777,3777,3777,3777,3777,3777,3777,3777,3777,3777
VDC,3777,3777,3777,3777,3777,3777,3777,3777,3777,3777,3777
VCR,3777,3777,3777,3777,3777,3777,3777,3777,3777,3777,3777
JD,2518,2518,2518,2518,2518,2518,2518,2518,2518,2518,2518
VDE,2518,2518,2518,2518,2518,2518,2518,2518,2518,2518,2518
...,...,...,...,...,...,...,...,...,...,...,...
RSI,426,426,426,426,426,426,426,426,426,426,426
ADTX,381,381,381,381,381,381,381,381,381,381,381
RKT,355,355,355,355,355,355,355,355,355,355,355


In [346]:
with sql.connect("../data/interim/discord/discord.db") as con:
    emote_df = pd.read_sql("SELECT * FROM comments WHERE emojis != ''", con=con)

In [351]:
emote_agg = {}
def agg_emojis(comma_sep_emotes):
    for e in comma_sep_emotes.split(","):
        if e in emote_agg.keys():
            emote_agg[e] += 1
        else:
            emote_agg[e] = 1

In [352]:
emote_df.emojis.apply(agg_emojis)

0       None
1       None
2       None
3       None
4       None
        ... 
2331    None
2332    None
2333    None
2334    None
2335    None
Name: emojis, Length: 2336, dtype: object

In [373]:
emote_agg_df = pd.Series(emote_agg).reset_index(name="count").rename(columns={"index": "emote"})

In [375]:
emoji_parse = emoji.UNICODE_EMOJI

In [383]:
emote_agg_df = emote_agg_df.assign(unicode_name = lambda e: e.emote.apply(lambda x: emoji_parse[x])).sort_values("count", ascending=False)

In [388]:
with sql.connect("../data/interim/discord/discord.db") as con:
    emote_agg_df.to_sql("chatEmotes", con=con, index_label='pk', if_exists="replace")

In [106]:
tickers_array = company_dailys.symbol.unique()

In [107]:
def stringNone(na):
    return na if na != None else ""

array(['SQQQ', 'VOX', 'VCR', 'VDC', 'VDE', 'VFH', 'VHT', 'VIS', 'VGT',
       'VAW', 'VNQ', 'VPU'], dtype=object)

In [113]:
keys = ["quoteType", "longName", "shortName", "logo_url", "sector", "industry", "holdings", "fundFamily", "category"]
k = 77
# company_info_df = pd.DataFrame()
for t in tickers_array[77:]:
    try:
        tick_info = yf.ticker.Ticker(t).get_info()
    except:
        print("Sleeping...")
        time.sleep(90)
        tick_info = yf.ticker.Ticker(t).get_info()
    
    for key in keys:
        company_info_df.loc[k, key] = stringNone(tick_info.get(key))
    k+= 1
    print(f"{t} Done.")

VOX Done.
VCR Done.
VDC Done.
VDE Done.
VFH Done.
VHT Done.
VIS Done.
VGT Done.
VAW Done.
VNQ Done.
VPU Done.


In [116]:
company_info_df.loc[:, "sector"] = company_info_df.sector.apply(lambda v: v if v != "" else np.nan).fillna(company_info_df.category)

In [117]:
company_info_df.loc[:, "symbol"] = pd.Series(tickers_array)

In [118]:
company_info_df[lambda x: (x.quoteType == "EQUITY")].sector.unique()

array(['Consumer Cyclical', 'Technology', 'Financial Services',
       'Healthcare', 'Communication Services', 'Industrials',
       'Consumer Defensive', 'Basic Materials', 'Utilities'], dtype=object)

In [119]:
company_info_df.iloc[-15:, :]

,quoteType,longName,shortName,logo_url,sector,industry,holdings,fundFamily,category,symbol
73,ETF,Invesco QQQ Trust,"Invesco QQQ Trust, Series 1",,Large Growth,,"[{'symbol': 'AAPL', 'holdingName': 'Apple Inc'...",Invesco,Large Growth,QQQ
74,ETF,SPDR S&P 500 ETF Trust,SPDR S&P 500,,Large Blend,,"[{'symbol': 'AAPL', 'holdingName': 'Apple Inc'...",SPDR State Street Global Advisors,Large Blend,SPY
75,ETF,iPath Series B S&P 500 VIX Short-Term Futures ETN,iPath Series B S&P 500 VIX Shor,,Trading--Miscellaneous,,[],Milleis Investissements Funds,Trading--Miscellaneous,VXX
76,ETF,ProShares UltraPro Short QQQ,ProShares UltraPro Short QQQ,,Trading--Inverse Equity,,"[{'symbol': '', 'holdingName': 'Nasdaq 100 Ind...",ProShares,Trading--Inverse Equity,SQQQ
77,ETF,Vanguard Communication Services Index Fund ETF...,Vanguard Communication Services,,Communications,,"[{'symbol': 'FB', 'holdingName': 'Facebook Inc...",Vanguard,Communications,VOX
78,ETF,Vanguard Consumer Discretionary Index Fund ETF...,Vanguard Consumer Discretion ET,,Consumer Cyclical,,"[{'symbol': 'AMZN', 'holdingName': 'Amazon.com...",Vanguard,Consumer Cyclical,VCR
79,ETF,Vanguard Consumer Staples Index Fund ETF Shares,Vanguard Consumer Staples ETF,,Consumer Defensive,,"[{'symbol': 'PG', 'holdingName': 'Procter & Ga...",Vanguard,Consumer Defensive,VDC
80,ETF,Vanguard Energy Index Fund ETF Shares,Vanguard Energy ETF,,Equity Energy,,"[{'symbol': 'XOM', 'holdingName': 'Exxon Mobil...",Vanguard,Equity Energy,VDE
81,ETF,Vanguard Financials Index Fund ETF Shares,Vanguard Financials ETF,,Financial,,"[{'symbol': 'JPM', 'holdingName': 'JPMorgan Ch...",Vanguard,Financial,VFH
82,ETF,Vanguard Health Care Index Fund ETF Shares,Vanguard Health Care ETF,,Health,,"[{'symbol': 'JNJ', 'holdingName': 'Johnson & J...",Vanguard,Health,VHT


In [120]:
sectors = {
    "Large Growth": "Technology",
    "Large Blend": "Mutual Fund",
    "Trading--Miscellaneous": "Inverse Equity",
    "Trading--Inverse Equity": "Inverse Equity",
    "Communications": "Communications Services",
    "Equity Energy": "Energy",
    "Financial": "Financials",
    "Financial Services": "Financials",
    "Health": "Healthcare",
    'Basic Materials': 'Materials',
    "Natural Resources": "Materials",
    "Consumer Defensive": "Consumer Staples",
    "Consumer Cyclical": "Consumer Discretionary",
}

In [121]:
company_info_df_w_sectors = company_info_df.copy(deep=True)
company_info_df_w_sectors.loc[:, "sector"] = company_info_df_w_sectors.loc[:, "sector"].apply(lambda x: x if x not in sectors.keys() else sectors[x])

In [122]:
company_info_df_w_sectors[lambda f: f.sector == ""]

,quoteType,longName,shortName,logo_url,sector,industry,holdings,fundFamily,category,symbol
72,CRYPTOCURRENCY,,Bitcoin USD,,,,,,,BTC-USD


In [123]:
company_info_df_w_sectors.loc[72, "logo_url"] = "https://bitcoin.org/img/icons/opengraph.png?1641218872"

In [127]:
cleaned_articles = pd.read_csv("../data/interim/articlesCK.csv")

In [136]:
gush = [1, 2, 3, 4, 5]

In [149]:
equitys = company_info_df_w_sectors[lambda x: x.quoteType == "EQUITY"].symbol.values

In [157]:
equitys[29:]

array(['WWR', 'HYLN', 'SNDL', 'JD', 'SNAP', 'RKT', 'ES', 'JKS', 'NNDM',
       'GME', 'SE', 'IT', 'MA', 'KO', 'MARA', 'HEAR', 'CRM', 'CGC',
       'IZEA', 'CBAT', 'INTC', 'SPCE', 'AAL', 'SPI', 'ON', 'AREC', 'NFLX',
       'UONE', 'BLNK', 'HD', 'ING', 'DIS', 'MO', 'TLRY', 'ADTX', 'LMNL',
       'JMIA', 'EXAS', 'SOLO', 'XERS', 'DPW', 'GILD', 'VLDR'],
      dtype=object)

In [174]:
# recommendation_history = pd.DataFrame()
for sym in equitys[29:]:
    try:
        recom = yf.ticker.Ticker(sym).get_recommendations()
    except:
        time.sleep(60)
        recom = yf.ticker.Ticker(sym).get_recommendations()
    
    if type(recom) != pd.DataFrame:
        next
    else:
        recom = recom.reset_index().assign(symbol=sym)
        recommendation_history = pd.concat([recommendation_history, recom], axis=0, ignore_index=True)
    
    print("Done with {}".format(sym))

Done with WWR
Done with HYLN
Done with SNDL
Done with JD
Done with SNAP
Done with RKT
Done with ES
Done with JKS
Done with NNDM
Done with GME
Done with SE
Done with IT
Done with MA
Done with KO
Done with MARA
Done with HEAR
Done with CRM
Done with CGC
Done with IZEA
Done with CBAT
Done with INTC
Done with SPCE
Done with AAL
Done with SPI
Done with ON
Done with AREC
Done with NFLX
Done with UONE
Done with BLNK
Done with HD
Done with ING
Done with DIS
Done with MO
Done with TLRY
Done with ADTX
Done with LMNL
Done with JMIA
Done with EXAS
Done with SOLO
Done with XERS
Done with DPW
Done with GILD
Done with VLDR


In [195]:
pd.concat([recommendation_history["To Grade"], recommendation_history["From Grade"]], axis=0).unique()

array(['Overweight', 'Hold', 'Buy', 'Neutral', 'Underperform',
       'Outperform', 'Equal-Weight', 'Sell', '', 'Underweight',
       'Sector Perform', 'Market Perform', 'Perform', 'Sector Weight',
       'In-Line', 'Underperformer', 'Market Outperform', 'Peer Perform',
       'Negative', 'Positive', 'Sector Outperform', 'Strong Buy',
       'Long-term Buy', 'Long-Term Buy', 'Equal-weight', 'Reduce',
       'Accumulate', 'Mixed', 'Fair Value', 'Trim', 'Outperformer',
       'Top Pick', 'Speculative Buy', 'Average', 'Market Underperform',
       'Market Weight', 'Below Average', 'Hold Neutral', 'Conviction Buy',
       'Sector Underperform', 'Sector Performer', 'Above Average'],
      dtype=object)

In [211]:
analyst_dict = {
    'Overweight': 'Bullish',
    'Hold': 'Neutral',
    'Buy': 'Very Bullish',
    'Neutral': 'Neutral',
    'Underperform': 'Bearish',
    'Outperform': 'Bullish',
    'Equal-Weight': 'Neutral',
    'Sell': 'Very Bearish',
    '': '',
    'Underweight': 'Bearish',
    'Sector Perform': 'Neutral',
    'Market Perform': 'Neutral',
    'Perform': 'Neutral', 'Sector Weight': 'Neutral',
    'In-Line': 'Neutral', 'Underperformer': 'Bearish',
    'Market Outperform': 'Bullish',
    'Peer Perform': 'Neutral',
    'Negative': 'Bearish',
    'Positive': 'Bullish',
    'Sector Outperform': 'Bullish',
    'Strong Buy': 'Very Bullish',
    'Long-term Buy': 'Very Bullish',
    'Long-Term Buy': 'Very Bullish',
    'Equal-weight': 'Neutral',
    'Reduce': 'Bearish',
    'Accumulate': 'Bullish',
    'Mixed': 'Neutral',
    'Fair Value': 'Neutral',
    'Trim': 'Bearish',
    'Outperformer': 'Bullish',
    'Top Pick': 'Very Bullish',
    'Speculative Buy': "Bullish",
    'Average': 'Neutral',
    'Market Underperform': 'Bearish',
    'Market Weight': 'Neutral',
    'Below Average': 'Bearish',
    'Hold Neutral': 'Neutral',
    'Conviction Buy': 'Very Bullish',
    'Sector Underperform': 'Bearish',
    'Sector Performer': 'Neutral',
    'Above Average': 'Bullish'
}

In [199]:
recommendation_history.shape

(9388, 6)

In [210]:
recommendation_history = recommendation_history[lambda x: x["To Grade"] != ""]

In [215]:
recommendation_history = recommendation_history.assign(new_grade = lambda x: x["To Grade"].apply(lambda c: analyst_dict[c])).assign(prev_grade = lambda x: x["From Grade"].apply(lambda c: analyst_dict[c]))

In [217]:
recommendation_history.Action.unique()

array(['main', 'down', 'init', 'up', 'reit'], dtype=object)

In [242]:
yt = recommendation_history.copy(deep=True)

In [250]:
up_and_down = {'up': {'Very Bullish': 'Bullish', 'Bullish': 'Neutral', 'Neutral': 'Bearish', 'Bearish': 'Very Bearish'},
'down': {'Bullish': 'Very Bullish', 'Neutral': 'Bullish', 'Bearish': 'Neutral', 'Very Bearish': 'Bearish'}}

In [251]:
for i, r in recommendation_history.iterrows():
    grade = r["prev_grade"]
    act = r["Action"]
    if grade == '':
        if act in ['main', 'reit', 'init']:
            recommendation_history.loc[i, 'prev_grade'] = r['new_grade']
        elif act == 'down':
            recommendation_history.loc[i, 'prev_grade'] = up_and_down['down'][r['new_grade']]
        elif act == 'up':
            recommendation_history.loc[i, 'prev_grade'] = up_and_down['up'][r['new_grade']]

In [ ]:
# recommendation_history, company_info_df_w_sectors, daily table from stocks and articlesCK

In [260]:
with sql.connect('../data/interim/stocks.db') as con:
    dailys_df = pd.read_sql("SELECT * FROM daily", con=con)

In [264]:
dailys_df = dailys_df.drop('pk', axis=1)

In [266]:
cleaned_articles = cleaned_articles.drop('pk', axis=1)

In [271]:
mentions = pd.read_csv('../data/interim/trading_counts.csv')

In [273]:
mentions = mentions.rename(columns={'company': 'symbol'})

In [278]:
cleaned_articles = cleaned_articles.rename(columns={'ticker': 'symbol'})

In [281]:
dailys_df = dailys_df.rename(columns={'company': 'symbol'})

In [ ]:
dailys_df.Date = pd.to_datetime(dailys_df.Date)

In [290]:
cleaned_articles.date = pd.to_datetime(cleaned_articles.date)

In [296]:
with sql.connect('../data/raw/companies.db') as con:
    dailys_df.to_sql("daily", con=con, index=True, index_label='pk', if_exists='replace')
    recommendation_history.to_sql('recommendations',  con=con, index=True, index_label='pk', if_exists='replace')
    company_info_df_w_sectors.drop('holdings', axis=1).to_sql('info', con=con, index=True, index_label='pk', if_exists='replace')
    cleaned_articles.to_sql('info', con=con, index=True, index_label='pk', if_exists='replace')
    mentions.to_sql('mentions', con=con, index=True, index_label='pk', if_exists='replace')

In [11]:
driver = webdriver.Chrome("../../../Python/scraping/chromedriver2.exe", desired_capabilities=caps)
driver.get("https://seekingalpha.com/symbol/HEAR/news")
time.sleep(4.2)
butt = driver.find_element(By.CSS_SELECTOR, 'button[data-test-id="header-button-sign-in"]')
butt.click()
but = driver.find_element(By.CSS_SELECTOR, 'button[data-test-id="sign-in-with-google"]')
time.sleep(4.2)
but.click()
email = driver.find_element(By.CSS_SELECTOR, '[type="email"]')
time.sleep(22.3)
email.send_keys("kylel9815@gmail.com")
time.sleep(10)
driver.quit()

C:\Users\Kyle\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  """Entry point for launching an IPython kernel.


In [12]:
driver.quit()

In [27]:
import bs4
import pyautogui

In [67]:
bs4

In [163]:
with open('sqqq.htm', encoding='utf-8') as fp:
    soup = bs4.BeautifulSoup(fp.read(), 'html5lib')
    for city in soup.find_all('h3'):
        print(city.text)

Nasdaq 100 slides at the open, which means three inverse QQQ funds are rallying
BofA fund manager survey triggers first sell signal since just before February crash
Earnings season was a smasher amid election 2020 distraction
QQQs head lower after hours as Apple and Amazon slip post-earnings


In [98]:
copied = "?from=2020-05-01T16%3A00%3A00.000Z&to=2021-09-01T16%3A00%3A00.000Z"
pyautogui.moveTo(2499, 135, duration=2)
pyautogui.click()
pyautogui.typewrite(['a', 'm', 'z', 'n', 'enter'], .5)
pyautogui.moveTo(2406, 433, duration=2)
pyautogui.click()
pyautogui.moveTo(2424, 47, duration=1.2)
pyautogui.click(clicks=3, interval=1)
pyautogui.hotkey('ctrl', 'v')
pyautogui.hotkey('enter')
time.sleep(5)
for l in range(1000):
    pyautogui.keyDown('pagedown')
    if l % 200 == 0:
        print(f'{l} at {dt.datetime.fromtimestamp(time.time()).strftime("%I:%M:%S")}')
pyautogui.moveTo(2079, 590, duration=1.2)
pyautogui.hotkey('ctrl', 's')
time.sleep(2)
pyautogui.typewrite(['a', 'm', 'z', 'n', '.', 'm', 'h', 't', 'm', 'l', 'enter'], .15)
time.sleep(2)
pyautogui.hotkey('enter')
time.sleep(3)
pyautogui.hotkey('home')
time.sleep(2)

0 at 02:43:15
100 at 02:43:26
200 at 02:43:37
300 at 02:43:48


In [128]:
with sql.connect('../data/interim/companies.db') as con:
    company_info_df_w_sectors.drop('holdings', axis=1).to_sql("info", con=con, index=True, index_label="pk", if_exists="append")

In [161]:
company_info_df_w_sectors[lambda x: ~(x.symbol.apply(str.lower).isin(['amzn', 'dis', 'wwr', 'pltr', 'dkng', 'amd', 'nio', 'nvda', 'snap', 'pins', 'izea', 'nflx', 'aapl', 'tsla']))].iloc[1+3+3+8+2+3+3+3+2+4+3+3+5+5+3:, -1]

65       JMIA
66       EXAS
67       SOLO
68       XERS
69        DPW
70       GILD
71       VLDR
72    BTC-USD
73        QQQ
74        SPY
75        VXX
76       SQQQ
77        VOX
78        VCR
79        VDC
80        VDE
81        VFH
82        VHT
83        VIS
84        VGT
85        VAW
86        VNQ
87        VPU
Name: symbol, dtype: object

In [271]:
for folder, sf, files in os.walk(r"H:\Big Data\articles"):
    for f in files[1:2]:
        if f.endswith(".htm"):
            f_name = "H:\\Big Data\\articles\\"  + f
            with open(f_name, encoding='latin-1') as file:
                soup = bs4.BeautifulSoup(file.read(), 'html5lib')
                print("="*18)
                print(f.replace(".htm", "").upper())
                print("="*18)
                for headline in soup.find_all('div', {'div':'[data-test-id="content-container"]'}):
                    print(headline.text.replace("\n", " "))

AAPL


In [273]:
soup = bs4.BeautifulSoup(open("H:\\Big Data\\articles\\aapl.htm", encoding='latin-1').read(), 'html5lib')

In [291]:
len(articles)

772

In [289]:
# soup.select('div[data-test-id="content-container"]') # article content
# soup.select('h3[data-test-id="post-list-item-title"]') # title
# soup.select('span[data-test-id="post-list-comments"]') # engagement
# soup.select('span[data-test-id="post-list-date"]') # date
# soup.select('span[data-test-id="post-list-author"]') # author

In [343]:
crypt_articles = pd.DataFrame()
columns = ['content', 'headline', 'link', 'comments', 'date',  'symbol', 'publisher']
art_links = []
o = 0
for folder, sf, files in os.walk(r"H:\Big Data\articles"):
    for file in files:
        if file.endswith(".htm"):
            art_links.append(file)

for sym in art_links:
    soup = bs4.BeautifulSoup(open(f"H:\\Big Data\\articles\\{sym}", encoding='latin-1').read().replace("\n", ""), 'html5lib')
    sym = sym.replace(".htm", "")
    articles = soup.select('article[data-test-id="post-list-item"]') 
    for a in articles:
        cont =  a.select('div[data-test-id="content-container"]') # article content
        headline = a.select('h3[data-test-id="post-list-item-title"]') # title
        comms = a.select('a[data-test-id="post-list-comments"]') # engagement
        date_art = a.select('span[data-test-id="post-list-date"]') # date
        link =  a.select('h3[data-test-id="post-list-item-title"]')[0].select('a')[0].attrs['href'] # article link
        symbol = sym
        publisher = 'SA'
        scrapes = [cont, headline, comms, date_art]
        scraped = []
        for s in scrapes:
            if len(s) != 0:
                scraped.append(s[0].text)
            else:
                scraped.append('')
        
        
        crypt_articles.loc[o, ['article', 'headline', 'comments', 'date', 'link', 'symbol', 'publisher']] = scraped + [link, sym.upper(), 'Seeking Alpha']
        o+=1
        

In [301]:
with sql.connect('../data/interim/companies.db') as con:
    arts = pd.read_sql("select * from articles", index_col='pk', con=con)

In [7]:
crypt_articles = crypt_articles.assign(comments = lambda x: x.comments.apply(str)).assign(comments = lambda x: x.comments.apply(lambda c: re.sub(r"Comment(s)?", "", c)))

In [8]:
crypt_articles = crypt_articles.assign(headline = lambda x: x.headline.apply(str.strip)).assign(article = lambda x: x.article.apply(str.strip))

In [9]:
crypt_articles.loc[:, "date"] = pd.to_datetime(crypt_articles.date)

In [20]:
crypt_articles = crypt_articles.assign(article = lambda x: x.article.apply(bytes, encoding='utf-8')).assign(article = lambda x: x.article.apply(cryptor.encrypt))

In [23]:
crypt_articles = crypt_articles.assign(headline = lambda x: x.headline.apply(bytes, encoding='utf-8')).assign(headline = lambda x: x.headline.apply(cryptor.encrypt))

In [24]:
crypt_articles = crypt_articles.assign(link = lambda x: x.link.apply(bytes, encoding='utf-8')).assign(link = lambda x: x.link.apply(cryptor.encrypt))

In [29]:
crypt_articles = crypt_articles.assign(comments = lambda x: x.comments.apply(bytes, encoding='utf-8')).assign(comments = lambda x: x.comments.apply(cryptor.encrypt))

In [31]:
crypt_articles = crypt_articles.assign(date = lambda x: x.date.apply(str).apply(bytes, encoding='utf-8')).assign(date = lambda x: x.date.apply(cryptor.encrypt))

In [33]:
crypt_articles = crypt_articles.assign(publisher = lambda x: x.publisher.apply(str).apply(bytes, encoding='utf-8')).assign(publisher = lambda x: x.publisher.apply(cryptor.encrypt))

In [35]:
crypt_articles = crypt_articles.assign(symbol = lambda x: x.symbol.apply(str).apply(bytes, encoding='utf-8')).assign(symbol = lambda x: x.symbol.apply(cryptor.encrypt))

In [57]:
with sql.connect('../data/raw/crypt.db') as con:
    crypt_articles.to_sql("crypt_articles", index=True,index_label='pk', con=con, if_exists='replace')

#### Find Slang 

In [5]:
with sql.connect('../data/interim/companies.db') as con:
    articles = pd.read_sql("select * from articles", index_col="pk", con=con)

In [113]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [10]:
articles

,id,title,link,date,publisher,symbol,article
pk,,,,,,,
0,2467735,Tesla tells China car cameras not activated ou...,https://www.investing.com/news/economy/tesla-s...,2021-04-07 00:00:00,Reuters,TSLA,Cameras in Tesla (NASDAQ:TSLA) cars are not ac...
1,2469136,Tesla lashes out at German red tape ahead of p...,https://www.investing.com/news/stock-market-ne...,2021-04-08 00:00:00,Reuters,TSLA,Tesla (NASDAQ:TSLA) sharply criticised lengthy...
2,2467806,"Sun, sea and cybernauts: the long road for Gre...",https://www.investing.com/news/technology-news...,2021-04-07 00:00:00,Reuters,TSLA,"From her garden on a sunny Greek island, Rowen..."
3,2467802,CEO Mary Barra bets GM can grow beyond cars an...,https://www.investing.com/news/stock-market-ne...,2021-04-07 00:00:00,Reuters,TSLA,Pam Fletcher wants to change the way General M...
4,2467233,"S&P 500 Flirts With Record, But 'Overbought' J...",https://www.investing.com/news/stock-market-ne...,2021-04-06 00:00:00,Investing.com,TSLA,The S&P 500 continued to trade near record hig...
...,...,...,...,...,...,...,...
11479,1759417,Stocks lower ahead of action-packed week,https://www.investing.com/news/stock-market-ne...,2019-01-28 00:00:00,Seeking Alpha,SQQQ,Global shares are starting the week on the bac...
11480,1748359,"Stocks weather Brexit debacle, pound steadies",https://www.investing.com/news/stock-market-ne...,2019-01-16 00:00:00,Seeking Alpha,SQQQ,Markets and the pound appear to have weathered...
11481,1746970,Equities boosted by China stimulus announcement,https://www.investing.com/news/stock-market-ne...,2019-01-15 00:00:00,Seeking Alpha,SQQQ,Stocks gained ground overnight as China announ...


In [114]:
sia = SentimentIntensityAnalyzer()

In [115]:
def sentiment_art(art):
    return sia.polarity_scores(art.replace("  ", '').replace('\x92', "'"))

In [21]:
articles = articles.assign(compound_sent = lambda x: x.article.apply(sentiment_art))

In [32]:
articles= articles.assign(pos_sent = lambda x: x.compound_sent.apply(lambda y: y['pos']))\
    .assign(neg_sent = lambda x: x.compound_sent.apply(lambda y: y['neg']))\
        .assign(neu_sent = lambda x: x.compound_sent.apply(lambda y: y['neu']))\
            .assign(comp_sent = lambda x: x.compound_sent.apply(lambda y: y['compound']))

In [34]:
articles.drop("compound_sent", axis=1, inplace=True)

In [36]:
with sql.connect('../data/raw/crypt.db') as con:
    crypt_articles = pd.read_sql("select * from crypt_articles", index_col="pk", con=con)

In [85]:
crypt_articles = crypt_articles.assign(sentiment = lambda x: x.article.apply(cryptor.decrypt).apply(str, encoding='utf-8').apply(sentiment_art))

In [86]:
crypt_articles = crypt_articles.assign(pos_sent = lambda x: x.sentiment.apply(lambda y: y['pos']))\
    .assign(neg_sent = lambda x: x.sentiment.apply(lambda y: y['neg']))\
        .assign(neu_sent = lambda x: x.sentiment.apply(lambda y: y['neu']))\
            .assign(comp_sent = lambda x: x.sentiment.apply(lambda y: y['compound']))

In [87]:
crypt_articles.drop("sentiment", axis=1, inplace=True)

In [88]:
crypt_articles

,article,headline,comments,date,link,symbol,publisher,pos_sent,neg_sent,neu_sent,comp_sent
pk,,,,,,,,,,,
0,b'GPRLBtWdnY_RZcIhFw1I_zO6kgjSRPGbPiJtEEq0Vfs=...,b'Jqc6cDZhIcu6aNfkqyzfELn7KxIpIge0W1jApWaJ_iQ=...,b'YlLzFOG-Xec6-61v5jY0E2pD1Eo572iW8WYx99XeBfE=...,b'CuGllAvrxyYSr0-fB0EuU07l-QRZIQEdbewugAs0kgM=...,b'qaEtC-ufuGelB1bHBC1wVj_gVPxagArmLAVwkzeRc1A=...,b'2N7lFkxliIikw5YxhCE-xZZuoOBcsNzZ7w_8q8WJh24=...,b'iEqM7yp5q1dD9E5ffh-q3I8i3Ot9RN-9TW6mHdb7Hik=...,0.111,0.029,0.861,0.9325
1,b'wIN9CiTc_XLewNlpYPGLvPRbCWRV6d2eIoUPXS-j-Ko=...,b'MmssHZw5N6QiPfWPe_1sS53s4vLgtpnmul7xh63617I=...,b'C--kMS-JiqtjT9irkZM9ArUCXRICre3raTElsaqfKyk=...,b'Ol_501yn3fCC0QCcOjLUtnrO8Ix-BXy-z1BmDtI7Agk=...,b'CRZ4uDkFkfGBUd2un4QtzL4qbgOgdQmdHjzrg85uSLI=...,b'e4_N0A8W-YST2-PPh5bPDojLjDRVgx6LswFDTfg8mqQ=...,b'4vqaOCcT1K6yPtnKEj_35QcyK6PO8fdhgTtFFac61NQ=...,0.073,0.024,0.904,0.8360
2,b'PK1nnYlzL3TekPT49yIf75Ao9zjkAcZK-IfbvnBMLl8=...,b'dSI3TD_qWPddyrgY1lLKfq6GPKHo-nUIWsbmHAVCl8U=...,b'fNJoAgeOc0YqoVRXEX0Wt4DuTTb2UvojOG6nPDWBA0w=...,b'P-P5di0wLHphlX_MkXnaacCaJPVY4ggKxc7aX_hov3s=...,b'0U7mMfkmVgtoosoO_VZukRA0K7JWnaG42jEQnytwT_E=...,b'vUme-p7pQwYUp4k4lnwqD3HeJLWFql4XMGgWxt5NGXI=...,b'3iYHSCuqjIl1c_e9PG8yPpjtvxhIF5s9huWS5QE-WT4=...,0.070,0.051,0.879,0.5106
3,b'CPMuf42Elv6xCSyBMukTma8Qv6la83EgjcjNiXFTwxU=...,b'hG9-8gAJzpPFcJoriTRXfGtkd9iLHqUDJEHR9qBdY8o=...,b'zyelglAChrb2pGJBkxGXv4b79GaMYOTuCGLhzqrO3Jw=...,b'ijrnlZOKYXHQ-_YEN84BD66TIXA9qK9YOhZ2rzVM33g=...,b'dvD9UiWPefqPqmLBkaHcnKVI2AkU0yat-MWCXBbV5HY=...,b'amZas7P4RguV7fhchUiInz6ptmIU9fUgfI7ffyxSyJc=...,b'SvGwk9y1zXolFtEeRsql0T3o-6QUl-KPDZRFzJvaBjw=...,0.050,0.032,0.917,0.6365
4,b'APDsLi1C-5Cs8qMKED1HGGP3_ELt-243bE90jNgCSt8=...,b'uXqeB7Z26UVUMe9lWDn7aJNKt85DJYCGy76zpglG0MQ=...,b'crbCNwpk8otn8PhgtUYtbkj0ZoYJskf-6h79kAgM3cU=...,b'rs0e1220jKHb13-yvX3IROuRhkr39mZbcBCzximkXpc=...,b'G9w5fHXsM5YcrNzMme-ZeMvZGc-YsIIR9WarI8OaeQQ=...,b'yHd4h4VIxdMJvtAys5KVCRTy3fCPF4nWlJ6HiYdzeCU=...,b'Y4DJ1EhgR7YfxiSxQ-g2_70CVO7r4AXjt-MQQOWgraU=...,0.075,0.049,0.876,0.6577
...,...,...,...,...,...,...,...,...,...,...,...
15321,b'ekFkZh6D0fKhtrX9JWTyMWqsxk5Tvkbr9DLlhKefioA=...,b'lT4Z1qmrzbC5YrbugaSrI3sG_Q0MZjTAiUiNoJjGLvU=...,b'waI0mWynnA6486ldfqBhoWyvcrE4SsU0IZOc6Pj_59E=...,b'TOrrZfIXqoSTCDm93udYnQwaP2G9TmhgJQFG0oluX-E=...,b'BzPcf6DTOQI22CxuleVqYkWhXlc1JdrAS0hKzrQctmc=...,b'3DEEiE1Ce3Z2quhmRpHPQBKJDkVmpofZIhjW-RrFnx4=...,b'9HsUPZeESNABVDzysHjjfAtHrYQO6yvDizzAfJLW9JA=...,0.160,0.028,0.812,0.9307
15322,b'rLXMpTb1i59gaFOEkkgvAD3kT5aaOuxnF6fcjsQLLeM=...,b'4FMTpvLJr0R3WpM2f6c67OBAE0xcIECRGMEy1Q3uwi0=...,b'BU2FvsM3asD_-1E6yRCWn-quHT70p5_szt5IYbpnQJM=...,b'rp5SfaPJ0UrnTislTtU2IcLfGZJ9kRVP76zElyqsNzM=...,b'TcPy2tRPgK_q0bXMf8XlIRfaaFA2lsRL_kL3tmjrhnE=...,b'IbvPAn5Li7UTQzX730uKmlQ0Vgl27RUtdDsetgWXz1k=...,b'OrVjJUhI41jBz7EUlszp6A7kJNVv-XzR0OXffF8EERA=...,0.156,0.023,0.821,0.9118
15323,b'BUqL8lAJPC3bZXFcizOEGBBGWUab9SInPbMMPPEio4Y=...,b'WlLkwVXBuCumG2_1FCxpod6eMJTsuUXlInyQVMRZ6bU=...,b'lwK7TJ1IwgVjC_0yrmXQhoGFGlw1i15IvBziVCc52BE=...,b'GgfoxDEsvLBh5UVTv0Yas3spzazV2omdqO8dMLKC5fI=...,b'xPFqI9ISYLyvitmkvD_tn8F9zjnz5WegxBpKGonKH-s=...,b'6jHw_5PSh4HSubF4cqPEVKwgybGuwp92G57lMQlQlEk=...,b'zm1hG9ivgNhIKXYRZoYj2rERK7MYxbak8EzYEJnJ6hM=...,0.129,0.000,0.871,0.8834


In [97]:
crypt_articles.loc[:, 'article'] = crypt_articles.article.apply(cryptor.decrypt).apply(str, encoding='utf-8').apply(lambda x: x.replace('  ', '').replace('\x92', "'")).apply(bytes, encoding='utf-8').apply(cryptor.encrypt)

In [98]:
crypt_articles

,article,headline,comments,date,link,symbol,publisher,pos_sent,neg_sent,neu_sent,comp_sent
pk,,,,,,,,,,,
0,b'irJmPVY100-TVuNIsOyq99aC9s9oFX4l1Snzs47oW60=...,b'Jqc6cDZhIcu6aNfkqyzfELn7KxIpIge0W1jApWaJ_iQ=...,b'YlLzFOG-Xec6-61v5jY0E2pD1Eo572iW8WYx99XeBfE=...,b'CuGllAvrxyYSr0-fB0EuU07l-QRZIQEdbewugAs0kgM=...,b'qaEtC-ufuGelB1bHBC1wVj_gVPxagArmLAVwkzeRc1A=...,b'2N7lFkxliIikw5YxhCE-xZZuoOBcsNzZ7w_8q8WJh24=...,b'iEqM7yp5q1dD9E5ffh-q3I8i3Ot9RN-9TW6mHdb7Hik=...,0.111,0.029,0.861,0.9325
1,b'dm4lZQAFo6mpCER-TQjfKvTtxI7FC1fuvtnETRlAnhw=...,b'MmssHZw5N6QiPfWPe_1sS53s4vLgtpnmul7xh63617I=...,b'C--kMS-JiqtjT9irkZM9ArUCXRICre3raTElsaqfKyk=...,b'Ol_501yn3fCC0QCcOjLUtnrO8Ix-BXy-z1BmDtI7Agk=...,b'CRZ4uDkFkfGBUd2un4QtzL4qbgOgdQmdHjzrg85uSLI=...,b'e4_N0A8W-YST2-PPh5bPDojLjDRVgx6LswFDTfg8mqQ=...,b'4vqaOCcT1K6yPtnKEj_35QcyK6PO8fdhgTtFFac61NQ=...,0.073,0.024,0.904,0.8360
2,b'-WmOUAHApUlh3GC982zwTG8TJChUnK4BZf0y7yR35ds=...,b'dSI3TD_qWPddyrgY1lLKfq6GPKHo-nUIWsbmHAVCl8U=...,b'fNJoAgeOc0YqoVRXEX0Wt4DuTTb2UvojOG6nPDWBA0w=...,b'P-P5di0wLHphlX_MkXnaacCaJPVY4ggKxc7aX_hov3s=...,b'0U7mMfkmVgtoosoO_VZukRA0K7JWnaG42jEQnytwT_E=...,b'vUme-p7pQwYUp4k4lnwqD3HeJLWFql4XMGgWxt5NGXI=...,b'3iYHSCuqjIl1c_e9PG8yPpjtvxhIF5s9huWS5QE-WT4=...,0.070,0.051,0.879,0.5106
3,b'9LSNvfqF7Q32Mpbdtl5FnxunGhycKQQ7-KsgipJKoTw=...,b'hG9-8gAJzpPFcJoriTRXfGtkd9iLHqUDJEHR9qBdY8o=...,b'zyelglAChrb2pGJBkxGXv4b79GaMYOTuCGLhzqrO3Jw=...,b'ijrnlZOKYXHQ-_YEN84BD66TIXA9qK9YOhZ2rzVM33g=...,b'dvD9UiWPefqPqmLBkaHcnKVI2AkU0yat-MWCXBbV5HY=...,b'amZas7P4RguV7fhchUiInz6ptmIU9fUgfI7ffyxSyJc=...,b'SvGwk9y1zXolFtEeRsql0T3o-6QUl-KPDZRFzJvaBjw=...,0.050,0.032,0.917,0.6365
4,b'tZhuCVTVSuIkM2LEoxRM0zbCfwCrFJbyu7aNbX7spjM=...,b'uXqeB7Z26UVUMe9lWDn7aJNKt85DJYCGy76zpglG0MQ=...,b'crbCNwpk8otn8PhgtUYtbkj0ZoYJskf-6h79kAgM3cU=...,b'rs0e1220jKHb13-yvX3IROuRhkr39mZbcBCzximkXpc=...,b'G9w5fHXsM5YcrNzMme-ZeMvZGc-YsIIR9WarI8OaeQQ=...,b'yHd4h4VIxdMJvtAys5KVCRTy3fCPF4nWlJ6HiYdzeCU=...,b'Y4DJ1EhgR7YfxiSxQ-g2_70CVO7r4AXjt-MQQOWgraU=...,0.075,0.049,0.876,0.6577
...,...,...,...,...,...,...,...,...,...,...,...
15321,b'L3Yd-S0WDzU1bHOnrlNMO5-vfp1mBniXdZKgwMbZlrs=...,b'lT4Z1qmrzbC5YrbugaSrI3sG_Q0MZjTAiUiNoJjGLvU=...,b'waI0mWynnA6486ldfqBhoWyvcrE4SsU0IZOc6Pj_59E=...,b'TOrrZfIXqoSTCDm93udYnQwaP2G9TmhgJQFG0oluX-E=...,b'BzPcf6DTOQI22CxuleVqYkWhXlc1JdrAS0hKzrQctmc=...,b'3DEEiE1Ce3Z2quhmRpHPQBKJDkVmpofZIhjW-RrFnx4=...,b'9HsUPZeESNABVDzysHjjfAtHrYQO6yvDizzAfJLW9JA=...,0.160,0.028,0.812,0.9307
15322,b'fau0eli751lInoJjPq9tJsdK6MVjBhnuLjKCfknx288=...,b'4FMTpvLJr0R3WpM2f6c67OBAE0xcIECRGMEy1Q3uwi0=...,b'BU2FvsM3asD_-1E6yRCWn-quHT70p5_szt5IYbpnQJM=...,b'rp5SfaPJ0UrnTislTtU2IcLfGZJ9kRVP76zElyqsNzM=...,b'TcPy2tRPgK_q0bXMf8XlIRfaaFA2lsRL_kL3tmjrhnE=...,b'IbvPAn5Li7UTQzX730uKmlQ0Vgl27RUtdDsetgWXz1k=...,b'OrVjJUhI41jBz7EUlszp6A7kJNVv-XzR0OXffF8EERA=...,0.156,0.023,0.821,0.9118
15323,b'ZXkbDgPSamX8tXBifISZRHlTt4zOw6eD3evGj8R6GGQ=...,b'WlLkwVXBuCumG2_1FCxpod6eMJTsuUXlInyQVMRZ6bU=...,b'lwK7TJ1IwgVjC_0yrmXQhoGFGlw1i15IvBziVCc52BE=...,b'GgfoxDEsvLBh5UVTv0Yas3spzazV2omdqO8dMLKC5fI=...,b'xPFqI9ISYLyvitmkvD_tn8F9zjnz5WegxBpKGonKH-s=...,b'6jHw_5PSh4HSubF4cqPEVKwgybGuwp92G57lMQlQlEk=...,b'zm1hG9ivgNhIKXYRZoYj2rERK7MYxbak8EzYEJnJ6hM=...,0.129,0.000,0.871,0.8834


In [102]:
with sql.connect('../data/raw/crypt.db') as con:
    crypt_articles.to_sql("crypt_articles", index=True,index_label='pk', con=con, if_exists='replace')

In [104]:
with sql.connect('../data/interim/companies.db') as con:
    articles.to_sql("articles", index_label="pk", con=con, index=True, if_exists='replace')

In [16]:
with sql.connect("../data/interim/companies.db") as con:
    arts = pd.read_sql(f"SELECT * from articles", con=con, parse_dates={'date': '%Y-%m-%d %H:%M:%S'}).drop('id', axis=1)

with sql.connect("../data/raw/crypt.db") as con:
    second_arts = pd.read_sql('SELECT pk, headline as title, link, date, publisher, symbol, article, pos_sent, neg_sent, neu_sent, comp_sent FROM crypt_articles;', con=con)#.applymap(cryptor.decrypt)
    decrypt_cols = [x for x in second_arts.columns if x not in ['pk', 'pos_sent', "neu_sent", "neg_sent", "comp_sent"]]
    for col in decrypt_cols:
        second_arts.loc[:, col] = second_arts.loc[:, col].apply(bytes).apply(cryptor.decrypt).apply(str, encoding='utf-8')
        if col == 'date':
            second_arts.loc[:, col] = second_arts.loc[:, col].apply(str.split, sep=" ").apply(lambda x: x[0]).apply(pd.to_datetime)

arts = pd.concat([arts, second_arts], axis=0, ignore_index=True).sort_values('date')        

In [19]:
arts.groupby('symbol').agg(['first', 'last', 'count']).date.sort_values('first', ascending=False)

,first,last,count
symbol,,,
CRSR,2020-09-14,2021-08-31,39
RSI,2020-07-27,2021-08-30,41
VLDR,2020-06-27,2021-08-31,99
HYLN,2020-06-19,2021-08-31,65
PT,2020-06-15,2021-08-31,15
...,...,...,...
HEAR,2018-11-07,2021-11-29,61
SOLO,2018-10-29,2021-12-14,80
XERS,2018-10-23,2021-12-31,58


In [37]:
with sql.connect("../data/interim/companies.db") as con:
    mentions = pd.read_sql(f"SELECT * from mentions", con=con, index_col='pk')

with sql.connect("../data/interim/discord/discord.db") as con:
    comments = pd.read_sql(f"SELECT * from comments", con=con, index_col='pk')

In [59]:
mentioned_companies = mentions.symbol.values[:100]

In [72]:
mentions.iloc[:100, :]

,symbol,counts
pk,,
0,SPY,167
1,TSLA,166
2,NIO,151
3,AMD,133
4,AAPL,114
...,...,...
95,SOLO,15
96,XERS,15
97,DPW,14


In [124]:
company_regex = re.compile(r"|".join([x for x in mentioned_companies]))

In [148]:
value_comments = comments[lambda x: x.content.str.contains(company_regex)]

In [149]:
value_comments.loc[:, 'timestamp'] = pd.to_datetime(comments[lambda x: x.content.str.contains(company_regex)].timestamp)

C:\Users\Kyle\anaconda3\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [150]:
value_comments = value_comments[lambda x: (x.isBot==0)].reset_index().assign(symbols=lambda x: x.content.apply(lambda s: re.findall(company_regex, s)))

In [151]:
value_comments=value_comments.assign(symbols=lambda x: x.symbols.apply(str))

In [152]:
value_comments = value_comments.assign(content_cleaned = lambda x: x.content.apply(lambda s: re.sub(re.compile(r"<.+>|@everyone"), "", s)))

In [153]:
value_comments=value_comments.assign(compound_sent = lambda x: x.content_cleaned.apply(sentiment_art))

In [154]:
value_comments=value_comments.assign(pos_sent = lambda x: x.compound_sent.apply(lambda y: y['pos']))\
    .assign(neg_sent = lambda x: x.compound_sent.apply(lambda y: y['neg']))\
        .assign(neu_sent = lambda x: x.compound_sent.apply(lambda y: y['neu']))\
            .assign(comp_sent = lambda x: x.compound_sent.apply(lambda y: y['compound']))

In [155]:
value_comments=value_comments.drop(["compound_sent", "content_cleaned"], axis=1)

In [156]:
with sql.connect("../data/interim/companies.db") as con:
    value_comments.to_sql('symbol_comments', con=con,index=False, if_exists='replace')

In [157]:
with sql.connect("../data/interim/companies.db") as con:
    info = pd.read_sql(f"SELECT * from info", con=con, index_col='pk')

In [175]:
info.loc[info['sector']=='Communication Services', 'sector'] = "Communications Services"

In [162]:
comps = info[lambda v: ((v.quoteType=='ETF') & (v.shortName.str.contains('Vanguard')) | (v.quoteType=='ETF') & (v.shortName.str.contains('SPDR')))].loc[:, ['shortName', 'quoteType', 'sector', 'symbol']]

In [178]:
info[lambda v: (v.quoteType=='EQUITY')].loc[:, ['shortName', 'quoteType', 'sector', 'symbol']].merge(comps, on='sector', how='left')

,shortName_x,quoteType_x,sector,symbol_x,shortName_y,quoteType_y,symbol_y
0,"Tesla, Inc.",EQUITY,Consumer Discretionary,TSLA,Vanguard Consumer Discretion ET,ETF,VCR
1,NIO Inc.,EQUITY,Consumer Discretionary,NIO,Vanguard Consumer Discretion ET,ETF,VCR
2,"Advanced Micro Devices, Inc.",EQUITY,Technology,AMD,Vanguard Information Tech ETF,ETF,VGT
3,Apple Inc.,EQUITY,Technology,AAPL,Vanguard Information Tech ETF,ETF,VGT
4,DraftKings Inc.,EQUITY,Consumer Discretionary,DKNG,Vanguard Consumer Discretion ET,ETF,VCR
...,...,...,...,...,...,...,...
67,Electrameccanica Vehicles Corp.,EQUITY,Consumer Discretionary,SOLO,Vanguard Consumer Discretion ET,ETF,VCR
68,"Xeris Biopharma Holdings, Inc.",EQUITY,Healthcare,XERS,Vanguard Health Care ETF,ETF,VHT
69,"Ault Global Holdings, Inc.",EQUITY,Financials,DPW,Vanguard Financials ETF,ETF,VFH
70,"Gilead Sciences, Inc.",EQUITY,Healthcare,GILD,Vanguard Health Care ETF,ETF,VHT


In [169]:
comps

,shortName,quoteType,sector,symbol
pk,,,,
74,SPDR S&P 500,ETF,Mutual Fund,SPY
77,Vanguard Communication Services,ETF,Communications Services,VOX
78,Vanguard Consumer Discretion ET,ETF,Consumer Discretionary,VCR
79,Vanguard Consumer Staples ETF,ETF,Consumer Staples,VDC
80,Vanguard Energy ETF,ETF,Energy,VDE
81,Vanguard Financials ETF,ETF,Financials,VFH
82,Vanguard Health Care ETF,ETF,Healthcare,VHT
83,Vanguard Industrials ETF,ETF,Industrials,VIS
84,Vanguard Information Tech ETF,ETF,Technology,VGT


In [182]:
with sql.connect("../data/interim/companies.db") as con:
    info.reset_index(drop=True).to_sql("info", con=con, index_label='pk', index=True, if_exists='replace')

In [205]:
# pull equity from info table with associated GICS Sector VIA JOIN:
with sql.connect("../data/interim/companies.db") as con:
    reader = pd.read_sql("SELECT * FROM info JOIN (SELECT shortName compName, sector, symbol compSymbol FROM info WHERE compName LIKE '%Vanguard%' OR '%SPDR%') USING (sector) WHERE symbol='TSLA'", con=con, index_col='pk')

In [206]:
reader

,quoteType,longName,shortName,logo_url,sector,industry,holdings,fundFamily,category,symbol,compName,compSymbol
pk,,,,,,,,,,,,
0,EQUITY,"Tesla, Inc.","Tesla, Inc.",https://logo.clearbit.com/tesla.com,Consumer Discretionary,Auto Manufacturers,None,,,TSLA,Vanguard Consumer Discretion ET,VCR
